<a href="https://colab.research.google.com/github/417godmin/NER-/blob/main/NER%E6%97%B6%E9%97%B4%E5%9C%B0%E5%9D%80%E8%AF%86%E5%88%AB%E5%8A%A8%E6%80%81%E5%AD%A6%E4%B9%A0%E6%A1%86%E6%9E%B6%E6%A8%A1%E5%9E%8B%E8%AE%AD%E7%BB%83%E4%B8%8E%E6%B5%8B%E8%AF%95_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **谷歌云盘**
挂载谷歌云盘，便于通过代码保存和加载模型文件

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 构造数据集

完善中......


#### 调用大模型接口生成自然语言模板





###### 说明

每次运行为文件新增数据而非重写(已测试)

输入输出说明：

*  不需要输入

*  输出：/content/drive/MyDrive/NER/address_templates/address_templates_{地址数*（1/2/3/4/5）}/{语言名（中文）}_address_templates.jsonl

*  输出结构
```
/content
└── drive
    └── MyDrive
        └── NER
            ├── address_templates_0(整句纯负例)
                ├── 简体中文_address_templates.jsonl
                ├── 繁体中文_address_templates.jsonl
                 ......
                └—— 韩语_address_templates.jsonl
            ├── address_templates_1
                ├── 简体中文_address_templates.jsonl
                ├── 繁体中文_address_templates.jsonl
                 ......
                └—— 韩语_address_templates.jsonl
            ├── address_templates_2          
            ├── address_templates_3
            ├── address_templates_4
            └── address_templates_5
```

##### 生成训练模版

纯地址模版生成器


In [ ]:
# @title {"form-width":"35%"}
llm_api_key = ""

# 依赖：pip install httpx

import os
import re
import json
import random
import asyncio
import platform
from typing import List, Dict, Any, Optional
import httpx
from google.colab import drive

# ======================================================可按需修改的配置 ======================================================

# 生成目标
ADDRESS_SLOT_LIST   = [0, 1, 2, 3, 4, 5]     # 需要生成的“地址占位符个数”集合（最多到 5）
TARGET_PER_SLOT     = [1000, 2000, 2000, 1500, 500, 100]  # 与上面一一对应，每种占位数下，每语言目标条数
#TARGET_PER_SLOT     = [2,2,2,2,2,2]  # 测试


api_key = "sk-013SeYkFFCjw1G3IIwDWfj2iXFqWhUSF6fmC0L7KdJjE"  # @param {"type":"string","placeholder":"请输入大模型api key"}

TEMPLATE_BATCH_SIZE = 40            # @param {"type":"integer","placeholder":"一次让大模型产出n条"}（建议50以下，过大会增大 JSON 不规范风险）
TEMPERATURE = 0.8               # @param {"type":"number","placeholder":"模型温度"} 越高输出越多样，越低越遵循指令
MODEL_NAME = "turing/gpt-4o"         # @param {"type":"number","placeholder":"模型名"}（如需更换请保持模型名和服务器平台上的一致）

# =================================================================================================================================


# 确保挂载google drive
try:
    import google.colab
    from google.colab import drive  # 在 Colab 环境下可用
    drive.mount('/content/drive')
    print("云盘挂载成功")
except Exception:
    print("云盘挂载失败")


# 保存到drive
OUTPUT_BASE_DIR = "/content/drive/MyDrive/NER/address_templates"
SLOT2TARGET = dict(zip(ADDRESS_SLOT_LIST, TARGET_PER_SLOT))


# ================== 接口配置 ==================
BASE_URL = os.getenv("LLM_BASE_URL", "https://test-turing.cn.llm.tcljd.com")
ENDPOINT = os.getenv("LLM_ENDPOINT", "/api/v1/chat/completions")
API_KEY  = os.getenv("LLM_API_KEY", api_key)

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
}

# ================== 生成配置 ==================
LANGUAGES = [
    "简体中文", "繁体中文", "英语", "丹麦语", "俄语", "土耳其语", "德语",
    "意大利语", "日语", "法语", "瑞典语", "荷兰语", "葡萄牙语",
    "西班牙语", "韩语"
]


TIMEOUT = 90                    # 单次请求超时
RETRY_TIMES = 2                   # 每批重试次数

#  HTTP 客户端（全局）
client: Optional[httpx.AsyncClient] = None


# 根据地址数量选择prompt中的示例
prompt_example1 = [
    '{"template": "这里是一条模板文本，包含{address}。"}',
    '{"template": "这里是一条模板文本，包含{address}和{address}。"}',
    '{"template": "这里是一条模板文本，包含{address}和{address}，还包含{address}。"}',
    '{"template": "这里是一条模板文本，包含{address}和{address}。以及{address}，还有{address}"}',
    '{"template": "这里是一条模板文本，包含{address}和{address}以及{address}。还有{address}和{address}"}',
]

prompt_example2 = [
      """
      [
        {{"template": "请到{address}办理入场登记。"}},
        {{"template": "Meet me at {address}, don't be late."}}
      ]
      """,
      """
      [
        {{"template": "请到{address}，再到{address}领取胸卡。"}},
        {{"template": "Meet me at {address}, don't be late. We can have lunch at {address} in the afternoon."}}
      ]
      """,
      """
      [
        {{"template": "{address}或{address}办理入场登记，再到{address}领取胸卡。"}},
        {{"template": "Meet me at {address}, don't be late. We can have lunch at {address} or {address}."}}
      ]
      """,
      """
      [
        {{"template": "请到{address}办理入场登记。再到{address}领取胸卡，在{address}办公。如果还有问题可以来{address}找我。"}},
        {{"template": "Meet me at {address}, don't be late. We can have lunch at {address} or {address} in the afternoon. In the evening, we might visit {address} for some fun."}}
      ]
      """,
      """
      [
        {{"template": "请到{address}或{address}办理入场登记。再到{address}领取胸卡，在{address}办公。如果还有问题可以来{address}找我。"}},
        {{"template": "Meet me at {address}, don't be late. We can have lunch at {address} or {address} in the afternoon. In the evening, we might visit {{address}} or {{address}}for some fun."}}
      ]
      """,

]

# ================== 工具函数 ==================
def build_template_prompt(language: str, n: int, n_address: int) -> str:
    if n_address == 0:
      return f"""
          你是一个不包含地址信息的剪贴板文本生成器。请用 **{language}** 语言生成 {n} 条“不包含地址的剪贴板文本”：
              - 模板里严禁出现任何地址
              - 风格多样：口语、正式通知、聊天记录、评论、邮件、日程卡片、社交帖子等，长度 1–3 句
              - 必须输出“纯 JSON 数组”，不要代码块、不要多余说明
              - JSON 文本中不要包含换行符 \\n

              示例（注意：这是示意，不要照抄，也不要输出示例之外任何文字）：
              [
                {{"template": "下午请找我办理入场登记。"}},
                {{"template": "Let's play games together tomorrow morning."}}
              ]

              现在请直接输出 **{n} 条**该语言的剪贴板文本 JSON 数组。"""

    else:
      return f"""你是一个模板生成器。请用 **{language}** 语言生成 {n} 条“上下文模板”，
              每条模板必须包含{n_address}个占位符：
              - 占位符：{{address}}（必须原样出现，不加引号）
              - 模板里严禁出现任何地址（只能用占位符）
              - 风格多样：口语、正式通知、聊天记录、评论、邮件、日程卡片、社交帖子等，长度 1–3 句
              - 必须输出“纯 JSON 数组”，不要代码块、不要多余说明，数组元素为：
                {prompt_example1[n_address-1]}
              - JSON 文本中不要包含换行符 \\n

              示例（注意：这是示意，不要照抄，也不要输出示例之外任何文字）：
              {prompt_example2[n_address-1]}
              现在请直接输出 **{n} 条**该语言的模板 JSON 数组。"""




def _strip_code_fence(s: str) -> str:
    """去除可能的Markdown代码块围栏，并截取第一个看起来像JSON数组的片段"""
    txt = s.strip()
    # 如果有 ``` 包裹，尝试找到第一个 [ 开始位置
    if txt.startswith("```"):
        i = txt.find("[")
        if i == -1:
            i = txt.find("{")
        if i != -1:
            txt = txt[i:]
        if txt.endswith("```"):
            txt = txt[:-3].strip()
    return txt

def _parse_json_array(text: str) -> List[Dict[str, Any]]:
    """尽量宽松地把返回内容解析为 JSON 数组"""
    txt = _strip_code_fence(text)

    # 先尝试直接解析
    try:
        data = json.loads(txt)
        if isinstance(data, list):
            return data
    except Exception:
        pass

    # 再尝试正则抽取首个JSON数组
    m = re.search(r"\[[\s\S]*\]", txt)
    if m:
        arr = m.group(0)
        return json.loads(arr)

    raise ValueError(f"无法解析为 JSON 数组，片段：{txt[:200]}...")

# 过滤生成的模板
def _post_process_templates(json_list: List[Dict[str, Any]], need: int, n_address: int) -> List[str]:
    out, seen = [], set()
    for item in json_list:
        if not isinstance(item, dict):
            continue
        t = item.get("template")
        if not isinstance(t, str):
            continue
        t = " ".join(t.strip().split())
        if not t or t in seen:
            continue

        if n_address == 0:
            if "{address}" in t:
                continue
        else:
            # 占位符数量必须符合要求
            if t.count("{address}") != n_address:
                continue

        out.append(t)
        seen.add(t)

    # 返回
    return out[:need]



async def chat_stream(payload: dict, *, collect: bool = True) -> str:
    """
    流式接口封装：
    - collect=True：收集完整响应文本并返回（适合后续 JSON 解析）
    """
    assert client is not None, "client 尚未初始化"
    text_buf: List[str] = []
    async with client.stream("POST", ENDPOINT, headers=HEADERS, content=json.dumps(payload)) as resp:
        async for line in resp.aiter_lines():
            if not line:
                continue
            line = line.strip()
            # 你的服务返回格式若与 OpenAI SSE 一致，形如：data: {...}
            if not line.startswith("data:"):
                continue
            data = line[5:].strip()
            if data == "[DONE]":
                break
            try:
                delta = json.loads(data)["choices"][0]["delta"].get("content", "")
            except Exception:
                continue
            if collect:
                text_buf.append(delta)
    return "".join(text_buf)

async def generate_templates_batch(language: str, n: int, n_address: int) -> List[str]:
    """
    生成该语言的 n 条模板；若 LLM 输出异常，抛出异常由上层处理重试。
    """
    prompt = build_template_prompt(language, n, n_address)
    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "stream": True,
        "temperature": TEMPERATURE,
    }
    raw = await chat_stream(payload, collect=True)
    data = _parse_json_array(raw)
    return _post_process_templates(data, n, n_address)


async def generate_templates_for_language(language: str, slot: int) -> str:
    """
    为不同地址数模板生成子目录
    为某个 language 生成 TEMPLATES_PER_LANG 条模板，
    按 TEMPLATE_BATCH_SIZE 分批请求，并输出到 templates_both/<language>_template.jsonl
    """
    target_per_lang = SLOT2TARGET.get(slot, 0)
    if target_per_lang <= 0:
        return ""

    out_dir = os.path.join(OUTPUT_BASE_DIR, f"address_templates_{slot}")
    os.makedirs(out_dir, exist_ok=True)

    out_path = os.path.join(out_dir, f"{language}_address_templates.jsonl")


    # 统计现有条数，续写
    existing = 0
    if os.path.exists(out_path):
        with open(out_path, "r", encoding="utf-8") as f:
            existing = sum(1 for _ in f)


    target = target_per_lang
    if existing >= target:
        print(f"[{language}] 已有 {existing} 条，已达到目标 {target}，跳过。→ {out_path}")
        return out_path


    bs = TEMPLATE_BATCH_SIZE
    total = existing
    print(f"[{language}] 目标 {target} 条，已存在 {existing} 条，需新增 {target - total} 条；batch={bs} → {out_path}")

    ROUND_CAP = 50          # 安全上限：最多打 50 轮
    EMPTY_STREAK_CAP = 3    # 连续空批最多 3 次就停，提示检查 prompt
    rounds_done = 0
    empty_streak = 0

    while total < target and rounds_done < ROUND_CAP:
        ask_n = min(bs, target - total)   # 也可以固定用 bs，看你偏好
        last_err = None

        for attempt in range(1, RETRY_TIMES + 2):
            try:
                batch = await generate_templates_batch(language, ask_n, slot)
                break
            except Exception as e:
                last_err = e
                print(f"[{language}] 第 {rounds_done+1} 轮，第 {attempt} 次尝试失败：{e}")
                await asyncio.sleep(1.2 * attempt)
        else:
            raise RuntimeError(f"[{language}] 连续失败：{last_err}")

        got = len(batch)
        if got == 0:
            empty_streak += 1
            print(f"[{language}] 空批（第 {empty_streak} 次），总进度仍为 {total}/{target}")
            if empty_streak >= EMPTY_STREAK_CAP:
                print(f"[{language}] 连续空批达到 {EMPTY_STREAK_CAP}，提前停止，请检查提示词或模型输出。")
                break
        else:
            empty_streak = 0
            with open(out_path, "a", encoding="utf-8") as f:
                for t in batch:
                    rec = {"language": language, "template": t}
                    f.write(json.dumps(rec, ensure_ascii=False) + "\n")
            total += got
            print(f"[{language}] 进度：{total}/{target} (+{got})")

        rounds_done += 1

    if total < target:
        print(f"[{language}] 未达标：{total}/{target}。可提高 TEMPLATE_BATCH_SIZE 或改写提示词。")
    else:
        print(f"[{language}] 完成，合计 {total} 条 → {out_path}")

    return out_path

async def generate_templates_for_all_languages():
    """
    依次为不同地址数的每个 LANGUAGES 生成模板文件。
    """
    for slot in ADDRESS_SLOT_LIST:
        target = SLOT2TARGET.get(slot, 0)
        if target <= 0:
            print(f"[addr={slot}] 目标条数为 0，跳过。")
            continue
        print("=" * 12, f"开始：address数量： {slot}（每语言目标 {target} 条）", "=" * 12)
        for lang in LANGUAGES:
            await generate_templates_for_language(lang, slot)



# ================== 主程序 ==================
async def main():
    global client
    client = httpx.AsyncClient(base_url=BASE_URL, timeout=TIMEOUT)
    try:
        print(f"已连接到模型服务：{BASE_URL}{ENDPOINT} | model={MODEL_NAME}")
        print(f"输出根目录：{OUTPUT_BASE_DIR}")
        await generate_templates_for_all_languages()
        print("\n" + "=" * 50)
        print("全部模板生成完成！目录按地址占位数区分：address_templates_<slot>")
        print("文件命名：<语言>_address_templates.jsonl（每行一条 JSON）")
        print("=" * 50)
    finally:
        await client.aclose()


await main()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
云盘挂载成功
已连接到模型服务：https://test-turing.cn.llm.tcljd.com/api/v1/chat/completions | model=turing/gpt-4o
输出根目录：/content/drive/MyDrive/NER/address_templates
============ 开始：address数量： 0（每语言目标 1000 条） ============
[简体中文] 目标 1000 条，已存在 100 条，需新增 900 条；batch=40 → /content/drive/MyDrive/NER/address_templates/address_templates_0/简体中文_address_templates.jsonl
[简体中文] 进度：140/1000 (+40)
[简体中文] 进度：180/1000 (+40)
[简体中文] 进度：220/1000 (+40)
[简体中文] 进度：259/1000 (+39)
[简体中文] 进度：299/1000 (+40)
[简体中文] 进度：339/1000 (+40)
[简体中文] 进度：379/1000 (+40)
[简体中文] 进度：419/1000 (+40)
[简体中文] 进度：459/1000 (+40)
[简体中文] 进度：499/1000 (+40)
[简体中文] 进度：539/1000 (+40)
[简体中文] 进度：579/1000 (+40)
[简体中文] 进度：619/1000 (+40)
[简体中文] 进度：659/1000 (+40)
[简体中文] 进度：699/1000 (+40)
[简体中文] 进度：739/1000 (+40)
[简体中文] 进度：779/1000 (+40)
[简体中文] 进度：819/1000 (+40)
[简体中文] 进度：859/1000 (+40)
[简体中文] 进度：899/1000 (+40)
[简体中文] 进度：939/1000 (+40

##### 生成测试用负例（纯地址）

In [ ]:
import os

# 输出目录
output_dir = "/content/drive/MyDrive/NER/non_address"
os.makedirs(output_dir, exist_ok=True)

# 各语言常见楼栋表达
translations = {
    "简体中文": "xx栋",
    "繁体中文": "xx棟",
    "英语": "Building xx",
    "丹麦语": "Bygning xx",
    "俄语": "Здание xx",
    "土耳其语": "xx Blok",
    "德语": "Gebäude xx",
    "意大利语": "Edificio xx",
    "日语": "xx棟",
    "法语": "Bâtiment xx",
    "瑞典语": "Byggnad xx",
    "荷兰语": "Gebouw xx",
    "葡萄牙语": "Edifício xx",
    "西班牙语": "Edificio xx",
    "韩语": "xx동",
}

def generate_files():
    for lang, phrase in translations.items():
        filepath = os.path.join(output_dir, f"{lang}.txt")
        with open(filepath, "w", encoding="utf-8") as f:
            # 原有楼栋负例
            for i in range(1, 50):
                f.write(phrase.replace("xx", str(i)) + "\n")

            # 新增 xx# 样式（专门避免 16#、23# 这种被识别）
            for i in range(1, 50):
                f.write(f"{i}#\n")
        print(f"生成文件: {filepath}")

if __name__ == "__main__":
    generate_files()

生成文件: /content/drive/MyDrive/NER/non_address/简体中文.txt
生成文件: /content/drive/MyDrive/NER/non_address/繁体中文.txt
生成文件: /content/drive/MyDrive/NER/non_address/英语.txt
生成文件: /content/drive/MyDrive/NER/non_address/丹麦语.txt
生成文件: /content/drive/MyDrive/NER/non_address/俄语.txt
生成文件: /content/drive/MyDrive/NER/non_address/土耳其语.txt
生成文件: /content/drive/MyDrive/NER/non_address/德语.txt
生成文件: /content/drive/MyDrive/NER/non_address/意大利语.txt
生成文件: /content/drive/MyDrive/NER/non_address/日语.txt
生成文件: /content/drive/MyDrive/NER/non_address/法语.txt
生成文件: /content/drive/MyDrive/NER/non_address/瑞典语.txt
生成文件: /content/drive/MyDrive/NER/non_address/荷兰语.txt
生成文件: /content/drive/MyDrive/NER/non_address/葡萄牙语.txt
生成文件: /content/drive/MyDrive/NER/non_address/西班牙语.txt
生成文件: /content/drive/MyDrive/NER/non_address/韩语.txt


##### 纯时间模版生成器
参照纯地址模版生成器编写的时间模版生成代码，在prompt中示例内容进行了结构性优化，同时降采样到地址模板的四分之一（考虑到时间模板自身的结构多样性有限）

In [ ]:
# @title 纯时间模板生成器 {"form-width":"35%"}
# 依赖：pip install httpx

import os
import re
import json
import random
import asyncio
from typing import List, Dict, Any, Optional
import httpx

# ======================================================可按需修改的配置 ======================================================

# 原地址模板目标
ADDRESS_SLOT_LIST   = [0, 1, 2, 3, 4, 5]
TARGET_PER_SLOT     = [1000, 2000, 2000, 1500, 500, 100]

# 时间模板目标 = 地址目标的 1/4，只保留 0-3 slot
TIME_SLOT_LIST      = [0, 1, 2, 3]
TARGET_PER_SLOT_TIME = [int(x/4) for x in TARGET_PER_SLOT[:len(TIME_SLOT_LIST)]]

api_key = "sk-013SeYkFFCjw1G3IIwDWfj2iXFqWhUSF6fmC0L7KdJjE"  # @param {"type":"string","placeholder":"请输入大模型api key"}

TEMPLATE_BATCH_SIZE = 40
TEMPERATURE = 0.8
MODEL_NAME = "turing/gpt-4o"   # 你的模型名称

# 输出路径（独立于地址模板）
OUTPUT_BASE_DIR = "/content/drive/MyDrive/NER/time_templates"
SLOT2TARGET = dict(zip(TIME_SLOT_LIST, TARGET_PER_SLOT_TIME))

# ================== 接口配置 ==================
BASE_URL = os.getenv("LLM_BASE_URL", "https://test-turing.cn.llm.tcljd.com")
ENDPOINT = os.getenv("LLM_ENDPOINT", "/api/v1/chat/completions")
API_KEY  = os.getenv("LLM_API_KEY", api_key)

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
}

LANGUAGES = [
    "简体中文", "繁体中文", "英语", "丹麦语", "俄语", "土耳其语", "德语",
    "意大利语", "日语", "法语", "瑞典语", "荷兰语", "葡萄牙语",
    "西班牙语", "韩语"
]

TIMEOUT = 90
RETRY_TIMES = 2
client: Optional[httpx.AsyncClient] = None

# ================== 示例（重写高质量时间模板示例） ==================
# prompt_example1_time 用于告诉模型“每条模板只有占位符，风格多样”
# 索引 n_time-1，对应 slot = 1, 2, 3
prompt_example1_time = [
    '{"template": "这里是一条模板文本，包含{time}。"}',
    '{"template": "这里是一条模板文本，包含{time}和{time}。"}',
    '{"template": "这里是一条模板文本，包含{time}、{time}和{time}。"}',
]

# prompt_example2_time：每个slot对应一个中英示例数组，全部时间用占位符，不出现真实时间词
prompt_example2_time = [
    # slot=0（纯负例，不含任何时间概念词）
    """
    [
      {"template": "这个方案我们需要进一步完善后再提交。"},
      {"template": "Your proposal looks promising, let's polish it more."}
    ]
    """,
    # slot=1（一个 {time} 占位符，中英母语化风格）
    """
    [
      {"template": "请在{time}参加培训会，不要迟到。"},
      {"template": "Let’s meet at {time}, I’ll bring the new designs."}
    ]
    """,
    # slot=2（两个 {time} 占位符，中英母语化风格）
    """
    [
      {"template": "从{time}开始，一直到{time}结束，请全程参与。"},
      {"template": "We’ll start at {time} and finish by {time}, be prepared."}
    ]
    """,
    # slot=3（三个 {time} 占位符，分时段安排）
    """
    [
      {"template": "在{time}召开会议，{time}午餐，{time}继续讨论方案。"},
      {"template": "Kick-off at {time}, lunch at {time}, resume work by {time}."}
    ]
    """
]

# ================== 工具函数（重写后的 build_template_prompt） ==================
def build_template_prompt(language: str, n: int, n_time: int) -> str:
    """
    构造生成时间模板的 Prompt。
    规则：
      - slot=0: 不允许出现任何时间相关的词汇或暗示
      - slot>0: 所有时间类词汇用 {time} 占位符代替
    """
    if n_time == 0:
        # 纯负例，不含时间
        return f"""
            你是一个不包含时间信息的剪贴板文本生成器。请用 **{language}** 语言生成 {n} 条“不包含时间的剪贴板文本”：
                - 模板里严禁出现任何时间相关词汇（包括日期、时刻、星期、上午、下午、中午、夜晚、昨天、今天、明天、稍后、之后等）
                - 风格多样：口语、正式通知、聊天记录、评论、邮件、日程卡片、社交帖子等，长度 1–3 句
                - 必须输出“纯 JSON 数组”，不要代码块、不要多余说明
                - JSON 文本中不要包含换行符 \\n
            示例：
            {prompt_example2_time[0]}
            现在请直接输出 **{n} 条**该语言的剪贴板文本 JSON 数组。
        """
    else:
        # 时间占位符模板
        return f"""
            你是一个模板生成器。请用 **{language}** 语言生成 {n} 条“上下文模板”，
            每条模板必须包含 {n_time} 个占位符：
                - 占位符：{{time}}（必须原样出现，不加引号）
                - 模板里严禁出现任何真实时间相关词汇（上午、下午、中午、夜晚、昨天、今天、明天、日期、时刻等），一律用 {{time}} 占位符替代
                - 风格多样：口语、正式通知、聊天记录、评论、邮件、日程卡片、社交帖子等，长度 1–3 句
                - 必须输出“纯 JSON 数组”，不要代码块、不要多余说明，数组元素为：
                  {prompt_example1_time[n_time-1]}
                - JSON 文本中不要包含换行符 \\n
            示例：
            {prompt_example2_time[n_time]}
            现在请直接输出 **{n} 条**该语言的模板 JSON 数组。
        """

def _strip_code_fence(s: str) -> str:
    txt = s.strip()
    if txt.startswith("```"):
        i = txt.find("[")
        if i == -1: i = txt.find("{")
        if i != -1: txt = txt[i:]
        if txt.endswith("```"):
            txt = txt[:-3].strip()
    return txt

def _parse_json_array(text: str) -> List[Dict[str, Any]]:
    txt = _strip_code_fence(text)
    try:
        data = json.loads(txt)
        if isinstance(data, list):
            return data
    except Exception:
        pass
    m = re.search(r"$$[\s\S]*$$", txt)
    if m:
        arr = m.group(0)
        return json.loads(arr)
    raise ValueError(f"无法解析为 JSON 数组，片段：{txt[:200]}...")

def _post_process_templates(json_list: List[Dict[str, Any]], need: int, n_time: int) -> List[str]:
    out, seen = [], set()
    for item in json_list:
        if not isinstance(item, dict): continue
        t = item.get("template")
        if not isinstance(t, str): continue
        t = " ".join(t.strip().split())
        if not t or t in seen: continue
        if n_time == 0:
            if "{time}" in t:
                continue
        else:
            if t.count("{time}") != n_time:
                continue
        out.append(t)
        seen.add(t)
    return out[:need]

async def chat_stream(payload: dict, *, collect: bool = True) -> str:
    assert client is not None
    text_buf: List[str] = []
    async with client.stream("POST", ENDPOINT, headers=HEADERS, content=json.dumps(payload)) as resp:
        async for line in resp.aiter_lines():
            if not line: continue
            line = line.strip()
            if not line.startswith("data:"): continue
            data = line[5:].strip()
            if data == "[DONE]": break
            try:
                delta = json.loads(data)["choices"][0]["delta"].get("content", "")
            except Exception:
                continue
            if collect:
                text_buf.append(delta)
    return "".join(text_buf)

async def generate_templates_batch(language: str, n: int, n_time: int) -> List[str]:
    prompt = build_template_prompt(language, n, n_time)
    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "stream": True,
        "temperature": TEMPERATURE,
    }
    raw = await chat_stream(payload, collect=True)
    data = _parse_json_array(raw)
    return _post_process_templates(data, n, n_time)

async def generate_templates_for_language(language: str, slot: int) -> str:
    target_per_lang = SLOT2TARGET.get(slot, 0)
    if target_per_lang <= 0: return ""
    out_dir = os.path.join(OUTPUT_BASE_DIR, f"time_templates_{slot}")
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"{language}_time_templates.jsonl")

    existing = 0
    if os.path.exists(out_path):
        with open(out_path, "r", encoding="utf-8") as f:
            existing = sum(1 for _ in f)
    target = target_per_lang
    if existing >= target:
        print(f"[{language}] 已有 {existing} 条，已达到目标 {target}，跳过。→ {out_path}")
        return out_path

    bs, total = TEMPLATE_BATCH_SIZE, existing
    print(f"[{language}] 目标 {target} 条，已有 {existing} 条 → {out_path}")
    ROUND_CAP, EMPTY_STREAK_CAP = 50, 3
    rounds_done, empty_streak = 0, 0

    while total < target and rounds_done < ROUND_CAP:
        ask_n = min(bs, target - total)
        for attempt in range(1, RETRY_TIMES + 2):
            try:
                batch = await generate_templates_batch(language, ask_n, slot)
                break
            except Exception as e:
                print(f"[{language}] 第{rounds_done+1}轮，第{attempt}次失败：{e}")
                await asyncio.sleep(1.2 * attempt)
        else:
            raise RuntimeError(f"[{language}] 连续失败")

        got = len(batch)
        if got == 0:
            empty_streak += 1
            if empty_streak >= EMPTY_STREAK_CAP:
                break
        else:
            empty_streak = 0
            with open(out_path, "a", encoding="utf-8") as f:
                for t in batch:
                    rec = {"language": language, "template": t}
                    f.write(json.dumps(rec, ensure_ascii=False) + "\n")
            total += got
            print(f"[{language}] 进度：{total}/{target} (+{got})")
        rounds_done += 1

    return out_path

async def generate_templates_for_all_languages():
    for slot in TIME_SLOT_LIST:
        target = SLOT2TARGET.get(slot, 0)
        if target <= 0: continue
        print("="*12, f"开始：time数量 {slot}（每语言目标 {target} 条）", "="*12)
        for lang in LANGUAGES:
            await generate_templates_for_language(lang, slot)

# ================== 主程序 ==================
async def main():
    global client
    client = httpx.AsyncClient(base_url=BASE_URL, timeout=TIMEOUT)
    try:
        await generate_templates_for_all_languages()
        print("全部时间模板生成完成！")
    finally:
        await client.aclose()

await main()

============ 开始：time数量 0（每语言目标 250 条） ============
[简体中文] 目标 250 条，已有 0 条 → /content/drive/MyDrive/NER/time_templates/time_templates_0/简体中文_time_templates.jsonl
[简体中文] 进度：40/250 (+40)
[简体中文] 进度：80/250 (+40)
[简体中文] 进度：120/250 (+40)
[简体中文] 进度：160/250 (+40)
[简体中文] 进度：200/250 (+40)
[简体中文] 进度：240/250 (+40)
[简体中文] 进度：250/250 (+10)
[繁体中文] 目标 250 条，已有 0 条 → /content/drive/MyDrive/NER/time_templates/time_templates_0/繁体中文_time_templates.jsonl
[繁体中文] 进度：40/250 (+40)
[繁体中文] 进度：80/250 (+40)
[繁体中文] 进度：120/250 (+40)
[繁体中文] 进度：160/250 (+40)
[繁体中文] 进度：200/250 (+40)
[繁体中文] 进度：240/250 (+40)
[繁体中文] 进度：250/250 (+10)
[英语] 目标 250 条，已有 0 条 → /content/drive/MyDrive/NER/time_templates/time_templates_0/英语_time_templates.jsonl
[英语] 进度：40/250 (+40)
[英语] 进度：80/250 (+40)
[英语] 进度：120/250 (+40)
[英语] 进度：160/250 (+40)
[英语] 进度：200/250 (+40)
[英语] 进度：240/250 (+40)
[英语] 进度：250/250 (+10)
[丹麦语] 目标 250 条，已有 0 条 → /content/drive/MyDrive/NER/time_templates/time_templates_0/丹麦语_time_templates.jsonl
[丹麦语] 进度：39/250 (+39)
[丹麦语] 进度：

##### 生成测试用负例（纯时间）

In [ ]:
import os

# 输出目录
output_dir = "/content/drive/MyDrive/NER/non_time"
os.makedirs(output_dir, exist_ok=True)

# 各语言模糊时间词
translations = {
    "简体中文": "最近",
    "繁体中文": "最近",
    "英语": "soon",
    "丹麦语": "snart",
    "俄语": "скоро",
    "土耳其语": "yakında",
    "德语": "bald",
    "意大利语": "presto",
    "日语": "近いうちに",
    "法语": "bientôt",
    "瑞典语": "snart",
    "荷兰语": "binnenkort",
    "葡萄牙语": "em breve",
    "西班牙语": "pronto",
    "韩语": "곧",
}

# 其他一些模糊时间词（每语言10条可以变化）
extra_fuzzy = {
    "简体中文": ["稍后", "过一阵子", "以后", "不久之后", "近来", "早晚", "迟早", "随后", "日后", "以后再说"],
    "繁体中文": ["稍後", "過一陣子", "以後", "不久之後", "近來", "早晚", "遲早", "隨後", "日後", "以後再說"],
    "英语": ["later", "after a while", "eventually", "someday", "in the near future", "soon enough", "at some point", "in due time", "before long", "down the road"],
    "丹麦语": ["senere", "efter et stykke tid", "på et tidspunkt", "snart nok", "inden længe", "engang", "i den nærmeste fremtid", "før eller siden", "på sigt", "en dag"],
    "俄语": ["позже", "через какое-то время", "рано или поздно", "в ближайшем будущем", "когда-нибудь", "в скоро времени", "на днях", "со временем", "в будущем", "не сразу"],
    "土耳其语": ["daha sonra", "bir süre sonra", "yakın zamanda", "eninde sonunda", "ileride", "bir gün", "vaktinde", "çok geçmeden", "bir zamanlar", "sonunda"],
    "德语": ["später", "nach einer Weile", "irgendwann", "bald genug", "in naher Zukunft", "früher oder später", "eines Tages", "mit der Zeit", "kurz darauf", "demnächst"],
    "意大利语": ["più tardi", "fra un po'", "alla fine", "un giorno", "in futuro", "presto o tardi", "col tempo", "entro breve", "appena possibile", "successivamente"],
    "日语": ["後で", "しばらくして", "いずれ", "近いうちに", "やがて", "そのうち", "今後", "後ほど", "ほどなく", "いつか"],
    "法语": ["plus tard", "après un certain temps", "un jour", "dans un avenir proche", "tôt ou tard", "à la longue", "au fil du temps", "bientôt", "ultérieurement", "à un moment donné"],
    "瑞典语": ["senare", "efter ett tag", "någon gång", "förr eller senare", "snart nog", "i framtiden", "en dag", "så småningom", "i sinom tid", "inom kort"],
    "荷兰语": ["later", "na een tijdje", "ooit", "vroeg of laat", "binnenkort", "in de nabije toekomst", "mettertijd", "tegen die tijd", "op termijn", "eens"],
    "葡萄牙语": ["mais tarde", "daqui a pouco", "algum dia", "no futuro", "em breve", "cedo ou tarde", "com o tempo", "logo depois", "no devido tempo", "eventualmente"],
    "西班牙语": ["más tarde", "al cabo de un rato", "algún día", "en un futuro cercano", "pronto", "tarde o temprano", "con el tiempo", "dentro de poco", "en su momento", "eventualmente"],
    "韩语": ["나중에", "잠시 후", "언젠가", "곧", "조만간", "서서히", "머지않아", "후에", "차후에", "훗날"],
}

def generate_files():
    for lang, phrase in translations.items():
        filepath = os.path.join(output_dir, f"{lang}.txt")
        # 前一个固定词 + 后面 extra_fuzzy 结合
        fuzzy_list = [phrase] + extra_fuzzy.get(lang, [])
        with open(filepath, "w", encoding="utf-8") as f:
            for word in fuzzy_list:
                f.write(word + "\n")
        print(f"生成文件: {filepath}")

if __name__ == "__main__":
    generate_files()

生成文件: /content/drive/MyDrive/NER/non_time/简体中文.txt
生成文件: /content/drive/MyDrive/NER/non_time/繁体中文.txt
生成文件: /content/drive/MyDrive/NER/non_time/英语.txt
生成文件: /content/drive/MyDrive/NER/non_time/丹麦语.txt
生成文件: /content/drive/MyDrive/NER/non_time/俄语.txt
生成文件: /content/drive/MyDrive/NER/non_time/土耳其语.txt
生成文件: /content/drive/MyDrive/NER/non_time/德语.txt
生成文件: /content/drive/MyDrive/NER/non_time/意大利语.txt
生成文件: /content/drive/MyDrive/NER/non_time/日语.txt
生成文件: /content/drive/MyDrive/NER/non_time/法语.txt
生成文件: /content/drive/MyDrive/NER/non_time/瑞典语.txt
生成文件: /content/drive/MyDrive/NER/non_time/荷兰语.txt
生成文件: /content/drive/MyDrive/NER/non_time/葡萄牙语.txt
生成文件: /content/drive/MyDrive/NER/non_time/西班牙语.txt
生成文件: /content/drive/MyDrive/NER/non_time/韩语.txt


##### 混合模版生成器（时间0—3/地点0—3）（时间插入地点）

**本段落主要用于从最小元数据结构开始**以{address}和{time}为主要元素向大模型发送请求，做组词造句式模板，生成结构与之前两段模板生成器完全一致。但本段代码采用：**“创建正确格式文件夹来选择性生成你需要数据”**的数据控制方式。第一次运行时数据分布安排如下（尾号分别代表address个数和time个数，如01代表0个address，1个time的句子）：


---
分布矩阵如下：   
*   0 1  2  3
*   0 1600 500 500 500
*   1 500 4000 500 500
*   2 500 500 500 500
*   3 500 500 500 500









In [ ]:
import os
import re
import json
import random
import asyncio
from typing import List, Dict, Any, Optional
import httpx
from google.colab import userdata

# API Key和接口配置
api_key = userdata.get('api_key')
BASE_URL   = os.getenv("LLM_BASE_URL", "https://test-turing.cn.llm.tcljd.com")
ENDPOINT   = os.getenv("LLM_ENDPOINT", "/api/v1/chat/completions")
API_KEY    = os.getenv("LLM_API_KEY", api_key)
HEADERS    = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

# 输出根目录
OUTPUT_BASE_DIR  = "/content/drive/MyDrive/NER/combine_templates"

LANGUAGES = [
    "简体中文", "繁体中文", "英语", "丹麦语", "俄语", "土耳其语", "德语",
    "意大利语", "日语", "法语", "瑞典语", "荷兰语", "葡萄牙语", "西班牙语", "韩语"
]

TEMPLATE_BATCH_SIZE = 40
TEMPERATURE = 0.8
MODEL_NAME = "turing/gpt-4o"
TIMEOUT = 90

client: Optional[httpx.AsyncClient] = None

# 西文介词黑名单（禁止出现在 {time} 前）
PREPOSITION_BLACKLIST = {
    "英语": ["at", "by", "on", "before", "after"],
    "西班牙语": ["a", "para", "en", "por", "antes de", "después de", "hacia", "sobre"],
    "葡萄牙语": ["a", "para", "em", "por", "antes de", "depois de", "às", "à"],
    "意大利语": ["a", "per", "in", "su", "prima di", "dopo"]
}

# 演示样例（多语言）
SAMPLE_PAIRS = {
    "简体中文": [
        "我{time}打算去{address}看看，听说那边的{address}景色很迷人。",
        "{time}我们将在{address}集合，并一同前往{address}参加活动。",
        "传闻{address}与{address}相连的步道在{time}最适合拍照。",
        "我喜欢在{time}去{address}散步，然后顺道去{address}吃晚餐。",
        "{address}位于{address}，在{time}举办了盛大的灯会。",
        "{time}刚吃完晚饭就去了{address}散步，到了{time}又和朋友在{address}看电影。"
    ],
    "繁体中文": [
        "我{time}會去{address}走走，聽朋友說{address}的風景非常漂亮。",
        "{time}我們約在{address}碰面，再一起去{address}逛逛。",
        "聽說位於{address}的{address}在{time}非常熱鬧。",
        "我喜歡{time}到{address}喝茶，順便去{address}買點小吃。",
        "{address}鄰近{address}，並於{time}舉辦花卉展覽。",
        "{time}剛下班路過{address}買麵包，{time}又與家人到{address}散步。"
    ],
    "英语": [
        "I'm heading to {address} {time} to catch the view, then stopping by {address} for coffee.",
        "We’re meeting at {address} {time} before walking to {address} for the exhibition.",
        "{time} is the perfect moment to explore {address} and then visit {address} nearby.",
        "Locals say {address}, next to {address}, shines best {time}.",
        "{address} was the venue for a major cultural fair {time}.",
        "{time} I left {address}, {time} I arrived at {address} for dinner."
    ],
    "丹麦语": [
        "Jeg vil tage til {address} {time} for at nyde udsigten og derefter besøge {address}.",
        "Vi mødes ved {address} {time} og går sammen til {address} for arrangementet.",
        "{time} er det bedste tidspunkt at gå en tur fra {address} til {address}.",
        "{address} ligger tæt på {address} og havde en stor festival {time}.",
        "Der er jazzkoncerter i {address}, nær {address}, {time}.",
        "{time} jeg gik til {address}, {time} jeg kom til {address} for koncerten."
    ],
    "俄语": [
        "Я пойду в {address} {time}, а потом зайду в {address} на обед.",
        "Мы встречаемся у {address} {time} и идём вместе в {address} на выставку.",
        "{time} идеально подходит для прогулки из {address} в {address}.",
        "Местные говорят, что {address} рядом с {address} особенно красив {time}.",
        "В {address}, недалеко от {address}, {time} прошёл ярмарка.",
        "{time} я вышел из {address}, {time} я пришёл в {address} к встрече."
    ],
    "土耳其语": [
        "{time} {address}'te buluşup ardından {address}'e geçeceğiz.",
        "En güzel manzara {time} {address} ve {address} arasında görülür.",
        "{address}, {address} yakınında {time} düzenlenen festivalin merkeziydi.",
        "{time} günü {address}'e gidip oradan {address}'e alışverişe geçmeyi planlıyorum.",
        "{address}'teki kahve molasından sonra {address}'e yürüyüş yapacağız.",
        "{time} {address}'ten çıktım, {time} {address}'e geldim."
    ],
    "德语": [
        "Ich gehe {time} zum {address} und danach in den {address} spazieren.",
        "Wir treffen uns {time} bei {address}, dann besuchen wir den {address}.",
        "Von {address} zum {address} wandert man am besten {time}.",
        "{address} in der Nähe von {address} war {time} Austragungsort eines Festes.",
        "Ein Nachmittag {time} im {address} und anschließend im {address} ist perfekt.",
        "{time} ich verließ {address}, {time} ich ging zum {address} für das Treffen."
    ],
    "意大利语": [
        "Visito {address} {time} e poi passo al famoso {address} per un gelato.",
        "{time} ci incontriamo a {address} e andiamo insieme a {address} per lo spettacolo.",
        "{address} vicino a {address} ospita una fiera {time}.",
        "{time} è il momento ideale per camminare da {address} a {address}.",
        "{address} è stato luogo di un grande concerto {time}.",
        "{time} sono uscito da {address}, {time} sono arrivato a {address}."
    ],
    "日语": [
        "{time}に{address}へ行って、その後{address}でお茶をします。",
        "{address}の近くの{address}では{time}に花祭りが開催されます。",
        "{time}は{address}から{address}まで散歩するのに最高です。",
        "{address}と{address}の間の小道が{time}にとても綺麗です。",
        "{address}にある博物館は{time}に特別展を開きました。",
        "{time}{address}を出て、{time}{address}に着いた。"
    ],
    "法语": [
        "Je vais à {address} {time} puis prendre un café près de {address}.",
        "Nous nous retrouvons {time} à {address} avant d’aller à {address} pour le marché.",
        "Entre {address} et {address}, la balade {time} est magnifique.",
        "{address} près de {address} a accueilli un festival {time}.",
        "Un après-midi {time} dans {address} et ensuite {address} est parfait.",
        "{time} je suis parti de {address}, {time} je suis arrivé à {address}."
    ],
    "瑞典语": [
        "Jag besöker {address} {time} och går sedan till {address} för middag.",
        "{time} träffas vi vid {address} och går till {address} tillsammans.",
        "Vandringen mellan {address} och {address} är bäst {time}.",
        "{address} nära {address} höll en stor marknad {time}.",
        "En kväll {time} i {address} följt av {address} är underbar.",
        "{time} jag lämnade {address}, {time} jag kom till {address}."
    ],
    "荷兰语": [
        "Ik ga {time} naar {address} en daarna naar {address} om te winkelen.",
        "We ontmoeten elkaar {time} bij {address} en wandelen naar {address}.",
        "Tussen {address} en {address} lopen is prachtig {time}.",
        "{address} vlakbij {address} had {time} een muziekfestival.",
        "Een ochtend {time} in {address} en daarna naar {address} is perfect.",
        "{time} ik vertrok uit {address}, {time} ik kwam aan bij {address}."
    ],
    "葡萄牙语": [
         "{time} visitei {address} e depois caminhei pelas ruas históricas de {address}.",
         "Durante {time} participamos de um festival gastronômico no {address}.",
         "No {address}, {time} assistimos a um concerto ao ar livre.",
         "Passei {time} todo no {address} tirando fotos e aproveitando a vista.",
         "{time} fui ao {address} para devolver livros e depois ao {address} para comprar pão.",
          "Entre {time} e {time} viajamos por {address}, {address} e {address}."
    ],
    "西班牙语": [
      "{time} visité {address} y luego paseé por el casco antiguo de {address}.",
      "Durante {time} asistimos a una feria de arte en {address}.",
      "En {address}, {time} escuchamos un concierto de música clásica.",
      "Pasé todo {time} en {address} tomando fotos y disfrutando la vista.",
      "{time} fui a {address} para devolver unos libros y luego a {address} para comprar pan.",
      "Entre {time} y {time} recorrimos {address}, {address} y {address}.",
      "Anoche cenamos en {address} junto al {address}.",
    ],
    "韩语": [
        "{time}에 {address}에 들른 후에 {address}에서 저녁을 먹을 예정입니다.",
        "{time} {address}에서 만나 {address}로 함께 갈 거예요.",
        "{address} 근처의 {address}에서는 {time}에 축제가 열립니다.",
        "{time}에는 {address}와 {address} 사이 길이 가장 아름답습니다.",
        "{address}는 {address}에 위치하며 {time}에 문화 행사를 개최했습니다.",
        "{time}{address}에서 나왔고, {time}{address}에 도착했다."
    ]
}
extra_samples = {
    "简体中文": [
        "{time}我从{address}坐车出发，大约{time}到达{address}开始排队，等到{time}才轮到我们进去。",
        "我在{time}抵达{address}喝咖啡，{time}前往{address}与朋友共进晚餐。",
        "{time}我从{address}启程，在{time}抵达{address}短暂休息，接着{time}又前往{address}参加晚宴。",
        "我{time}出发去{address}，{time}到达后开始拍摄，{time}离开时已近黄昏。",
        "刚到{time}我已在{address}与同事会面，{time}驱车去{address}处理事务，直到{time}才返回{address}。"
    ],
    "繁体中文": [
        "在{time}與{time}之間，我們走訪了{address}、{address}和{address}的文化中心。",
        "{time}到達{address}拍攝，然後前往{address}參展，最後{time}返回{address}整理相片。",
        "聽說{address}與{address}相連的小路在{time}與{time}都有不同的花景，所以我們特意安排{address}之旅。",
        "{time}於{address}辦研討會，{time}移師{address}，{time}結束並返回{address}。"
    ],
    "英语": [
        "Between {time} and {time}, we visited {address}, {address}, and {address} for sightseeing.",
        "{time} at {address} for the meeting, {time} moved to {address} for lunch, then {time} back at {address} to wrap up.",
        "{time} reporters gathered at {address}, then headed to {address} for interviews, finally at {time} returned to {address} for editing.",
        "\"Meet me at {address} {time}, then we'll go to {address} and {address},\" she texted."
    ],
    "丹麦语": [
        "Mellem {time} og {time} besøgte vi {address}, {address} og {address}.",
        "{time} startede vi ved {address}, {time} tog til {address} for at se udstillingen, {time} gik vi til {address} for aftensmad.",
        "{time} ankom fotografer til {address}, {time} fortsatte mod {address}, og {time} vendte tilbage til {address}.",
        "Vi planlægger at tage fra {address} {time}, besøge {address}, og derefter {address} inden aften."
    ],
    "俄语": [
        "Между {time} и {time} мы посетили {address}, {address} и {address}.",
        "Между {time} и {time} мы встретились в {address}, обсудили детали в {address}, а к {time} вернулись в {address} для подготовки отчёта.",
        "{time} журналисты прибыли в {address}, {time} провели интервью, а к {time} вернулись в {address} для монтажа материала.",
        "В {time} началась эвакуация из {address}, и только {time} люди добрались до {address} для получения помощи.",
    ],
    "土耳其语": [
        "{time} ile {time} arasında {address}, {address} ve {address} ziyaret ettik.",
        "{time} {address}'te başladık, {time} {address}'e geçtik, {time} günü {address}'te bitirdik.",
        "{time} gazeteciler {address}'te toplandı, {address}'e gitti, {time} tekrar {address}'e döndü.",
        "{time} {address}'ten yola çıkıp {address}'e uğradım, sonra {address}'e devam ettim."
    ],
    "德语": [
        "Zwischen {time} und {time} besuchten wir {address}, {address} und {address}.",
        "{time} starteten wir am {address}, {time} gingen wir zum {address}, und {time} beendeten wir den Tag im {address}.",
        "{time} versammelten sich Reporter im {address}, fuhren zum {address} und kehrten {time} ins {address} zurück.",
        "Ich plane, den {address} {time} zu verlassen, zum {address} zu fahren und dann zum {address}."
    ],
    "意大利语": [
        "Tra {time} e {time} abbiamo visitato {address}, {address} e {address}.",
        "{time} siamo partiti da {address}, {time} abbiamo pranzato al {address}, {time} abbiamo finito al {address}.",
        "{time} i giornalisti si sono riuniti al {address}, poi al {address}, e {time} erano di nuovo al {address}.",
        "Parto da {address} {time}, passo al {address} e poi al {address}."
    ],
    "日语": [
        "{time}と{time}の間に{address}、{address}、{address}を巡った。",
        "{time}{address}を出発し、{time}{address}へ移動して展示会を見学、{time}{address}で解散した。",
        "{time}{address}で記者会見、その後{address}へ移動し、{time}{address}に戻った。",
        "{time}{address}を出て{address}を訪問し、その後{address}へ向かった。"
    ],
    "法语": [
        "Entre {time} et {time}, nous avons visité {address}, {address} et {address}.",
        "{time} nous avons commencé à {address}, {time} avons déjeuné à {address}, puis {time} avons terminé au {address}.",
        "{time} les journalistes se sont rassemblés à {address}, puis sont allés à {address}, et à {time} sont revenus à {address}.",
        "Je pars de {address} {time}, vais à {address}, puis à {address}."
    ],
    "瑞典语": [
        "Mellan {time} och {time} besökte vi {address}, {address} och {address}.",
        "{time} började vi vid {address}, {time} fortsatte till {address}, {time} avslutade vid {address}.",
        "{time} samlades reportrar vid {address}, fortsatte till {address}, och {time} tillbaka till {address}.",
        "Jag lämnar {address} {time}, besöker {address} och {address}."
    ],
    "荷兰语": [
        "Tussen {time} en {time} bezochten we {address}, {address} en {address}.",
        "{time} begonnen we bij {address}, {time} gingen naar {address}, en {time} eindigden bij {address}.",
        "{time} verzamelden journalisten zich bij {address}, gingen naar {address}, en {time} keerden terug naar {address}.",
        "Ik vertrek uit {address} {time}, ga langs {address} en {address}."
    ],
    "葡萄牙语": [
        "Ontem à noite jantamos no {address} perto da {address}.",
        "No verão passado acampamos nas montanhas de {address}.",
        "Sem hora marcada, encontrei amigos no {address}.",
        "Partimos de {address} {time} e chegamos ao {address} {time}.",
        "Ontem comprei flores no {address}.",
        "{time} não saí de casa."
    ],
    "西班牙语": [
          "En invierno esquiamos en {address}.",
         "Sin hora fija, me encontré con amigos en {address}.",
         "Salimos de {address} {time} y llegamos a {address} {time}.",
        "Ayer compré flores en {address}.",
        "{time} no salí de casa."
    ],
    "韩语": [
        "{time}과 {time} 사이에 {address}, {address}, {address}를 방문했다.",
        "{time}{address}에서 출발해, {time}{address}로 이동 후, {time}{address}에서 마무리했다.",
        "{time}{address}에서 기자회견을 열고, {address}로 이동해, {time}{address}로 돌아왔다.",
        "{time}{address}를 떠나 {address}를 거쳐 {address}로 향했다."
    ]
}

for lang, sentences in extra_samples.items():
    SAMPLE_PAIRS[lang].extend(sentences)
def build_prompt(language: str, addr_count: int, time_count: int) -> str:
    # 构造演示文本
    examples = SAMPLE_PAIRS.get(language, [])
    examples_text = "\n".join(json.dumps({"language": language, "template": ex}, ensure_ascii=False) for ex in examples)
    return f"""
你是一名多语言模板设计专家，要在所给语言中造句。

### 规则：
1. 必须使用固定数量的{{address}}和{{time}}占位符：
   - {{address}}出现次数 = {addr_count}
   - {{time}}出现次数 = {time_count}
2. 不得出现任何真实地址或真实时间用词，唯一的地址类词只能是{{address}}，唯一的时间类词只能是{{time}}。
3. 句子必须自然流畅，符合同语言的表达习惯，可带有修饰成分，要求生成的句子要尽可能针对各种语言进行母语化出路，符合各语种日常生活特征。
4. 尽可能丰富的多种语境（旅游、工作、活动、新闻、故事、论文、小说、日常聊天等）混合生成。
5. 西文语言（英语、西班牙语、葡萄牙语、意大利语）中{{time}}前禁止加介词。
6. 输出为JSONL，每行一个JSON对象：{{"language": "<语言>", "template": "<句子>"}}
7. 禁止输出JSONL以外内容。
8. 当要求出现的address和time出现次数都为0时，要求你生成不含任何时间地址（包括“今天"，“昨天"，“这个周末"等）且无占位符的整句负例。
9. 重点审查：演示样例中的占位符数量可能与规则1.中声明的不同，因此需要你严格依赖规则1.中声明的占位符数量生成模板，从样例中请你尽量学习句式即可。
10. 演示样例仅供参考，请你从网络上大量学习该语种表达习惯以丰富句式，严格禁止过度模仿重复样例,可以采用多种人称叙述方式，不要过度使用第一人称。

=== 演示样例 ===
{examples_text}

请用{language}生成{TEMPLATE_BATCH_SIZE}条符合以上规则的句子。
""".strip()

async def chat_stream(payload: dict) -> str:
    assert client is not None
    text_buf: List[str] = []
    async with client.stream("POST", ENDPOINT, headers=HEADERS, content=json.dumps(payload)) as resp:
        async for line in resp.aiter_lines():
            if not line.startswith("data:"):
                continue
            data = line[5:].strip()
            if data == "[DONE]":
                break
            try:
                delta = json.loads(data)["choices"][0]["delta"].get("content", "")
            except Exception:
                continue
            text_buf.append(delta)
    return "".join(text_buf)

def parse_jsonl(text: str) -> List[Dict[str, Any]]:
    lines = [ln.strip() for ln in text.strip().splitlines() if ln.strip()]
    out = []
    for ln in lines:
        try:
            obj = json.loads(ln)
            if isinstance(obj, dict):
                out.append(obj)
        except:
            pass
    return out

def post_process(data: List[Dict[str, Any]], addr_count: int, time_count: int, language: str):
    out, seen = [], set()
    rejected = []
    blacklist = PREPOSITION_BLACKLIST.get(language, [])

    for item in data:
        t = str(item.get("template", "")).strip()
        if not t or t in seen:
            rejected.append((t, "空或重复"))
            continue
        if t.count("{address}") != addr_count:
            rejected.append((t, f"{t.count('{address}')}个{{address}}占位符"))
            continue
        if t.count("{time}") != time_count:
            rejected.append((t, f"{t.count('{time}')}个{{time}}占位符"))
            continue
        # 检查西文介词黑名单
        if any(re.search(rf"\b{prep}\s*\{{time\}}", t, re.IGNORECASE) for prep in blacklist):
            rejected.append((t, "时间前出现介词"))
            continue

        out.append(t)
        seen.add(t)
    return out, rejected

async def generate_batch(language: str, addr_count: int, time_count: int):
    prompt = build_prompt(language, addr_count, time_count)
    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "stream": True,
        "temperature": TEMPERATURE
    }
    raw = await chat_stream(payload)
    parsed = parse_jsonl(raw)
    valid, rejected = post_process(parsed, addr_count, time_count, language)
    return valid, rejected

def parse_counts_from_dir(dirname: str):
    match = re.match(r"combine_templates_(\d)(\d)", dirname)
    if not match:
        raise ValueError(f"目录名不合法: {dirname}")
    return int(match.group(1)), int(match.group(2))

async def generate_for_dir(dirname: str):
    addr_count, time_count = parse_counts_from_dir(dirname)
    dir_path = os.path.join(OUTPUT_BASE_DIR, dirname)
    os.makedirs(dir_path, exist_ok=True)

    # === 根据文件夹名末尾自动调整 target ===
    suffix = dirname.split('_')[-1]  # 比如 combine_templates_11 => '11'
    if suffix == "11":
        target = 4000
    elif suffix == "00":
        target = 1000
    else:
        target = 500  # 你可以随时改这个默认值

    print(f"[Info] {dirname} -> 每语言产出目标:{target}")

    for lang in LANGUAGES:
        out_path = os.path.join(dir_path, f"{lang}_combine_templates.jsonl")
        existing = sum(1 for _ in open(out_path, "r", encoding="utf-8")) if os.path.exists(out_path) else 0
        total = existing
        while total < target:
            try:
                batch, rejected = await generate_batch(lang, addr_count, time_count)
            except Exception as e:
                print(f"[{lang}] 批次失败: {e}")
                await asyncio.sleep(1)
                continue
            if batch:
                with open(out_path, "a", encoding="utf-8") as f:
                    for t in batch:
                        f.write(json.dumps({"language": lang, "template": t}, ensure_ascii=False) + "\n")
                total += len(batch)
                print(f"[{lang}] 生成进度: {total}/{target}")
            else:
                print(f"[{lang}] 无可用模板，跳过一次")
                if rejected:
                    print(f"[{lang}] 被淘汰模板示例:")
                    for sample, reason in rejected[:5]:  # 显示前 5 个
                        print(f"    ❌ 原文: {sample} | 原因: {reason}")
                await asyncio.sleep(1)

async def main():
    global client
    client = httpx.AsyncClient(base_url=BASE_URL, timeout=TIMEOUT)
    try:
        for dirname in os.listdir(OUTPUT_BASE_DIR):
            if dirname.startswith("combine_templates_"):
                await generate_for_dir(dirname)
    finally:
        await client.aclose()

# 执行
await main()

[Info] combine_templates_11 -> 每语言产出目标:4000
[繁体中文] 生成进度: 2511/4000
[繁体中文] 生成进度: 2551/4000
[繁体中文] 生成进度: 2579/4000
[繁体中文] 生成进度: 2619/4000
[繁体中文] 生成进度: 2659/4000
[繁体中文] 生成进度: 2699/4000
[繁体中文] 生成进度: 2739/4000
[繁体中文] 生成进度: 2779/4000
[繁体中文] 生成进度: 2818/4000
[繁体中文] 生成进度: 2857/4000
[繁体中文] 生成进度: 2896/4000
[繁体中文] 生成进度: 2936/4000
[繁体中文] 生成进度: 2976/4000
[繁体中文] 生成进度: 3016/4000
[繁体中文] 生成进度: 3054/4000


CancelledError: 

**输入输出说明：**
*  **输入**：15种语言（中文分简体繁体）的纯地址txt文件，命名为语言名.txt，文件夹为real_address


输入结构：


1. 真实地址

```
/content
└── drive
    └── MyDrive
        └── NER
            └── real_address(共17个文件，要自己上传到谷歌云盘)
                ├── 简体中文.txt
                ├── 繁体中文.txt
                  ......
                └── 韩语.txt
            └── non_address
                ├── 简体中文.txt
                ├── 繁体中文.txt
                  ......
                └── 韩语.txt           
```
2. 模板文件，文件结构见本文件中的模板生成代码（在上面）

---

*  **输出：**


```
/content
└── drive
    └── MyDrive
        └── NER
            ├── train_datasets(共16个文件)
                ├── train_简体中文.conll
                ├── train_繁体中文.conll
                ├── train_英语.conll
                ......
                └── train_all.conll

```

**标记规则：**

1. 含1~n个地址的数据
    *   正例：应该被识别的地址->标记为ADDRESS
        *   加噪声
        *   不加噪声

    *   负例（如有硬性不允许被识别的地址）：不应该被识别的地址->标记为O （注意覆盖全面和多样性）

2. 含0个地址的数据
    
    整句标记为O



**参数说明**
1. N_PER_LANG:
    
    每种语言生成多少条训练数据，不输入（None）自动为每个模板合成，再按纯负例占比RATIO_NEG合成负例。

2. PREVIEW_N:

    合成结束后预览多少条数据
  
3. RATIO_NEG:
    
    整句负例概率

4. NEG_ADDR_PROB:

    地址负例概率


In [ ]:
# @title 生成训练数据（CONLL）（纯地址） {"form-width":"40%"}
from google.colab import drive  # 在 Colab 环境下可用
import os, re, json, random, unicodedata, argparse, sys
from typing import List, Tuple, Dict, Optional
from transformers import AutoTokenizer


# 挂载谷歌云盘
drive.mount('/content/drive')
print("云盘挂载成功")


# ======================================================可按需修改的配置 ======================================================


N_PER_LANG = 10000 # @param {"type":"raw","placeholder":"请输入每个语言的训练数据量，不输入默认遍历所有模板"}

RATIO_NEG = 0.2 # @param {"type":"number","placeholder":"整句负例占比（相对每语言总量）"}
NEG_ADDR_PROB = 0.3  # @param {"type":"number","placeholder":"插入地址负例的概率"}

# 预览， >0 时打印样例
PREVIEW_N = 5  # @param {"type":"integer","placeholder":"请输入预览数据量"}

# 噪声配置
NOISE_PROB_CASE = 0.15               # 大小写扰动概率（适用于拉丁文字）
NOISE_PROB_FULLWIDTH = 0.10          # 全/半角互换概率（中日朝/数字/标点）
NOISE_PROB_SEP_REPLACE = 0.10        # 分隔符替换概率
NOISE_PROB_INSERT = 0            # 插入字符概率（会触发标签移位/内部I-延续）
# 插入字符串的噪声
INSERT_CHARS = [",", ".", "·", "—", "-", "/", ":", "、", "・","?","？", "！", "!", "，", "。", "；", ";", " "]

# 仅 address / 仅 time 的样本比例（“几乎不含O”）
RATIO_STRONG_POS = 0.10  # 在正例中抽取的比例，构造只有 address 或只有 time 的短样本（近乎全实体）


# =================================================================================================================================


LANGS = ["简体中文","繁体中文", "英语","丹麦语","俄语","土耳其语","德语","意大利语","日语",
         "法语","瑞典语","荷兰语","葡萄牙语","西班牙语","韩语"]


# ================== 输入路径 ==================
# 真实地址与非地址片段
REAL_ADDR_ROOT = "/content/drive/MyDrive/NER/real_address"   # <语言>.txt 与 non_address.txt
NON_ADDR_ROOT  = "/content/drive/MyDrive/NER/non_address"

# 模板根目录（包含 address_templates_0..5，其中 0 = 整句纯负例模板）
ADDR_TPL_ROOT = "/content/drive/MyDrive/NER/address_templates"


# ================== 输出路径 ==================
OUT_DIR = "/content/drive/MyDrive/NER/train_datasets"
os.makedirs(OUT_DIR, exist_ok=True)

PER_LANG_CONLL = lambda lang: os.path.join(OUT_DIR, f"train_{lang}.conll") # 拼接每语言输出路径
MERGED_CONLL = os.path.join(OUT_DIR, "train_all.conll") # 拼接全部数据输出路径


TOKENIZER_NAME = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME, use_fast=True)

# ================== 基础IO ==================
def read_lines(path: str) -> List[str]:
    if not os.path.exists(path):
        return []
    with open(path, "r", encoding="utf-8") as f:
        return [ln.strip() for ln in f if ln.strip()]

# def choose_time(lang: str, time_map: Dict[str, List[str]], time_numeric: List[str]) -> str:
#     if random.random() < RATIO_TIME_NUMERIC and time_numeric:
#         return random.choice(time_numeric)
#     arr = time_map.get(lang, [])
#     if not arr:  # 回退
#         arr = time_numeric or ["2025-08-12", "14:30", "2025-08-12 14:30"]
#     return random.choice(arr)

def choose_addr(lang: str, addr_map: Dict[str, List[str]]) -> str:
    arr = addr_map.get(lang, [])
    if not arr:
        # 最弱回退
        return "Main Street 1"
    return random.choice(arr)

def read_jsonl_templates(path: str) -> list:
    items = []
    if not os.path.exists(path):
        return items
    with open(path, "r", encoding="utf-8") as f:
        for ln in f:
            ln = ln.strip()
            if not ln:
                continue
            try:
                obj = json.loads(ln)
                t = obj.get("template")
                if isinstance(t, str) and t.strip():
                    items.append(" ".join(t.split()))
            except Exception:
                continue
    return items

def load_templates_for_lang(lang: str) -> Dict[str, list]:
    """
    遍历 ADDR_TPL_ROOT 下所有 address_templates_*
      - address_templates_0：整句纯负例模板（neg_sent）
      - address_templates_1..N：地址模板（addr_only）
    统一读取 <语言>_address_templates.jsonl 的 "template" 字段。
    返回 {"addr_only": [...], "neg_sent": [...]}。
    """
    addr_only, neg_sent = [], []
    if not os.path.isdir(ADDR_TPL_ROOT):
        return {"addr_only": addr_only, "neg_sent": neg_sent}

    prefix = "address_templates_"
    filename = f"{lang}_address_templates.jsonl"   # 统一命名

    for name in os.listdir(ADDR_TPL_ROOT):
        if not name.startswith(prefix):
            continue
        subdir = os.path.join(ADDR_TPL_ROOT, name)
        if not os.path.isdir(subdir):
            continue

        fpath = os.path.join(subdir, filename)
        items = read_jsonl_templates(fpath)
        if not items:
            continue

        # 0 号目录视为整句纯负例模板
        slot = name[len(prefix):]
        if slot.isdigit() and int(slot) == 0:
            neg_sent.extend(items)
        else:
            addr_only.extend(items)

    return {"addr_only": addr_only, "neg_sent": neg_sent}



# 增强稳健性：用非地址片段填充整句负例模板（即使含 {address} 也不记录 span，生成全 O）
def fill_template_all_non_address(template: str, lang: str, non_addr_map: Dict[str, List[str]]) -> str:
    out, cursor = [], 0
    for m in PLACEHOLDER_RE.finditer(template):
        out.append(template[cursor:m.start()])
        out.append(choose_non_address(lang, non_addr_map))
        cursor = m.end()
    out.append(template[cursor:])
    return "".join(out)




# ================== 噪声：不改变“可整体识别为日期”的语义，但可能插入/替换字符 ==================
def toggle_case(s: str) -> str:
    def _flip(c):
        if "a" <= c <= "z": return c.upper()
        if "A" <= c <= "Z": return c.lower()
        return c
    return "".join(_flip(c) for c in s)

def to_fullwidth(c: str) -> str:
    # 半角 -> 全角
    code = ord(c)
    if 33 <= code <= 126:
        return chr(code + 0xFEE0)
    if c == " ":
        return "\u3000"
    return c

def to_halfwidth(c: str) -> str:
    # 全角 -> 半角
    if c == "\u3000":
        return " "
    code = ord(c)
    if 65281 <= code <= 65374:
        return chr(code - 0xFEE0)
    return c

def fullwidth_halfwidth_flip(s: str) -> str:
    out = []
    for ch in s:
        if unicodedata.east_asian_width(ch) in ("F","W") and random.random() < 0.5:
            out.append(to_halfwidth(ch))
        elif random.random() < 0.2:
            out.append(to_fullwidth(ch))
        else:
            out.append(ch)
    return "".join(out)

def replace_separators(s: str) -> str:
    # 常见分隔符替换
    trans = {
        "-": random.choice(["—","-","–"]),
        "/": random.choice(["/","／","∕"]),
        ".": random.choice([".","·","．"]),
        ":": random.choice([":","："])
    }
    return "".join(trans.get(ch, ch) for ch in s)

# 核心：插入字符并维护实体跨度

def insert_char_with_spans_multi(text: str,
                                 spans: List[Tuple[int,int]],
                                 p: Optional[int] = None,
                                 ch: Optional[str] = None):
    if not text:
        return text, spans
    if p is None:
        p = random.randint(0, len(text))
    if ch is None:
        ch = random.choice(INSERT_CHARS)

    new_text = text[:p] + ch + text[p:]
    new_spans: List[Tuple[int,int]] = []
    for (s, e) in spans:
        if p < s:
            new_spans.append((s+1, e+1))         # 插到实体前 -> 整段右移
        elif s <= p < e:
            new_spans.append((s, e+1))           # 插到实体内 -> 扩展实体右端
        else:
            new_spans.append((s, e))             # 插到实体后 -> 不变
    return new_text, new_spans


# 对整个字符串加噪，并保持实体下标对齐
def apply_noise(text: str,
                spans: Dict[str, Optional[Tuple[int,int]]],
                lang: str) -> Tuple[str, List[Tuple[int,int]]]:
    """
    仅用不会破坏“整体可被识别为日期/地址”的噪声：
    - 大小写扰动（en/da/de/it/fr/sv/nl/pt/es/tr/ru部分拉丁字母）
    - 全/半角替换（中日朝/数字/标点）
    - 分隔符替换
    - 插入字符（需维护跨度）
    """
    s = text

    # 仅对拉丁字母系语言做大小写扰动
    latin_langs = {"英语","丹麦语","德语","意大利语","法语","瑞典语",
                   "荷兰语","葡萄牙语","西班牙语","土耳其语"}
    # 大小写
    if lang in latin_langs and random.random() < NOISE_PROB_CASE:
        s = toggle_case(s)
    # 全/半角
    if random.random() < NOISE_PROB_FULLWIDTH:
        s = fullwidth_halfwidth_flip(s)
    # 分隔符
    if random.random() < NOISE_PROB_SEP_REPLACE:
        s = replace_separators(s)
    # 插入字符（影响span）
    s, spans = insert_char_with_spans_multi(s, spans, NOISE_PROB_INSERT)
    return s, spans


# 仅对 ADDRESS（可多段）进行 BIO 标注
def spans_to_bio_addresses(text: str,
                           addr_spans: List[Tuple[int,int]]) -> Tuple[List[str], List[str]]:
    enc = tokenizer(text, add_special_tokens=False, return_offsets_mapping=True)
    tokens = tokenizer.convert_ids_to_tokens(enc["input_ids"])
    offsets = enc["offset_mapping"]

    # 计算重叠长度
    def overlap_len(a, b):
        s = max(a[0], b[0]); e = min(a[1], b[1])
        return max(0, e - s)

    labels = []
    for (s, e) in offsets:
        tok_span = (s, e)
        # 找与该 token 重叠最多的 address 段
        best = None
        best_ol = 0
        for span in addr_spans:
            ol = overlap_len(tok_span, span)
            if ol > best_ol:
                best_ol = ol; best = span
        if best is None or best_ol == 0:
            labels.append("O")
            continue
        # 如果开头字符被包含在该token中，标记为B-
        is_begin = (s <= best[0] < e)
        labels.append(("B-" if is_begin else "I-") + "ADDRESS")
    return tokens, labels



# 选择非地址文本（优先用当前语言的 negatives 池；退化到内置若干短语）
def choose_non_address(lang: str, non_addr_map: Dict[str, List[str]]) -> str:
    pool = non_addr_map.get(lang, [])
    return random.choice(pool) if pool else ""



# ================== 组装样本 ==================

PLACEHOLDER_RE = re.compile(r'\{address\}')


# def make_sample_from_template(template: str,
#                               t_text: Optional[str],
#                               a_text: Optional[str]) -> Tuple[str, Optional[Tuple[int,int]], Optional[Tuple[int,int]]]:
#     """
#     用正则分段拼接，逐段累积长度来记录 {time}/{address} 的起止位置。
#     避免先 .replace() 再 .index() 引发的偏移/跨越问题；也避免多语标点导致的混淆。
#     若模板里占位符出现多次，只标注“第一处”实体（其他处视为普通文本），保证一条样本只有一个 TIME/ADDRESS。
#     """
#     parts = PLACEHOLDER_RE.split(template)  # 会保留分隔符
#     out_chunks: List[str] = []
#     time_span = None
#     addr_span = None

#     for seg in parts:
#         if seg == "{time}" and (t_text is not None):
#             start = sum(len(x) for x in out_chunks)
#             out_chunks.append(t_text)
#             if time_span is None:                # 只记录第一处
#                 time_span = (start, start + len(t_text))
#         elif seg == "{address}" and (a_text is not None):
#             start = sum(len(x) for x in out_chunks)
#             out_chunks.append(a_text)
#             if addr_span is None:                # 只记录第一处
#                 addr_span = (start, start + len(a_text))
#         else:
#             out_chunks.append(seg)

#     text = "".join(out_chunks)
#     return text, time_span, addr_span



# 将模板中的每个 {address} 依次替换为“地址”或“非地址”文本；
# 仅对“替换为地址”的片段记录 span，其余（非地址）不记录 -> 标为 O
def fill_template_with_addresses(template: str,
                                 lang: str,
                                 addrs_map: Dict[str, List[str]],
                                 non_addr_map: Dict[str, List[str]],   # CHANGED: 新增
                                 neg_prob: float = NEG_ADDR_PROB
                                 ) -> Tuple[str, List[Tuple[int,int]]]:
    addr_spans: List[Tuple[int,int]] = []
    out = []
    cursor = 0
    for m in PLACEHOLDER_RE.finditer(template):
        # 先加占位符之前的原文
        out.append(template[cursor:m.start()])
        # non_address修改处
        # 按比例选择正/负文本
        if random.random() < neg_prob:
            rep = choose_non_address(lang, non_addr_map)              # 负例：不应被识别为地址
            # 不记录 span -> 该段将被标为 O
        else:
            rep = choose_addr(lang, addrs_map)          # 正例：应标为 ADDRESS
            start = sum(len(s) for s in out)
            out.append(rep)
            addr_spans.append((start, start + len(rep)))
            cursor = m.end()
            continue
        out.append(rep)
        cursor = m.end()
    # 结尾剩余文本
    out.append(template[cursor:])
    text = "".join(out)
    return text, addr_spans



def strong_positive_from_entity(ent_text: str, ent_type: str, lang: str) -> Tuple[str, Optional[Tuple[int,int]], Optional[Tuple[int,int]]]:
    """
    构造几乎全实体的样本（只有 address 或只有 time）
    """
    if ent_type == "TIME":
        s = ent_text
        return s, (0, len(s)), None
    else:
        s = ent_text
        return s, None, (0, len(s))


def _compute_targets(addr_tpls: List[str], n_per_lang: Optional[int]) -> Tuple[int,int,bool]:
    """
    返回 (pos_target, neg_target, strict_once)
    - 若 n_per_lang 为空：pos_target = 模板条数；neg_target = pos * RATIO_NEG / (1 - RATIO_NEG)
      并置 strict_once=True（正例优先“一模板一条”）
    - 若 n_per_lang 给定：按配额计算，strict_once=False（允许随机采样模板补足）
    """
    if n_per_lang is None:
        pos = len(addr_tpls)
        denom = max(1e-9, (1.0 - RATIO_NEG))
        neg = int(round(pos * RATIO_NEG / denom))
        return pos, neg, True
    else:
        neg = int(n_per_lang * RATIO_NEG)
        pos = max(0, n_per_lang - neg)
        return pos, neg, False



# 构造主逻辑
def build_for_lang(lang: str,
                   templates: Dict[str, List[str]],
                   addrs_map: Dict[str, List[str]],
                   non_addr_map: Dict[str, List[str]],      # CHANGED: 新增
                   preview_n: int = 0,
                   n_per_lang: Optional[int] = None) -> List[Tuple[List[str],List[str]]]:

    samples_tok: List[Tuple[List[str],List[str]]] = []
    addr_tpls = templates.get("addr_only", []) or ["{address}"]
    neg_tpls  = templates.get("neg_sent", [])   or []  # 整句负例模板（来自 address_templates_0）

    # 统一计算目标条数
    if n_per_lang is None:
        pos_target = len(addr_tpls)
        denom = max(1e-9, (1.0 - RATIO_NEG))
        neg_target = int(round(pos_target * RATIO_NEG / denom))
        strict_once = True
    else:
        neg_target = int(n_per_lang * RATIO_NEG)
        pos_target = max(0, n_per_lang - neg_target)
        strict_once = False

    # === 正例：模板内部仍按 NEG_ADDR_PROB 注占位负片段 ===
    if strict_once and pos_target == len(addr_tpls):
        tpl_iter = addr_tpls
    else:
        tpl_iter = (random.choice(addr_tpls) for _ in range(pos_target))

    for tpl in tpl_iter:
        if "{address}" not in tpl:
            txt, _ = apply_noise(tpl, [], lang)
            tokens, labels = spans_to_bio_addresses(txt, [])
            samples_tok.append((tokens, labels))
            continue
        txt, addr_spans = fill_template_with_addresses(tpl, lang, addrs_map, non_addr_map, NEG_ADDR_PROB)
        txt, addr_spans = apply_noise(txt, addr_spans, lang)
        tokens, labels = spans_to_bio_addresses(txt, addr_spans)
        samples_tok.append((tokens, labels))


    # === 整句负例：优先用 address_templates_0 的模板；否则回退到 non_address.txt 直接成句 ===
    if neg_target > 0:
        if neg_tpls:
            # 用整句负例模板构造全 O 的句子
            for tpl in (random.choice(neg_tpls) for _ in range(neg_target)):
                s = fill_template_all_non_address(tpl, lang, non_addr_map)  # 全部用非地址替换
                s, _ = apply_noise(s, [], lang)
                tokens, labels = spans_to_bio_addresses(s, [])
                samples_tok.append((tokens, labels))
        else:
            print("""😿注意：没有找到整句负例模板，直接取非地址片段组成句子,可能影响质量
                    如需添加负例句子，请将上面的模板生成文件地址配置改为
                    ADDRESS_SLOT_LIST   = [0]
                    TARGET_PER_SLOT     = [300（请改成想要的负例句子数）] """)
            sys.exit(1)
    # 预览
    if preview_n > 0:
        print(f"\n[Preview {lang}] =====")
        for i in range(min(preview_n, len(samples_tok))):
            toks, labs = samples_tok[i]
            print("TOK:", " ".join(toks))
            print("LBL:", " ".join(labs))
    return samples_tok





# ================== 写 CONLL ==================
def write_conll(path: str, data: List[Tuple[List[str],List[str]]]):
    with open(path, "w", encoding="utf-8") as f:
        for toks, labs in data:
            for t, y in zip(toks, labs):
                f.write(f"{t} {y}\n")
            f.write("\n")

# ================== 主入口 ==================
def main():
    n_per_lang = int(N_PER_LANG) if N_PER_LANG else None
    preview = PREVIEW_N

    addrs_map = {lang: read_lines(os.path.join(REAL_ADDR_ROOT, f"{lang}.txt")) for lang in LANGS}
    non_addr_map = {lang: read_lines(os.path.join(NON_ADDR_ROOT,  f"{lang}.txt")) for lang in LANGS}

    merged = []
    for lang in LANGS:
        templates_per_lang = load_templates_for_lang(lang)
        data = build_for_lang(
            lang=lang,
            templates=templates_per_lang,
            addrs_map=addrs_map,
            non_addr_map=non_addr_map,
            preview_n=preview,
            n_per_lang=n_per_lang
        )
        write_conll(PER_LANG_CONLL(lang), data)
        merged.extend(data)
        print(f"[OK] {lang}: {len(data)} samples -> {PER_LANG_CONLL(lang)}")

    # 合并打乱
    random.shuffle(merged)
    write_conll(MERGED_CONLL, merged)
    print(f"\n[OK] merged: {len(merged)} samples -> {MERGED_CONLL}")




if __name__ == "__main__":
    random.seed()
    main()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
云盘挂载成功

[Preview 简体中文] =====
TOK: ▁ , 拜 访 客户的 地址 是 4 栋 , 之后 我们 去 燕 岗 吃饭 。
LBL: O O O O O O O O O O O O O B-ADDRESS I-ADDRESS O O
TOK: ▁/ 会议 已经 改 到 青海 湖 乡 二 郎 剑 村 26 组 115 号 进行 , 请 知 悉 。
LBL: O O O O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O O O O O O
TOK: ▁ 、 对 不起 , 您 提供的 耀 才 證券 暂时 无法 送 货 。
LBL: O O O O O O O B-ADDRESS I-ADDRESS I-ADDRESS O O O O O
TOK: ▁ 。 会议 将于 D ong chen ▁retail ▁of ▁machine ry 举行 , 请 从 联 旺 信息 科技 入口 进入 , 并在 藁 城区 廉 州 镇 第二 中学 签 到 。
LBL: O O O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O O O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O O O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O O O
TOK: ▁ , 会议 地点 已经 确定 在 曲 阳 街道 社 保 中心 , 请 准 时 参加 。
LBL: O O O O O O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O

In [ ]:
# @title 生成训练数据（CONLL）（混合模板） {"form-width":"40%"}
from google.colab import drive
import os, re, json, random, unicodedata, sys
from typing import List, Tuple, Dict, Optional
from transformers import AutoTokenizer

drive.mount('/content/drive')
print("云盘挂载成功")

# ======== 可修改配置 ========
N_PER_LANG = 500 # @param {"type":"raw","placeholder":"请输入每个语言的训练数据量，不输入默认遍历所有模板"}
RATIO_NEG = 0.15 # @param {"type":"number","placeholder":"整句负例占比（相对每语言总量）"}
NEG_ADDR_PROB = 0.2  # @param {"type":"number","placeholder":"插入地址负例的概率"}
NEG_TIME_PROB = 0.2  # @param {"type":"number","placeholder":"插入时间负例的概率"}
# 预览， >0 时打印样例
PREVIEW_N = 5  # @param {"type":"integer","placeholder":"请输入预览数据量"}

#数据比例控制
per_00=0.10# @param
per_11=0.30# @param

# 噪声概率
NOISE_PROB_CASE = 0.15
NOISE_PROB_FULLWIDTH = 0.10
NOISE_PROB_SEP_REPLACE = 0.10
NOISE_PROB_INSERT = 0.10
INSERT_CHARS = [",", ".", "·", "—", "-", "/", ":", "、", "・","?","？", "！", "!", "，", "。", "；", ";", " "]
RATIO_STRONG_POS = 0.10

LANGS = ["简体中文","繁体中文", "英语","丹麦语","俄语","土耳其语","德语","意大利语","日语",
         "法语","瑞典语","荷兰语","葡萄牙语","西班牙语","韩语"]

# ====== 新路径结构 ======
COMBINE_TPL_ROOT = "/content/drive/MyDrive/NER/combine_templates"
REAL_ADDR_ROOT = "/content/drive/MyDrive/NER/real_address"
REAL_TIME_ROOT = "/content/drive/MyDrive/NER/real_time"
NON_ADDR_ROOT  = "/content/drive/MyDrive/NER/non_address"
NON_TIME_ROOT  = "/content/drive/MyDrive/NER/non_time"

OUT_DIR = "/content/drive/MyDrive/NER/train_datasets_AC"
os.makedirs(OUT_DIR, exist_ok=True)
PER_LANG_CONLL = lambda lang: os.path.join(OUT_DIR, f"train_{lang}.conll")
MERGED_CONLL = os.path.join(OUT_DIR, "train_all.conll")

TOKENIZER_NAME = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME, use_fast=True)

# ===== 基础 IO =====
def read_lines(path: str) -> List[str]:
    if not os.path.exists(path):
        return []
    with open(path, "r", encoding="utf-8") as f:
        return [ln.strip() for ln in f if ln.strip()]

def read_jsonl_templates(path: str) -> list:
    items = []
    if not os.path.exists(path):
        return items
    with open(path, "r", encoding="utf-8") as f:
        for ln in f:
            ln = ln.strip()
            if not ln:
                continue
            try:
                obj = json.loads(ln)
                t = obj.get("template")
                if isinstance(t, str) and t.strip():
                    items.append(" ".join(t.split()))
            except Exception:
                continue
    return items

def load_templates_for_lang(lang: str) -> Dict[str, list]:
    # 分三类：正例11、正例其他、纯负例00
    pos_11_tpls, pos_other_tpls, neg_sent_tpls = [], [], []
    filename = f"{lang}_combine_templates.jsonl"
    for name in os.listdir(COMBINE_TPL_ROOT):
        if not name.startswith("combine_templates_"):
            continue
        subdir = os.path.join(COMBINE_TPL_ROOT, name)
        if not os.path.isdir(subdir):
            continue
        fpath = os.path.join(subdir, filename)
        items = read_jsonl_templates(fpath)
        if not items:
            continue
        slot = name[len("combine_templates_"):]
        if slot.isdigit():
            if int(slot) == 0:
                neg_sent_tpls.extend(items)     # 纯负例
            elif int(slot) == 11:
                pos_11_tpls.extend(items)       # 正例 11
            else:
                pos_other_tpls.extend(items)    # 正例 其他
    return {
        "pos_11": pos_11_tpls,
        "pos_other": pos_other_tpls,
        "neg_sent": neg_sent_tpls
    }

# ===== 噪声处理（保留原逻辑） =====
def toggle_case(s: str) -> str:
    def _flip(c):
        if "a" <= c <= "z": return c.upper()
        if "A" <= c <= "Z": return c.lower()
        return c
    return "".join(_flip(c) for c in s)

def to_fullwidth(c: str) -> str:
    code = ord(c)
    if 33 <= code <= 126:
        return chr(code + 0xFEE0)
    if c == " ":
        return "\u3000"
    return c

def to_halfwidth(c: str) -> str:
    if c == "\u3000": return " "
    code = ord(c)
    if 65281 <= code <= 65374:
        return chr(code - 0xFEE0)
    return c

def fullwidth_halfwidth_flip(s: str) -> str:
    out=[]
    for ch in s:
        if unicodedata.east_asian_width(ch) in ("F","W") and random.random()<0.5:
            out.append(to_halfwidth(ch))
        elif random.random() < 0.2:
            out.append(to_fullwidth(ch))
        else:
            out.append(ch)
    return "".join(out)

def replace_separators(s: str) -> str:
    trans = {
        "-": random.choice(["—","-","–"]),
        "/": random.choice(["/","／","∕"]),
        ".": random.choice([".","·","．"]),
        ":": random.choice([":","："])
    }
    return "".join(trans.get(ch, ch) for ch in s)

def insert_char_with_spans_multi(text, spans, p=None, ch=None):
    if not text: return text, spans
    if p is None:
      if random.random() < 0.5:   # 开头噪声概率50%
        p = 0
      else:
        p = random.randint(1, len(text))
    if ch is None:
        ch = random.choice(INSERT_CHARS)
    new_text = text[:p] + ch + text[p:]
    new_spans=[]
    for (s,e) in spans:
        if p < s:
            new_spans.append((s+1,e+1))
        elif p == s:  # 插在实体起点
            new_spans.append((s+1,e+1))
        elif s < p < e:
            new_spans.append((s,e+1))
        else:
            new_spans.append((s,e))
    return new_text, new_spans

def apply_noise(text: str, spans: List[Tuple[int,int]], lang: str) -> Tuple[str, List[Tuple[int,int]]]:
    latin_langs = {...}
    if lang in latin_langs and random.random() < NOISE_PROB_CASE:
        text = toggle_case(text)
    if random.random() < NOISE_PROB_FULLWIDTH:
        text = fullwidth_halfwidth_flip(text)
    if random.random() < NOISE_PROB_SEP_REPLACE:
        text = replace_separators(text)
    if random.random() < NOISE_PROB_INSERT:   # 只有命中概率才会插
        text, spans = insert_char_with_spans_multi(text, spans)
    return text, spans

# ===== BIO 标注（支持 ADDRESS 和 TIME） =====
def spans_to_bio_tokens(
    text: str,
    addr_spans: List[Tuple[int,int]],
    time_spans: List[Tuple[int,int]]
):
    # 可替换成你原来的 tokenizer
    enc = tokenizer(text, add_special_tokens=False, return_offsets_mapping=True)
    tokens = tokenizer.convert_ids_to_tokens(enc["input_ids"])
    offsets = enc["offset_mapping"]

    # 定义噪声字符集合（和INSERT_CHARS类似，但用于token层）
    NOISE_CHARS = set([
        "!", ".", ",", "·", "—", "-", "/", ":", "、", "・",
        "?", "？", "！", "，", "。", "；", ";",
        "(", ")", "[", "]", "\"", "\'", " "
    ])

    def is_noise_token(tok_text: str) -> bool:
        """
        判断一个分词后的token是否为纯噪声（去掉▁后只剩噪声字符）
        """
        stripped = tok_text.lstrip("▁")
        # 如果去掉起始的▁后，剩下的每个字符都在NOISE_CHARS里，说明是纯噪声
        return stripped != "" and all(ch in NOISE_CHARS for ch in stripped)

    def assign_bio(spans, label_type):
        res = ["O"] * len(tokens)
        for sp in spans:
            first_found = False
            for i, (tok_s, tok_e) in enumerate(offsets):
                # 判断token和span是否有重叠
                if tok_e <= sp[0] or tok_s >= sp[1]:
                    continue
                if not first_found:
                    tok_text = tokens[i]
                    if is_noise_token(tok_text):
                        # 纯噪声token，标为O，继续找下一个token做B
                        res[i] = "O"
                        continue
                    else:
                        res[i] = f"B-{label_type}"
                        first_found = True
                else:
                    res[i] = f"I-{label_type}"
        return res

    addr_labels = assign_bio(addr_spans, "ADDRESS")
    time_labels = assign_bio(time_spans, "TIME")

    final_labels = []
    for a_lbl, t_lbl in zip(addr_labels, time_labels):
        if a_lbl != "O" and t_lbl == "O":
            final_labels.append(a_lbl)
        elif t_lbl != "O" and a_lbl == "O":
            final_labels.append(t_lbl)
        else:
            final_labels.append("O")

    return tokens, final_labels

# ===== 占位符替换 =====
PLACEHOLDER_ADDR = re.compile(r'\{address\}')
PLACEHOLDER_TIME = re.compile(r'\{time\}')

def fill_template_with_entities(template: str, lang: str, addrs_map: Dict[str,List[str]], times_map: Dict[str,List[str]],
                                 non_addr_map: Dict[str,List[str]], non_time_map: Dict[str,List[str]]) -> Tuple[str, List[Tuple[int,int]], List[Tuple[int,int]]]:
    addr_spans=[]; time_spans=[]
    out=[]; cursor=0
    for m in re.finditer(r'\{address\}|\{time\}', template):
        out.append(template[cursor:m.start()])
        if m.group() == "{address}":
            if random.random() < NEG_ADDR_PROB:
                rep = random.choice(non_addr_map.get(lang, [""]))  # negative
            else:
                rep = random.choice(addrs_map.get(lang, ["Main Street"]))
                start=sum(len(s) for s in out)
                addr_spans.append((start, start+len(rep)))
            out.append(rep)
        elif m.group() == "{time}":
            if random.random() < NEG_TIME_PROB:
                rep = random.choice(non_time_map.get(lang, [""]))
            else:
                rep = random.choice(times_map.get(lang, ["2024-07-01"]))
                start=sum(len(s) for s in out)
                time_spans.append((start, start+len(rep)))
            out.append(rep)
        cursor=m.end()
    out.append(template[cursor:])
    return "".join(out), addr_spans, time_spans
#新建方法，用于修正标注对齐问题
import regex as re

def is_symbol(tok: str) -> bool:
    """判断 token 是否是纯符号或标点"""
    stripped = tok.replace('▁', '')
    return bool(re.fullmatch(r'[\p{P}\p{S}]+', stripped))

def merge_tokens_by_underscore(tokens, labels):
    """
    只合并字母或数字之间的拆分，不合并符号。
    保留符号及符号前空格原样。
    """
    merged_tokens = []
    merged_labels = []
    cur_tok = ""
    cur_lab = None

    for tok, lab in zip(tokens, labels):
        if tok is None:
            tok = ""
        elif not isinstance(tok, str):
            tok = str(tok)

        # 符号 或 新词起始（▁开头） → 另起一个 token
        if is_symbol(tok) or tok.startswith("▁"):
            # 如果之前在拼接单词，先保存
            if cur_tok:
                merged_tokens.append(cur_tok)
                merged_labels.append(cur_lab)
            # 新开 token
            cur_tok = tok
            cur_lab = lab
        else:
            # 普通常规 subword（单词内部拆分），合并
            cur_tok += tok
            cur_lab = cur_lab or lab  # 保留第一次的标签

    # 保存最后一个 token
    if cur_tok:
        merged_tokens.append(cur_tok)
        merged_labels.append(cur_lab)

    return merged_tokens, merged_labels


def fill_and_align_to_bio(tpl, lang, addrs_map, times_map, non_addr_map, non_time_map, add_noise=True):
    SPACE_LANGS = [

    ]
    NO_SPACE_LANGS = ["简体中文","繁体中文","日语","韩语","英语","法语","德语","意大利语","西班牙语","葡萄牙语",
        "荷兰语","瑞典语","丹麦语","俄语","土耳其语"]

    # 占位符替换
    parts = []
    entities = []  # (start_char, end_char, label_type)
    last_pos = 0
    for m in re.finditer(r'\{address\}|\{time\}', tpl):
        parts.append(tpl[last_pos:m.start()])
        if m.group() == "{address}":
            if random.random() < NEG_ADDR_PROB:
                rep = random.choice(non_addr_map.get(lang, [""]))
            else:
                rep = random.choice(addrs_map.get(lang, ["Main Street"]))
                start_char = sum(len(p) for p in parts)
                entities.append((start_char, start_char + len(rep), "ADDRESS"))
            parts.append(rep)
        elif m.group() == "{time}":
            if random.random() < NEG_TIME_PROB:
                rep = random.choice(non_time_map.get(lang, [""]))
            else:
                rep = random.choice(times_map.get(lang, ["2024-07-01"]))
                start_char = sum(len(p) for p in parts)
                entities.append((start_char, start_char + len(rep), "TIME"))
            parts.append(rep)
        last_pos = m.end()
    parts.append(tpl[last_pos:])
    text = "".join(parts)

    # 噪声处理
    if add_noise:
        text, new_spans = apply_noise(text, [(s, e) for s, e, _ in entities], lang)
        for i, (_, _, typ) in enumerate(entities):
            entities[i] = (new_spans[i][0], new_spans[i][1], typ)

    # 分词 & BIO对齐
    enc = tokenizer(text, add_special_tokens=False, return_offsets_mapping=True)
    tokens = tokenizer.convert_ids_to_tokens(enc["input_ids"])
    offsets = enc["offset_mapping"]

    labels = ["O"] * len(tokens)
    for start_char, end_char, ent_type in entities:
        first_flag = True
        for i, (tok_s, tok_e) in enumerate(offsets):
            if tok_e <= start_char or tok_s >= end_char:
                continue
            if first_flag:
                labels[i] = f"B-{ent_type}"
                first_flag = False
            else:
                labels[i] = f"I-{ent_type}"

    # === 按▁合并为单词（仅空格语言）
    if lang in SPACE_LANGS:
        tokens, labels = merge_tokens_by_underscore(tokens, labels)

    return tokens, labels
# ===== 主构建 =====
def build_for_lang(lang: str, templates: Dict[str, list],
                   addrs_map, times_map, non_addr_map, non_time_map,
                   preview_n=0, n_per_lang=None):
    samples = []

    # 数据量控制
    total_target = n_per_lang if n_per_lang else len(templates.get("pos_11", [])) + len(templates.get("pos_other", [])) + len(templates.get("neg_sent", []))

    # 纯负例
    neg_target = int(total_target * (per_00))
    # 正例总量
    pos_target_total = total_target - neg_target
    # 正例中 11
    pos_11_target = int(pos_target_total * (per_11))
    pos_other_target = pos_target_total - pos_11_target

    pos_11_tpls = templates.get("pos_11", []) or ["{address} {time}"]
    pos_other_tpls = templates.get("pos_other", []) or ["{address} {time}"]
    neg_tpls = templates.get("neg_sent", []) or []

    # 正例11
    for _ in range(pos_11_target):
        tpl = random.choice(pos_11_tpls)
        toks, lbls = fill_and_align_to_bio(tpl, lang, addrs_map, times_map,
                                           non_addr_map, non_time_map, add_noise=True)
        samples.append((toks, lbls))

    # 正例其他
    for _ in range(pos_other_target):
        tpl = random.choice(pos_other_tpls)
        toks, lbls = fill_and_align_to_bio(tpl, lang, addrs_map, times_map,
                                           non_addr_map, non_time_map, add_noise=True)
        samples.append((toks, lbls))

    # 负例
    for _ in range(neg_target):
        if neg_tpls:
            tpl = random.choice(neg_tpls)
            toks, lbls = spans_to_bio_tokens(tpl, [], [])  # 全 O 标签
            samples.append((toks, lbls))

    # 预览
    if preview_n > 0:
        print(f"[Preview {lang}]")
        for i in range(min(preview_n, len(samples))):
            print("TOK:", " ".join(samples[i][0]))
            print("LBL:", " ".join(samples[i][1]))

    return samples
# ===== 写 CONLL =====
def write_conll(path, data):
    """将 (tokens, labels) 样本列表写入到 CONLL 格式文件"""
    with open(path, "w", encoding="utf-8") as f:
        for toks, lbls in data:
            for t, y in zip(toks, lbls):
                f.write(f"{t} {y}\n")
            f.write("\n")
# ===== 主入口 =====
def main():
    n_per_lang = N_PER_LANG if N_PER_LANG else None
    addrs_map={lang: read_lines(os.path.join(REAL_ADDR_ROOT,f"{lang}.txt")) for lang in LANGS}
    times_map={lang: read_lines(os.path.join(REAL_TIME_ROOT,f"{lang}.txt")) for lang in LANGS}
    non_addr_map={lang: read_lines(os.path.join(NON_ADDR_ROOT,f"{lang}.txt")) for lang in LANGS}
    non_time_map={lang: read_lines(os.path.join(NON_TIME_ROOT,f"{lang}.txt")) for lang in LANGS}
    merged=[]
    for lang in LANGS:
        templates_per_lang = load_templates_for_lang(lang)
        data = build_for_lang(lang, templates_per_lang, addrs_map, times_map, non_addr_map, non_time_map, preview_n=PREVIEW_N, n_per_lang=n_per_lang)
        write_conll(PER_LANG_CONLL(lang), data)
        merged.extend(data)
        print(f"[OK] {lang}: {len(data)} samples -> {PER_LANG_CONLL(lang)}")
    random.shuffle(merged)
    write_conll(MERGED_CONLL, merged)
    print(f"[OK] merged: {len(merged)} samples -> {MERGED_CONLL}")

main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
云盘挂载成功
[Preview 简体中文]
TOK: ▁ 七 月中旬 的一个 周五 下午 13 点 整 我们会 在 A 15 举办 活动 , 期待 你 的到来 !
LBL: B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME O O B-ADDRESS I-ADDRESS O O O O O O O
TOK: ▁1974 年 10 月 4 日 我和 同事 会在 大 万 世 居 进行 一次 重要的 会议 。
LBL: B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME O O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O O O O O
TOK: ▁我们 相 约 今年 国庆 节 下午 四 点 四 十五 分 在 12 # 见面 , 一起 探讨 未来的 合作 计划 。
LBL: O O O B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME O O O O O O O O O O O
TOK: ▁今年 冬 至 凌晨 三 点 我会 去 成都 高新技术 产业 开发区 ▁I 9 厂 房 与 老 朋友 碰 面 , 聊 聊 合作 事宜 。
LBL: B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O O O O O O O O O O O
TOK: ▁我 喜欢 十 月 5 日下午 6 点 整 去 陶 利 庙 南 散步 , 感受 自然 的 气息 。
LBL: O O B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME O B-ADDRESS I-ADDRESS I-A

#### 合成测试数据（json）
输入输出说明：


*   输入

*   输出





In [ ]:
import os
import json
import random
from typing import List, Dict, Iterable
from itertools import cycle

# ========= 配置 =========
ADDRESSES_DIR = "/content/drive/MyDrive/NER/real_address"
TIMES_DIR     = "/content/drive/MyDrive/NER/real_time"   # 新增：真实时间文件目录
TEMPLATES_DIR = "/content/drive/MyDrive/NER/combine_templates/combine_templates_22"

N_address = int(TEMPLATES_DIR[-1])  # 每条句子包含多少个地址
N_time    = 2                       # 每条句子包含多少个时间（可改）

OUTPUT_DIR    = os.path.join("./templates_out", str(N_address))
OUTPUT_NAME   = "{lang}.json"

LANGUAGES = [
    "简体中文", "英语", "丹麦语", "俄语", "土耳其语", "德语",
    "意大利语", "日语", "法语", "瑞典语", "荷兰语", "葡萄牙语",
    "西班牙语", "韩语",
]

# ========= 工具函数 =========
def ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

def read_list_from_txt(path: str) -> List[str]:
    items = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if s:
                items.append(s)
    random.shuffle(items)
    return items

def find_template_file(templates_dir: str, lang: str) -> str:
    prefix = f"{lang}_combine_templates"
    for fn in os.listdir(templates_dir):
        if fn.startswith(prefix):
            return os.path.join(templates_dir, fn)
    raise FileNotFoundError(f"未找到模板文件：{prefix}* 于 {templates_dir}")

def iter_templates_from_jsonl(path: str):
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
                t = obj.get("template", "")
                if isinstance(t, str) and t:
                    yield t
            except Exception as e:
                print(f"[WARN] JSONL 解析失败 {path}:{i} | {e}")

def read_templates(path: str) -> List[str]:
    templates: List[str] = []
    try:
        for t in iter_templates_from_jsonl(path):
            templates.append(t)
        if templates:
            return list(dict.fromkeys(templates))
    except Exception:
        pass

    with open(path, "r", encoding="utf-8") as f:
        content = f.read().strip()
    data = json.loads(content)
    out = []
    if isinstance(data, list):
        for item in data:
            if isinstance(item, dict):
                t = item.get("template", "")
                if isinstance(t, str) and t:
                    out.append(t)
    elif isinstance(data, dict):
        t = data.get("template", "")
        if isinstance(t, str) and t:
            out.append(t)
    else:
        raise ValueError(f"不支持的模板 JSON 结构：{path}")
    return list(dict.fromkeys(out))

def embed_addresses_and_times(template: str, address_list: List[str], time_list: List[str]) -> str:
    result = template
    for t in time_list:
        if "{time}" in result:
            result = result.replace("{time}", t, 1)
        else:
            break
    for addr in address_list:
        if "{address}" in result:
            result = result.replace("{address}", addr, 1)
        else:
            break
    return result

def write_json_array(path: str, records: List[Dict]):
    ensure_dir(os.path.dirname(path))
    with open(path, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)
    print(f"[OK] 写出：{path} ({len(records)} 条)")

# ========= 主流程 =========
def main():
    ensure_dir(OUTPUT_DIR)

    for lang in LANGUAGES:
        addr_file = os.path.join(ADDRESSES_DIR, f"{lang}.txt")
        time_file = os.path.join(TIMES_DIR,     f"{lang}.txt")

        if not os.path.isfile(addr_file):
            print(f"[SKIP] 未找到地址文件：{addr_file}")
            continue
        if not os.path.isfile(time_file):
            print(f"[SKIP] 未找到时间文件：{time_file}")
            continue

        addresses = read_list_from_txt(addr_file)
        times     = read_list_from_txt(time_file)
        if not addresses:
            print(f"[SKIP] 地址为空：{addr_file}")
            continue
        if not times:
            print(f"[SKIP] 时间为空：{time_file}")
            continue

        try:
            tmpl_path = find_template_file(TEMPLATES_DIR, lang)
        except FileNotFoundError as e:
            print(f"[SKIP] {e}")
            continue
        templates = read_templates(tmpl_path)
        if not templates:
            print(f"[SKIP] 模板为空：{tmpl_path}")
            continue

        tmpl_iter = cycle(templates)
        records: List[Dict] = []

        # 按地址数推进，时间也循环
        max_len = min(len(addresses) // N_address, len(times) // N_time) * N_address
        for idx in range(0, max_len, N_address):
            addr_list = addresses[idx: idx + N_address]
            time_idx  = (idx // N_address) * N_time
            time_list = times[time_idx: time_idx + N_time]

            templ = next(tmpl_iter)
            sentence = embed_addresses_and_times(templ, addr_list, time_list)

            records.append({
                "id": idx // N_address + 1,
                "sentence": sentence,
                "time_tokens": time_list,
                "location_tokens": addr_list
            })

        out_file = os.path.join(OUTPUT_DIR, OUTPUT_NAME.format(lang=lang))
        write_json_array(out_file, records)

    print("全部完成。输出目录：", OUTPUT_DIR)

if __name__ == "__main__":
    main()

[OK] 写出：./templates_out/2/简体中文.json (1900 条)
[OK] 写出：./templates_out/2/英语.json (3200 条)
[OK] 写出：./templates_out/2/丹麦语.json (2000 条)
[OK] 写出：./templates_out/2/俄语.json (2300 条)
[OK] 写出：./templates_out/2/土耳其语.json (1999 条)
[OK] 写出：./templates_out/2/德语.json (2200 条)
[OK] 写出：./templates_out/2/意大利语.json (2000 条)
[OK] 写出：./templates_out/2/日语.json (1900 条)
[OK] 写出：./templates_out/2/法语.json (2100 条)
[OK] 写出：./templates_out/2/瑞典语.json (2000 条)
[OK] 写出：./templates_out/2/荷兰语.json (1547 条)
[OK] 写出：./templates_out/2/葡萄牙语.json (2100 条)
[OK] 写出：./templates_out/2/西班牙语.json (2100 条)
[OK] 写出：./templates_out/2/韩语.json (1900 条)
全部完成。输出目录： ./templates_out/2


### **训练模型**

**参数设置说明**（可直接在代码右侧的空格处设置，或直接使用默认值）：
*   DATA_PATH: 数据集路径（conll格式，可运行本笔记本中的数据构造部分得到）
*   OUTPUT_DIR: 模型输出路径，如果没有保存到云盘则需要在长时间断开运行时前将模型下载或移动到云盘（移动代码见本笔记本的“文件操作”部分）
*   EPOCHS: 训练轮数
*   BATCH: 每次加载的批次大小
*   LR: 学习率
*   WARMUP: 学习率预热率

**参数设置策略：**


*   **EPOCHS:**

    * 理想情况：训练集与验证集的 loss 先下降，末期趋于平稳。

    * 若两者到最后仍持续下降 → 可能未收敛，可适当增大 EPOCHS。

    * 若训练集 loss 继续降、验证集先降后升 → 过拟合；通常应减小 EPOCHS或启用早停，不过由于代码里设置了metric_for_best_model='f1',会自动保存f1分数最高的模型，所以不太会影响最后的模型质量。

*   **BATCH**:
    *  最好是2的n次方，值越大训练得越快，但是不能超出gpu的显存限制，可以在训练时点击右上角的“RAM 磁盘”查看gpu ram的占用情况，如果离最大容量距离较大可以尝试增大BATCH SIZE来加速。

    *  建议遵循线性缩放法则：当 batch size 增大时，等比例增大学习率 LR（例如 batch×2 → LR×≈2）
*   **LR:**
    * 值越大模型收敛得越快。
    * 过大：训练/验证loss波动；
    * 过小：loss 下降缓慢；
*   **WARMUP:**
    * 含义： 前 warmup 比例的 step（一次参数更新为 1 step）内，将 LR 从 0 线性升到目标 LR，以避免冷启动震荡。
    * 训练初期 loss 大幅波动 → warmup 太短；
    * 初期 loss 下降过慢 → warmup 太长。


In [ ]:
# @title 训练代码 {"form-width":"35%"}

# Train XLM-R NER ConLL
!pip -q install -U transformers datasets evaluate seqeval accelerate

import os, numpy as np, torch, random
from dataclasses import dataclass
from typing import List, Dict, Tuple
from datasets import Dataset
import evaluate
from packaging import version
import transformers
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
from transformers import (
    AutoTokenizer, AutoConfig, AutoModelForTokenClassification,
    DataCollatorForTokenClassification, Trainer, TrainingArguments, set_seed
)

#  Config
# 训练集路径
DATA_PATH = "/content/drive/MyDrive/NER/train_datasets/train_all.conll" # @param {"type":"string","placeholder":"请输入数据集路径"}
OUTPUT_DIR = "/content/ner_xlmr_out" # @param {"type":"string","placeholder":"请输入模型保存路径"}


# 训练轮数（过大会导致过拟合）
EPOCHS = 3 # @param {"type":"integer"}

# batch size 256对应的是A100GPU，若选用显存较小的gpu请适当调小，防止cuda out of memory
# 如果调小batch size请同时把LR（学习率）也适当调小。
BATCH = 64 # @param {"type":"integer", "placeholder":"请训练轮数，默认为3"}
LR = 1.25e-5   # @param {"type":"number", "placeholder":"请输入学习率，默认为5e-5"}

# 学习率预热，在初期使用较小学习率，防止震荡或梯度爆炸
WARMUP = 0.1 # @param {"type":"number", "placeholder":"请输入学习率预热率，默认为0.1"}


PRETOKENIZED = True     # 已是XLM-R子词选True；还是是原词级未分词选False
MODEL_NAME = "xlm-roberta-base"
SEED = 42
set_seed(SEED)
os.makedirs(OUTPUT_DIR, exist_ok=True)

#  Labels
LABELS = ["O","B-ADDRESS", "I-ADDRESS"]
label2id = {l:i for i,l in enumerate(LABELS)}
id2label = {i:l for l,i in label2id.items()}

#  Read CoNLL
def read_conll(path: str):
    sents_tok, sents_lab = [], []
    toks, labs = [], []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line:
                if toks:
                    assert len(toks)==len(labs), (toks, labs)
                    sents_tok.append(toks); sents_lab.append(labs)
                    toks, labs = [], []
                continue
            if " " in line:
                t, y = line.split(" ", 1)
            else:
                t, y = line, "O"
            if y not in label2id: y = "O"
            toks.append(t); labs.append(y)
    if toks:
        sents_tok.append(toks); sents_lab.append(labs)
    return sents_tok, sents_lab

tokens_all, tags_all = read_conll(DATA_PATH)
print(f"Loaded {len(tokens_all)} sentences. Example tokens[:20]:", tokens_all[0][:20])
print("Example labels[:20]:", tags_all[0][:20])

#  Train/Val split
from sklearn.model_selection import train_test_split
idx = np.arange(len(tokens_all))
tr_idx, va_idx = train_test_split(idx, test_size=0.1, random_state=SEED, shuffle=True)
def pick(idxs):
    return [tokens_all[i] for i in idxs], [tags_all[i] for i in idxs]
train_tokens, train_tags = pick(tr_idx)
val_tokens, val_tags = pick(va_idx)

#  Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

#  Path A: PRETOKENIZED = True (直接映射到 vocab id) -
def build_dataset_pretokenized(tokens_list, tags_list) -> Dataset:
    CLS_ID = tokenizer.convert_tokens_to_ids(tokenizer.cls_token)  # <s>
    SEP_ID = tokenizer.convert_tokens_to_ids(tokenizer.sep_token)  # </s>
    UNK_ID = tokenizer.unk_token_id
    PAD_ID = tokenizer.pad_token_id

    recs = []
    for toks, labs in zip(tokens_list, tags_list):
        ids = tokenizer.convert_tokens_to_ids(toks)
        ids = [i if i is not None else UNK_ID for i in ids]
        input_ids = [CLS_ID] + ids + [SEP_ID]
        labels_ids = [-100] + [label2id[y] for y in labs] + [-100]
        attn = [1]*len(input_ids)
        recs.append({"input_ids": input_ids, "labels": labels_ids, "attention_mask": attn})
    return Dataset.from_list(recs)

#  Path B: PRETOKENIZED = False（原词级 -> 重新分词并对齐） -
def build_dataset_wordlevel(tokens_list, tags_list) -> Dataset:
    def _to_records(toks_list, labs_list):
        for t, y in zip(toks_list, labs_list):
            yield {"tokens": t, "ner_tags": y}
    ds = Dataset.from_list(list(_to_records(tokens_list, tags_list)))
    def tokenize_and_align(examples):
        tokenized = tokenizer(
            examples["tokens"], is_split_into_words=True,
            truncation=True, padding=False, max_length=256
        )
        aligned = []
        for i, labs in enumerate(examples["ner_tags"]):
            word_ids = tokenized.word_ids(i)
            prev = None; out = []
            for wid in word_ids:
                if wid is None:
                    out.append(-100)
                elif wid != prev:
                    out.append(label2id[labs[wid]])
                else:
                    lab = labs[wid]
                    if lab.startswith("B-"): lab = "I-"+lab[2:]
                    out.append(label2id[lab])
                prev = wid
            aligned.append(out)
        tokenized["labels"] = aligned
        return tokenized
    return ds.map(tokenize_and_align, batched=True)

if PRETOKENIZED:
    ds_train = build_dataset_pretokenized(train_tokens, train_tags)
    ds_val   = build_dataset_pretokenized(val_tokens,   val_tags)

    # 手写collator做padding
    from dataclasses import dataclass
    @dataclass
    class CollatorPT:
        pad_id: int = tokenizer.pad_token_id
        def __call__(self, batch):
            maxlen = max(len(x["input_ids"]) for x in batch)
            input_ids, attn, labels = [], [], []
            for x in batch:
                pad = maxlen - len(x["input_ids"])
                input_ids.append(x["input_ids"] + [self.pad_id]*pad)
                attn.append(x["attention_mask"] + [0]*pad)
                labels.append(x["labels"] + [-100]*pad)
            return {
                "input_ids": torch.tensor(input_ids),
                "attention_mask": torch.tensor(attn),
                "labels": torch.tensor(labels)
            }
    data_collator = CollatorPT()
else:
    ds_train = build_dataset_wordlevel(train_tokens, train_tags)
    ds_val   = build_dataset_wordlevel(val_tokens,   val_tags)
    from transformers import DataCollatorForTokenClassification
    data_collator = DataCollatorForTokenClassification(tokenizer)

#  Model
config = AutoConfig.from_pretrained(
    MODEL_NAME, num_labels=len(LABELS), id2label=id2label, label2id=label2id
)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, config=config)


#  Metrics
seqeval = evaluate.load("seqeval")
def compute_metrics(p):
    logits, labels = p
    preds = logits.argmax(-1)
    true_preds, true_labels = [], []
    for pred, lab in zip(preds, labels):
        cur_p, cur_l = [], []
        for pi, li in zip(pred, lab):
            if li == -100: continue
            cur_p.append(id2label[int(pi)]); cur_l.append(id2label[int(li)])
        true_preds.append(cur_p); true_labels.append(cur_l)
    res = seqeval.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": res["overall_precision"],
        "recall":    res["overall_recall"],
        "f1":        res["overall_f1"],
        "accuracy":  res["overall_accuracy"],
    }

#  Training
common_kwargs = dict(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    learning_rate=LR,
    warmup_ratio=WARMUP,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
    fp16=torch.cuda.is_available(),
    seed=SEED,
)

ver = version.parse(transformers.__version__)
try:
    if ver >= version.parse("4.55.0"):
        # 新版（>=4.55）优先使用 eval_strategy
        args = TrainingArguments(**common_kwargs, eval_strategy="epoch")
    else:
        # 旧版使用 evaluation_strategy
        args = TrainingArguments(**common_kwargs, evaluation_strategy="epoch")
except TypeError:
    # 极旧版本：先不传参创建，再在 Trainer 里设置
    args = TrainingArguments(**common_kwargs)



trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 统一设置评估策略
try:
    # 4.55+ 新API
    trainer.set_evaluate(strategy="epoch")
except Exception:
    try:
        # 通用老API
        from transformers.trainer_utils import IntervalStrategy
        trainer.args.evaluation_strategy = IntervalStrategy.EPOCH
    except Exception:
        # 字符串兜底
        trainer.args.evaluation_strategy = "epoch"


trainer.train()
print("== Dev metrics ==", trainer.evaluate())

trainer.save_model(OUTPUT_DIR); tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved to:", OUTPUT_DIR)


KeyboardInterrupt: 

In [ ]:
# @title 训练代码（兼容时间地点） {"form-width":"35%"}
!pip install "pyarrow>=14.0.0,<20.0.0" --force-reinstall
# Train XLM-R NER ConLL
!pip -q install -U transformers datasets evaluate seqeval accelerate

import os, numpy as np, torch, random
from dataclasses import dataclass
from typing import List, Dict, Tuple
from datasets import Dataset
import evaluate
from packaging import version
import transformers
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
from transformers import (
    AutoTokenizer, AutoConfig, AutoModelForTokenClassification,
    DataCollatorForTokenClassification, Trainer, TrainingArguments, set_seed
)

#  Config
# 训练集路径
DATA_PATH = "/content/drive/MyDrive/NER/train_datasets_AC/train_all.conll" # @param {"type":"string","placeholder":"请输入数据集路径"}
OUTPUT_DIR = "/content/ner_xlmr_out" # @param {"type":"string","placeholder":"请输入模型保存路径"}


# 训练轮数（过大会导致过拟合）
EPOCHS = 3 # @param {"type":"integer"}

# batch size 256对应的是A100GPU，若选用显存较小的gpu请适当调小，防止cuda out of memory
# 如果调小batch size请同时把LR（学习率）也适当调小。
BATCH = 128 # @param {"type":"integer", "placeholder":"请训练轮数，默认为3"}
LR = 2.5e-5   # @param {"type":"number", "placeholder":"请输入学习率，默认为5e-5"}

# 学习率预热，在初期使用较小学习率，防止震荡或梯度爆炸
WARMUP = 0.1 # @param {"type":"number", "placeholder":"请输入学习率预热率，默认为0.1"}


PRETOKENIZED = True     # 已是XLM-R子词选True；还是是原词级未分词选False
MODEL_NAME = "xlm-roberta-base"
SEED = 42
set_seed(SEED)
os.makedirs(OUTPUT_DIR, exist_ok=True)

#  Labels
LABELS = ["O", "B-ADDRESS", "I-ADDRESS", "B-TIME", "I-TIME"]
label2id = {l: i for i, l in enumerate(LABELS)}
id2label = {i: l for l, i in label2id.items()}

#  Read CoNLL
def read_conll(path: str):
    sents_tok, sents_lab = [], []
    toks, labs = [], []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line:
                if toks:
                    assert len(toks)==len(labs), (toks, labs)
                    sents_tok.append(toks); sents_lab.append(labs)
                    toks, labs = [], []
                continue
            if " " in line:
                t, y = line.split(" ", 1)
            else:
                t, y = line, "O"
            if y not in label2id: y = "O"
            toks.append(t); labs.append(y)
    if toks:
        sents_tok.append(toks); sents_lab.append(labs)
    return sents_tok, sents_lab

tokens_all, tags_all = read_conll(DATA_PATH)
print(f"Loaded {len(tokens_all)} sentences. Example tokens[:20]:", tokens_all[0][:20])
print("Example labels[:20]:", tags_all[0][:20])

#  Train/Val split
from sklearn.model_selection import train_test_split
idx = np.arange(len(tokens_all))
tr_idx, va_idx = train_test_split(idx, test_size=0.1, random_state=SEED, shuffle=True)
def pick(idxs):
    return [tokens_all[i] for i in idxs], [tags_all[i] for i in idxs]
train_tokens, train_tags = pick(tr_idx)
val_tokens, val_tags = pick(va_idx)

#  Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

#  Path A: PRETOKENIZED = True (直接映射到 vocab id) -
def build_dataset_pretokenized(tokens_list, tags_list) -> Dataset:
    CLS_ID = tokenizer.convert_tokens_to_ids(tokenizer.cls_token)  # <s>
    SEP_ID = tokenizer.convert_tokens_to_ids(tokenizer.sep_token)  # </s>
    UNK_ID = tokenizer.unk_token_id
    PAD_ID = tokenizer.pad_token_id

    recs = []
    for toks, labs in zip(tokens_list, tags_list):
        ids = tokenizer.convert_tokens_to_ids(toks)
        ids = [i if i is not None else UNK_ID for i in ids]
        input_ids = [CLS_ID] + ids + [SEP_ID]
        labels_ids = [-100] + [label2id[y] for y in labs] + [-100]
        attn = [1]*len(input_ids)
        recs.append({"input_ids": input_ids, "labels": labels_ids, "attention_mask": attn})
    return Dataset.from_list(recs)

#  Path B: PRETOKENIZED = False（原词级 -> 重新分词并对齐） -
def build_dataset_wordlevel(tokens_list, tags_list) -> Dataset:
    def _to_records(toks_list, labs_list):
        for t, y in zip(toks_list, labs_list):
            yield {"tokens": t, "ner_tags": y}
    ds = Dataset.from_list(list(_to_records(tokens_list, tags_list)))
    def tokenize_and_align(examples):
        tokenized = tokenizer(
            examples["tokens"], is_split_into_words=True,
            truncation=True, padding=False, max_length=256
        )
        aligned = []
        for i, labs in enumerate(examples["ner_tags"]):
            word_ids = tokenized.word_ids(i)
            prev = None; out = []
            for wid in word_ids:
                if wid is None:
                    out.append(-100)
                elif wid != prev:
                    out.append(label2id[labs[wid]])
                else:
                    lab = labs[wid]
                    if lab.startswith("B-"): lab = "I-"+lab[2:]
                    out.append(label2id[lab])
                prev = wid
            aligned.append(out)
        tokenized["labels"] = aligned
        return tokenized
    return ds.map(tokenize_and_align, batched=True)

if PRETOKENIZED:
    ds_train = build_dataset_pretokenized(train_tokens, train_tags)
    ds_val   = build_dataset_pretokenized(val_tokens,   val_tags)

    # 手写collator做padding
    from dataclasses import dataclass
    @dataclass
    class CollatorPT:
        pad_id: int = tokenizer.pad_token_id
        def __call__(self, batch):
            maxlen = max(len(x["input_ids"]) for x in batch)
            input_ids, attn, labels = [], [], []
            for x in batch:
                pad = maxlen - len(x["input_ids"])
                input_ids.append(x["input_ids"] + [self.pad_id]*pad)
                attn.append(x["attention_mask"] + [0]*pad)
                labels.append(x["labels"] + [-100]*pad)
            return {
                "input_ids": torch.tensor(input_ids),
                "attention_mask": torch.tensor(attn),
                "labels": torch.tensor(labels)
            }
    data_collator = CollatorPT()
else:
    ds_train = build_dataset_wordlevel(train_tokens, train_tags)
    ds_val   = build_dataset_wordlevel(val_tokens,   val_tags)
    from transformers import DataCollatorForTokenClassification
    data_collator = DataCollatorForTokenClassification(tokenizer)

#  Model
config = AutoConfig.from_pretrained(
    MODEL_NAME, num_labels=len(LABELS), id2label=id2label, label2id=label2id
)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, config=config)


#  Metrics
seqeval = evaluate.load("seqeval")
def compute_metrics(p):
    logits, labels = p
    preds = logits.argmax(-1)
    true_preds, true_labels = [], []
    for pred, lab in zip(preds, labels):
        cur_p, cur_l = [], []
        for pi, li in zip(pred, lab):
            if li == -100: continue
            cur_p.append(id2label[int(pi)]); cur_l.append(id2label[int(li)])
        true_preds.append(cur_p); true_labels.append(cur_l)
    res = seqeval.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": res["overall_precision"],
        "recall":    res["overall_recall"],
        "f1":        res["overall_f1"],
        "accuracy":  res["overall_accuracy"],
    }

#  Training
common_kwargs = dict(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    learning_rate=LR,
    warmup_ratio=WARMUP,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
    fp16=torch.cuda.is_available(),
    seed=SEED,
)

ver = version.parse(transformers.__version__)
try:
    if ver >= version.parse("4.55.0"):
        # 新版（>=4.55）优先使用 eval_strategy
        args = TrainingArguments(**common_kwargs, eval_strategy="epoch")
    else:
        # 旧版使用 evaluation_strategy
        args = TrainingArguments(**common_kwargs, evaluation_strategy="epoch")
except TypeError:
    # 极旧版本：先不传参创建，再在 Trainer 里设置
    args = TrainingArguments(**common_kwargs)



trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 统一设置评估策略
try:
    # 4.55+ 新API
    trainer.set_evaluate(strategy="epoch")
except Exception:
    try:
        # 通用老API
        from transformers.trainer_utils import IntervalStrategy
        trainer.args.evaluation_strategy = IntervalStrategy.EPOCH
    except Exception:
        # 字符串兜底
        trainer.args.evaluation_strategy = "epoch"


trainer.train()
print("== Dev metrics ==", trainer.evaluate())

trainer.save_model(OUTPUT_DIR); tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved to:", OUTPUT_DIR)


  Using cached pyarrow-19.0.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-19.0.1-cp312-cp312-manylinux_2_28_x86_64.whl (42.1 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 22.0.0
    Uninstalling pyarrow-22.0.0:
      Successfully uninstalled pyarrow-22.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.3.0 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have py

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/tmp/ipython-input-511366454.py:224: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.663400,0.064529,0.890069,0.916988,0.903328,0.985633
2,0.045400,0.028117,0.951469,0.958816,0.955128,0.992606
3,0.022800,0.026332,0.955272,0.962033,0.958641,0.993589


== Dev metrics == {'eval_loss': 0.026332031935453415, 'eval_precision': 0.9552715654952076, 'eval_recall': 0.9620334620334621, 'eval_f1': 0.9586405899326707, 'eval_accuracy': 0.993588543616623, 'eval_runtime': 0.9889, 'eval_samples_per_second': 758.448, 'eval_steps_per_second': 6.068, 'epoch': 3.0}
Saved to: /content/ner_xlmr_out


##### **在设置好的conll格式验证集下验证CheckPoints下模型效果**

In [ ]:
# Correct inference for PRETOKENIZED XLM-R tokens
import torch, random, numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification

MODEL_DIR = "/content/drive/MyDrive/NER/CheckPoints/checkpoint-6330_c2"      # 最新checkpoint根目录
DATA_PATH = "/content/drive/MyDrive/NER/train_datasets/train_all.conll"   # 或验证集conll路径

LABELS = ["O", "B-ADDRESS", "I-ADDRESS", "B-TIME", "I-TIME"]
#LABELS = ["O", "B-ADDRESS", "I-ADDRESS"]
label2id = {l:i for i,l in enumerate(LABELS)}
id2label = {i:l for l,i in label2id.items()}

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, use_fast=True)
model = AutoModelForTokenClassification.from_pretrained(MODEL_DIR)
model.eval()

def read_conll(path):
    sents_tok, sents_lab, toks, labs = [], [], [], []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line:
                if toks:
                    sents_tok.append(toks); sents_lab.append(labs)
                    toks, labs = [], []
                continue
            if " " in line:
                t, y = line.split(" ", 1)
            else:
                t, y = line, "O"
            if y not in label2id: y = "O"
            toks.append(t); labs.append(y)
    if toks:
        sents_tok.append(toks); sents_lab.append(labs)
    return sents_tok, sents_lab

tokens_all, tags_all = read_conll(DATA_PATH)

def predict_labels_for_pretokenized(tokens):
    # tokens -> ids（保持与训练一致，不再重新分词）
    ids = tokenizer.convert_tokens_to_ids(tokens)
    # 未知子词用 unk
    ids = [i if i is not None else tokenizer.unk_token_id for i in ids]
    # 加 special tokens
    CLS_ID = tokenizer.convert_tokens_to_ids(tokenizer.cls_token)  # <s>
    SEP_ID = tokenizer.convert_tokens_to_ids(tokenizer.sep_token)  # </s>
    input_ids = [CLS_ID] + ids + [SEP_ID]
    attn = [1] * len(input_ids)

    enc = {
        "input_ids": torch.tensor([input_ids]),
        "attention_mask": torch.tensor([attn]),
    }
    with torch.no_grad():
        logits = model(**enc).logits[0]  # (seq_len, num_labels)
    pred_ids = logits.argmax(-1).tolist()

    # 去掉 special tokens 的标签
    pred_ids = pred_ids[1:-1]
    assert len(pred_ids) == len(tokens), (len(pred_ids), len(tokens))
    return [id2label[i] for i in pred_ids]

# 随机抽 10 条看看
random.seed(42)
idxs = random.sample(range(len(tokens_all)), 10)
for i in idxs:
    toks = tokens_all[i]
    gold = tags_all[i]
    pred = predict_labels_for_pretokenized(toks)

    print("="*40)
    print("原始 Tokens:", " ".join(toks))
    print("真实 Labels:", " ".join(gold))
    print("预测 Labels:", " ".join(pred))
    # 一致性断言
    if len(pred) != len(toks):
        print("[WARN] 长度不一致！", len(pred), len(toks))


原始 Tokens: ▁— M öt et ▁börjar ▁klockan ▁10.00 ▁imorgon .
真实 Labels: O O O O O O O O O
预测 Labels: O O O O O I-TIME I-TIME O O
原始 Tokens: ▁ , b IT TE ▁DEN KEN ▁s IE ▁D ARAN , ▁DIE ▁d OKU MENTE ▁ BIS ▁m ON TAG ▁ ZU ▁SE NDEN .
真实 Labels: O O O O O O O O O O O O O O O O O O O O O O O O O O
预测 Labels: O O I-TIME I-TIME I-TIME I-TIME I-TIME O O O O O O O O O O O I-TIME I-TIME I-TIME O O O O O
原始 Tokens: ▁/ 请 前往 1 栋 登记 , 并 同时 关注 百 联 东 郊 购物 中心 进展 。
真实 Labels: O O O O O O O O O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O O
预测 Labels: O O O O O O O O O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O O
原始 Tokens: ▁- В чер а ▁я ▁был ▁в ▁Ме тро ▁« О кт яб рь ская » ▁( ради альная ), ▁и ▁мне ▁понравилось . ▁Надо ▁еще ▁с ходить ▁в ▁З да ние ▁10 ▁и ▁сравни ть . ▁Кстати , ▁как ▁тебе ▁Ми чу рин ский ▁проспект ▁и ▁ул . ▁Жу кова , ▁д . ▁5 , ▁ оф . ▁301 , ▁Волгоград , ▁Волгоград ская ▁область , ▁400 001 ?
真实 Labels: O O O O O O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-

##### 检查check point里的模型（pytorch）

In [ ]:
import os, torch, numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification

# 模型目录：可以是总目录（包含多个 checkpoint-* 子目录），也可以直接是某个 checkpoint-xxx
MODEL_DIR = "/content/drive/MyDrive/NER/CheckPoints/bestCheck"  # 实际目录

LABELS = ["O", "B-ADDRESS", "I-ADDRESS", "B-TIME", "I-TIME"]
#LABELS = ["O", "B-ADDRESS", "I-ADDRESS"]
id2label = {i:l for i,l in enumerate(LABELS)}

def latest_checkpoint(base_dir: str) -> str:
    if os.path.basename(base_dir).startswith("checkpoint-") and os.path.isdir(base_dir):
        return base_dir
    cks = [d for d in os.listdir(base_dir) if d.startswith("checkpoint-") and os.path.isdir(os.path.join(base_dir, d))]
    if not cks:
        return base_dir
    cks.sort(key=lambda x: int(x.split("-")[-1]))
    return os.path.join(base_dir, cks[-1])

CKPT = latest_checkpoint(MODEL_DIR)
print("Load from:", CKPT)

tokenizer = AutoTokenizer.from_pretrained(CKPT, use_fast=True)
model = AutoModelForTokenClassification.from_pretrained(CKPT)
model.eval()
if torch.cuda.is_available():
    model.cuda()

def predict_entities(text: str, max_len: int = 256):
    enc = tokenizer(text, return_offsets_mapping=True, truncation=True,
                    max_length=max_len, return_tensors="pt")
    offsets = enc.pop("offset_mapping")
    if torch.cuda.is_available():
        enc = {k: v.cuda() for k, v in enc.items()}

    with torch.no_grad():
        logits = model(**enc).logits[0]  # (seq_len, num_labels)

    pred_ids = logits.argmax(-1).detach().cpu().numpy().tolist()
    input_ids = enc["input_ids"][0].detach().cpu().tolist()
    offsets = offsets[0].detach().cpu().tolist()
    toks = tokenizer.convert_ids_to_tokens(input_ids)

    # 过滤 special tokens & 0 长度偏移
    keep = []
    for tok, (st, ed) in zip(toks, offsets):
        if tok in (tokenizer.cls_token, tokenizer.sep_token, "<s>", "</s>", tokenizer.pad_token):
            keep.append(False)
        elif (st == 0 and ed == 0):
            keep.append(False)
        else:
            keep.append(True)

    toks = [t for t,k in zip(toks, keep) if k]
    offs = [tuple(map(int, o)) for o,k in zip(offsets, keep) if k]
    labs = [id2label[i] for i,k in zip(pred_ids, keep) if k]

    # 合并 B-/I- 为实体片段（按字符偏移还原原文片段）
    spans = []
    cur_type, cur_s, cur_e = None, None, None
    for lab, (st, ed) in zip(labs, offs):
        if lab.startswith("B-"):
            if cur_type:
                spans.append((cur_type, cur_s, cur_e))
            cur_type, cur_s, cur_e = lab[2:], st, ed
        elif lab.startswith("I-") and cur_type == lab[2:]:
            cur_e = ed
        else:
            if cur_type:
                spans.append((cur_type, cur_s, cur_e))
            cur_type, cur_s, cur_e = None, None, None
    if cur_type:
        spans.append((cur_type, cur_s, cur_e))

    entities = [(typ, text[s:e]) for typ, s, e in spans]
    return toks, labs, entities

# 样例
samples = [
    "TCL手机25.8.12震撼发售",
    "25.8.12",
    "甲辰年七月初十",
    "下周五下午2点 客户拜访/地点：北京分公司",
    "明天早上来6楼会议室开会",
    "我站在南京市长江大桥眺望",
    "我想去东洞镇小亮村",
    "我打算農曆十月初五中午十二點零五分到西藏自治區山南市浪卡子縣羊卓雍錯參加讀書會,聽說活動很有趣。",
    "请在2026年4月8日早上9:30到恩施土家族苗族自治州集合。",
    "Meet me at 221B Baker Street, London at 7 pm tomorrow.",
    "6月5日（月曜日）に新潟県新潟市中央区万代3丁目7-8で集合しましょう。",
    "Perşembe 22 Ağustos 2024 saat 10:15'te İstiklal Cd. No:56, Beyoğlu, İstanbul'da buluşalım。",
    "今晚8点我们去尝尝新安街道的那家麦当劳怎么样，我没什么别的想吃的啦，还是说你打算在公司食堂吃？那样的话我就回西乡吃晚餐了。",
    "只有你好好听我们的话，后天晚上我们才有机会去西部欢乐园玩，否则一直到国庆节结束，你都得待在兰州西站的兰州中心2楼里的西西弗书店里乖乖看书。好好想想吧，明天晚上6点前我要你给出答复。"
]

for i, s in enumerate(samples, 1):
    toks, labs, ents = predict_entities(s)
    print("="*68)
    print(f"[{i}] Text:", s)
    print("TOK:", " ".join(toks))
    print("LBL:", " ".join(labs))
    print("Spans:", ents)


Load from: /content/drive/MyDrive/NER/CheckPoints/bestCheck
[1] Text: TCL手机25.8.12震撼发售
TOK: ▁T CL 手机 25 .8. 12 震撼 发 售
LBL: B-ADDRESS I-ADDRESS O B-TIME I-TIME I-TIME O O O
Spans: [('ADDRESS', 'TCL'), ('TIME', '25.8.12')]
[2] Text: 25.8.12
TOK: ▁25 .8. 12
LBL: B-TIME I-TIME I-TIME
Spans: [('TIME', '25.8.12')]
[3] Text: 甲辰年七月初十
TOK: ▁ 甲 辰 年 七 月初 十
LBL: B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME
Spans: [('TIME', '甲辰年七月初十')]
[4] Text: 下周五下午2点 客户拜访/地点：北京分公司
TOK: ▁ 下 周五 下午 2 点 ▁ 客户 拜 访 / 地点 : 北京 分公司
LBL: B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME O O O O O O O B-ADDRESS I-ADDRESS
Spans: [('TIME', '下周五下午2点'), ('ADDRESS', '北京分公司')]
[5] Text: 明天早上来6楼会议室开会
TOK: ▁ 明天 早上 来 6 楼 会议 室 开 会
LBL: B-TIME I-TIME I-TIME O O O O O O O
Spans: [('TIME', '明天早上')]
[6] Text: 我站在南京市长江大桥眺望
TOK: ▁我 站在 南京 市 长江 大 桥 眺 望
LBL: O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O O
Spans: [('ADDRESS', '南京市长江大桥')]
[7] Text: 我想去东洞镇小亮村
TOK: ▁我 想去 东 洞 镇 小 亮 村
LBL: O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS

##### **在json格式的datasets测试集下进行测试**

*   展示部分：前 SHOW_N 条样本输出预测实体对照 Gold 标签
*   词级指标：集合交集计算 precision/recall/F1
*   句级指标：如果 Gold 这个类型的 token 全部出现在预测 token 集合中，则该句该类实体句级准确率计 1，否则计 0
每个文件单独统计两个类别（TIME/ADDRESS）的指标

In [ ]:
import os, json, torch
import regex as re
from transformers import AutoTokenizer, AutoModelForTokenClassification
import plotly.graph_objects as go

# ========= 参数 =========
MODEL_DIR = "/content/ner_xlmr_out/checkpoint-159"
LABELS = ["O", "B-ADDRESS", "I-ADDRESS", "B-TIME", "I-TIME"]
id2label = {i: l for i, l in enumerate(LABELS)}
DATASET_DIR = "/content/drive/MyDrive/NER/datasets"
SHOW_N = 3  # 每个文件展示多少示例
STOPWORDS_DIR = "/content/drive/MyDrive/NER/stopwords"  # 停用词目录（新增）

SPACE_LANGS = [

]
NO_SPACE_LANGS = ["简体中文", "繁体中文", "日语", "英语", "法语", "德语", "意大利语", "西班牙语", "葡萄牙语",
    "荷兰语", "瑞典语", "丹麦语", "俄语", "土耳其语", "韩语"]

LANGUAGES = [
    "简体中文", "英语", "丹麦语", "俄语", "土耳其语", "德语",
    "意大利语", "日语", "法语", "瑞典语", "荷兰语", "葡萄牙语",
    "西班牙语", "韩语", "繁体中文"
]

# ========= 加载停用词 =========
def load_stopwords(stop_dir, languages):
    sw_dict = {}
    for lang in languages:
        path = os.path.join(stop_dir, f"{lang}.txt")
        sw_set = set()
        if os.path.isfile(path):
            with open(path, "r", encoding="utf-8") as f:
                for line in f:
                    w = line.strip()
                    if w:
                        sw_set.add(w.lower())
        sw_dict[lang] = sw_set
    return sw_dict

STOPWORDS = load_stopwords(STOPWORDS_DIR, LANGUAGES)

def process_entity(entity, sw_set, is_addr):
    """
    按规则过滤单个实体：
    1. 停用词在首尾 => 删除，保持实体个数不变
    2. 停用词在中间：地址 => 拆分成两个实体；时间 => 删除停用词直接合并
    """
    ent = entity.strip()
    ent_low = ent.lower()
    for sw in sw_set:
        if sw in ent_low:
            # 首部
            if ent_low.startswith(sw):
                cleaned = ent[len(sw):].strip()
                return [cleaned] if cleaned else []
            # 末尾
            elif ent_low.endswith(sw):
                cleaned = ent[:-len(sw)].strip()
                return [cleaned] if cleaned else []
            else:
                # 中间部分
                if is_addr:
                    parts = ent.split(sw)
                    parts = [p.strip() for p in parts if p.strip()]
                    return parts
                else:
                    cleaned = ent.replace(sw, "").strip()
                    return [cleaned] if cleaned else []
    return [ent]

def filter_entities_with_split(entities, lang, is_addr):
    sw_set = STOPWORDS.get(lang, set())
    result = []
    for ent in entities:
        result.extend(process_entity(ent, sw_set, is_addr))
    return result

# ========= 加载最新checkpoint =========
def latest_checkpoint(base_dir: str) -> str:
    if os.path.basename(base_dir).startswith("checkpoint-") and os.path.isdir(base_dir):
        return base_dir
    cks = [d for d in os.listdir(base_dir) if d.startswith("checkpoint-") and os.path.isdir(os.path.join(base_dir, d))]
    if not cks:
        return base_dir
    cks.sort(key=lambda x: int(x.split("-")[-1]))
    return os.path.join(base_dir, cks[-1])

CKPT = latest_checkpoint(MODEL_DIR)
print("Load from:", CKPT)

tokenizer = AutoTokenizer.from_pretrained(CKPT, use_fast=True)
model = AutoModelForTokenClassification.from_pretrained(CKPT)
model.eval()
if torch.cuda.is_available():
    model.cuda()

# ========= 工具函数 =========
def is_symbol(tok: str) -> bool:
    stripped = tok.replace('▁', '')
    return bool(re.fullmatch(r'[\p{P}\p{S}]+', stripped))

def merge_tokens_by_underscore_infer(tokens, labels, offsets):
    merged_tokens, merged_labels, merged_offsets = [], [], []
    cur_tok, cur_lab, cur_start, cur_end = "", None, None, None
    for tok, lab, (st, ed) in zip(tokens, labels, offsets):
        if tok is None:
            tok = ""
        elif not isinstance(tok, str):
            tok = str(tok)
        if tok.startswith("▁") or is_symbol(tok):
            if cur_tok:
                merged_tokens.append(cur_tok)
                merged_labels.append(cur_lab)
                merged_offsets.append((cur_start, cur_end))
            cur_tok = tok
            cur_lab = lab
            cur_start = st
            cur_end = ed
        else:
            cur_tok += tok
            cur_end = ed
    if cur_tok:
        merged_tokens.append(cur_tok)
        merged_labels.append(cur_lab)
        merged_offsets.append((cur_start, cur_end))
    return merged_tokens, merged_labels, merged_offsets

def predict_entities(text: str, lang: str, max_len: int = 256):
    enc = tokenizer(text, return_offsets_mapping=True, truncation=True,
                    max_length=max_len, return_tensors="pt")
    offsets = enc.pop("offset_mapping")
    if torch.cuda.is_available():
        enc = {k: v.cuda() for k, v in enc.items()}
    with torch.no_grad():
        logits = model(**enc).logits[0]
    pred_ids = logits.argmax(-1).detach().cpu().numpy().tolist()
    input_ids = enc["input_ids"][0].detach().cpu().tolist()
    offsets = offsets[0].detach().cpu().tolist()
    toks = tokenizer.convert_ids_to_tokens(input_ids)
    keep = []
    for tok, (st, ed) in zip(toks, offsets):
        if tok in (tokenizer.cls_token, tokenizer.sep_token, "<s>", "</s>", tokenizer.pad_token):
            keep.append(False)
        elif (st == 0 and ed == 0):
            keep.append(False)
        else:
            keep.append(True)
    toks = [t for t, k in zip(toks, keep) if k]
    offs = [tuple(map(int, o)) for o, k in zip(offsets, keep) if k]
    labs = [id2label[i] for i, k in zip(pred_ids, keep) if k]
    if lang in SPACE_LANGS:
        toks, labs, offs = merge_tokens_by_underscore_infer(toks, labs, offs)
    spans = []
    cur_type, cur_s, cur_e = None, None, None
    for lab, (st, ed) in zip(labs, offs):
        if lab.startswith("B-"):
            if cur_type:
                spans.append((cur_type, cur_s, cur_e))
            cur_type, cur_s, cur_e = lab[2:], st, ed
        elif lab.startswith("I-") and cur_type == lab[2:]:
            cur_e = ed
        else:
            if cur_type:
                spans.append((cur_type, cur_s, cur_e))
            cur_type, cur_s, cur_e = None, None, None
    if cur_type:
        spans.append((cur_type, cur_s, cur_e))
    entities = [(typ, text[s:e]) for typ, s, e in spans]
    return toks, labs, entities

def token_level_metrics(pred_tokens, gold_tokens):
    pred_set, gold_set = set(pred_tokens), set(gold_tokens)
    if not gold_set:
        return None
    tp = len(pred_set & gold_set)
    precision = tp / len(pred_set) if pred_set else 0.0
    recall = tp / len(gold_set) if gold_set else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision+recall) > 0 else 0.0
    return precision, recall, f1

def char_overlap_rate(pred_strs, gold_strs):
    if not gold_strs:
        return 1.0 if not pred_strs else 0.0
    pred_text = "".join(pred_strs)
    gold_text = "".join(gold_strs)
    common = sum(min(pred_text.count(c), gold_text.count(c)) for c in set(gold_text))
    return common / len(gold_text) if gold_text else 0.0
# 在循环开始前加：
time_F1_list = []
addr_F1_list = []
# ========= 主测试流程 =========
for fname in os.listdir(DATASET_DIR):
    if not fname.endswith(".json"):
        continue
    lang = fname.replace(".json", "")
    path = os.path.join(DATASET_DIR, fname)
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"\n==== Testing on file: {fname} ====")
    metrics_time_tok, metrics_addr_tok = [], []
    overlap_time_list, overlap_addr_list = [], []
    for idx, sample in enumerate(data):
        sid = sample.get("id")
        sent = sample.get("sentence")
        gold_time = sample.get("time_tokens", [])
        gold_addr = sample.get("location_tokens", [])
        toks, labs, ents = predict_entities(sent, lang)

        # Gold tokens
        gold_time_toks, gold_addr_toks = [], []
        for gt in gold_time:
            g_tokens = tokenizer.tokenize(gt)
            g_labels = ["B-TIME"] + ["I-TIME"] * (len(g_tokens)-1)
            g_offs = [(0,0)] * len(g_tokens)
            if lang in SPACE_LANGS:
                g_tokens, g_labels, g_offs = merge_tokens_by_underscore_infer(g_tokens, g_labels, g_offs)
            gold_time_toks.extend(g_tokens)
        for ga in gold_addr:
            g_tokens = tokenizer.tokenize(ga)
            g_labels = ["B-ADDRESS"] + ["I-ADDRESS"] * (len(g_tokens)-1)
            g_offs = [(0,0)] * len(g_tokens)
            if lang in SPACE_LANGS:
                g_tokens, g_labels, g_offs = merge_tokens_by_underscore_infer(g_tokens, g_labels, g_offs)
            gold_addr_toks.extend(g_tokens)

        pred_time_toks = [t for t, l in zip(toks, labs) if l.endswith("TIME")]
        pred_addr_toks = [t for t, l in zip(toks, labs) if l.endswith("ADDRESS")]

        m_time = token_level_metrics(pred_time_toks, gold_time_toks)
        m_addr = token_level_metrics(pred_addr_toks, gold_addr_toks)
        if m_time: metrics_time_tok.append(m_time)
        if m_addr: metrics_addr_tok.append(m_addr)

        # 原字符串列表
        pred_time_strs = [e[1] for e in ents if e[0] == "TIME"]
        pred_addr_strs = [e[1] for e in ents if e[0] == "ADDRESS"]

        # ==== 停用词后处理 ====
        pred_time_strs = filter_entities_with_split(pred_time_strs, lang, is_addr=False)
        pred_addr_strs = filter_entities_with_split(pred_addr_strs, lang, is_addr=True)

        ov_time = char_overlap_rate(pred_time_strs, gold_time)
        ov_addr = char_overlap_rate(pred_addr_strs, gold_addr)
        if ov_time is not None: overlap_time_list.append(ov_time)
        if ov_addr is not None: overlap_addr_list.append(ov_addr)

        if idx < SHOW_N:
            print("-"*68)
            print(f"[{sid}] Text:", sent)
            print("TOK:", " ".join(toks))
            print("LBL:", " ".join(labs))
            print(f"PRED TIME: {pred_time_strs}")
            print(f"GOLD TIME: {gold_time}")
            print(f"PRED ADDR: {pred_addr_strs}")
            print(f"GOLD ADDR: {gold_addr}")
            print(f"字符重合率: TIME={ov_time:.3f}, ADDR={ov_addr:.3f}")


    def avg_metrics(m_list):
        if not m_list: return (0,0,0)
        p = sum(x[0] for x in m_list) / len(m_list)
        r = sum(x[1] for x in m_list) / len(m_list)
        f1 = sum(x[2] for x in m_list) / len(m_list)
        return (p,r,f1)

    avg_time = avg_metrics(metrics_time_tok)
    avg_addr = avg_metrics(metrics_addr_tok)
    avg_overlap_time = sum(overlap_time_list) / len(overlap_time_list) if overlap_time_list else 0
    avg_overlap_addr = sum(overlap_addr_list) / len(overlap_addr_list) if overlap_addr_list else 0

    print(f"[Metrics TIME] token-level: P={avg_time[0]:.3f}, R={avg_time[1]:.3f}, F1={avg_time[2]:.3f}")
    print(f"[Metrics ADDR] token-level: P={avg_addr[0]:.3f}, R={avg_addr[1]:.3f}, F1={avg_addr[2]:.3f}")
    print(f"[Overlap] 平均字符重合率: TIME={avg_overlap_time:.3f}, ADDR={avg_overlap_addr:.3f}")
    #添加记录
    time_F1_list.append(avg_time[2])  # F1值
    addr_F1_list.append(avg_addr[2])
# ====================== 循环结束后绘图 ======================
fig = go.Figure()

fig.add_trace(go.Bar(
    x=LANGUAGES[:len(time_F1_list)],  # 只取测试到的语言
    y=time_F1_list,
    name='TIME F1',
    marker_color='#1f77b4',
    text=[f"{v:.3f}" for v in time_F1_list],
    textposition='outside'
))

fig.add_trace(go.Bar(
    x=LANGUAGES[:len(addr_F1_list)],
    y=addr_F1_list,
    name='ADDR F1',
    marker_color='#ff7f0e',
    text=[f"{v:.3f}" for v in addr_F1_list],
    textposition='outside'
))

fig.update_layout(
    barmode='group',
    bargap=0.25,
    plot_bgcolor='rgba(250,250,250,1)',
    paper_bgcolor='white',
    xaxis=dict(title='语言', tickangle=-30),
    yaxis=dict(title='F1 Score', range=[0,1], dtick=0.1),
    legend=dict(title=dict(text='指标类型'), orientation='h',
                yanchor='bottom', y=1.05,
                xanchor='right', x=1),
    font=dict(family='Microsoft YaHei, sans-serif', size=13)
)

fig.show()

Load from: /content/ner_xlmr_out/checkpoint-159

==== Testing on file: 简体中文.json ====
--------------------------------------------------------------------
[1] Text: 2024年6月15日早上8:10，在北京天坛公园晨跑，空气里全是青草味。
TOK: ▁20 24 年 6 月 15 日 早上 8 :10 , 在北京 天 坛 公园 晨 跑 , 空气 里 全 是 青 草 味 。
LBL: B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME O O I-ADDRESS I-ADDRESS I-ADDRESS O O O O O O O O O O O
PRED TIME: ['2024年6月15日早上8:10']
GOLD TIME: ['2024年6月15日早上8:10']
PRED ADDR: []
GOLD ADDR: ['北京天坛公园']
字符重合率: TIME=1.000, ADDR=0.000
--------------------------------------------------------------------
[2] Text: 重庆解放碑的洪鼎火锅尝了一次就上瘾，麻辣香气绕嘴不散，就是等位太久了。
TOK: ▁ 重庆 解放 碑 的 洪 鼎 火 锅 尝 了 一次 就 上 瘾 , 麻 辣 香 气 绕 嘴 不 散 , 就是 等 位 太 久 了 。
LBL: B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O O O O O O O O O O O O O O O O O O O O O O O
PRED TIME: []
GOLD TIME: []
PRED ADDR: ['重庆解放碑', '洪鼎火锅']
GOLD ADDR: ['重庆解放碑洪鼎火锅']
字符重合率: TIME=1.000, ADDR=1.000
------------------------------------

KeyboardInterrupt: 

### **模型量化与转tflite格式**

##### **转tflite同时进行动态int8量化**

In [ ]:
import tensorflow as tf

# pytorch模型文件路径（训练时保存的）
saved_model_dir = "/content/export_xlmr_tf/saved_model"  # @param {"type":"string","placeholder":"请输入模型路径"}
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
tflite_model_name = 'model_dy_quant.tflite'# @param {"type":"string","placeholder":"请输入tflite模型名称"}
# Save the model.
# tflite模型保存到左侧文件栏中，需要及时下载
with open(tflite_model_name, 'wb') as f:
  f.write(tflite_quant_model)

##### **直接转tflite**
正式流程中不需要使用，但是如果转tflite同时量化的结果不理想，可以运行改代码对比量化和未量化的模型来排查问题出在转tflite还是量化

In [ ]:
import tensorflow as tf

saved_model_dir = r"/content/export_xlmr_tf/saved_model"
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

### 系统优化模型代码：如何得到最佳模型？

#### 在确定了每种语言五百条数据时的模型性能已经普遍达标的情况下：
产生了一个重要的问题——
怎么样才能从海量模板和词汇中挑选出那些真正有价值的生成数据？

在一次同样参数数据设置下的不同模型测量性能出现较大区分后，发现了关键问题：
可以通过反复训练小模型并测量不同语言上的性能差异来得到所谓的最优训练集：
当出现某小语种上的性能显著高于之前版本模型时，将该语种的训练集保存到本地并作为
current_best训练集
这样从10000条模板和20000条实体间的组合中就能筛选出最优的训练集。
不断迭代整个过程，训练就会彻底成功。
将这整个流程组合在一起，就成了迭代得到当前最好模型的“系统优化代码”。

这其实就是在建立一个强化学习系统的雏形：有了迭代目标（最优训练集），有评价指标（F1），有模型更新的简单策略（mean值差异>0.01），有充分随机的向量探索空间（训练模板+实体池=训练集）


*   策略更新1：软更新
*   是在上一轮没有任何语言发生替换的情况下，为了防止模型停滞，针对已有的训练集做有约束范围的探索性小幅调整
*   保留原训练集的 70%，引入新合成的 30% 样本。
*   或者根据样本难度分数（loss高的样本）来替换掉最容易的样本。
*   这种“软更新”在RL中相当于**经验回放(Experience Replay)**中加权采样——既保留有价值的历史经验，也引入新探索。


In [ ]:
# ===== 安装依赖 =====
!pip install -q evaluate seqeval plotly

# ===== Step0: 全局设置 =====
from google.colab import drive
import os, json
import shutil
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
drive.mount('/content/drive')
from transformers import AutoTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import random
TOKENIZER_NAME = "xlm-roberta-base"

# 全局复用 tokenizer（初始为预训练模型版本）
GLOBAL_TOKENIZER = AutoTokenizer.from_pretrained(TOKENIZER_NAME, use_fast=True)
# 全局路径
TRAIN_DATA_DIR = "/content/drive/MyDrive/NER/train_datasets_AC"
BEST_SAVE_DIR = "/content/drive/MyDrive/NER/current_best"
BEST_RECORD_FILE = os.path.join(BEST_SAVE_DIR, "best_scores.json")
MODEL_OUTPUT_DIR = "/content/ner_xlmr_out"
TEST_DATA_ROOT = "/content/drive/MyDrive/NER/datasets"  # JSON测试集目录
STOPWORDS_DIR = "/content/drive/MyDrive/NER/stopwords"

# 全局参数
ITERATIONS = 3 #@param
IMPROVE_THRESHOLD = 0.0001 # @param
keep_ratios=0.95 # @param 保留比例
last_replace_happened = False  # 初始化为 False,直接开始寻找更优训练集
ENABLE_SOFT_UPDATE = True # 软更新可拔插参数 True 时启用软更新限制探索范围
# 全局标签定义
LABELS = ["O", "B-ADDRESS", "I-ADDRESS", "B-TIME", "I-TIME"]

# 语言列表
LANGS = ["简体中文", "繁体中文", "英语", "丹麦语", "俄语", "土耳其语",
         "德语", "意大利语", "日语", "法语", "瑞典语",
         "荷兰语", "葡萄牙语", "西班牙语", "韩语"]

os.makedirs(TRAIN_DATA_DIR, exist_ok=True)
os.makedirs(BEST_SAVE_DIR, exist_ok=True)
N_PER_LANG=500#@param

########################################
# === 工具函数：特征提取 ===
########################################

def compute_sample_embedding(model, tokenizer, text):
    """生成样本的 CLS 向量嵌入"""
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embed = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # [CLS] 向量
    return cls_embed[0]  # shape: (hidden_dim,)


def compute_confidence_map(model, tokenizer, dataset):
    """计算每个样本的平均预测置信度映射: sample_id -> confidence"""
    conf_map = {}
    for sample in dataset:
        inputs = tokenizer(sample['text'], return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits  # (1, seq_len, num_labels)
        probs = torch.softmax(logits, dim=-1).cpu().numpy()  # (1, L, C)
        max_probs = probs.max(axis=-1).mean()  # 平均最大概率
        conf_map[sample['id']] = float(max_probs)
    return conf_map


def compute_template_frequency(dataset):
    """统计模板使用频率"""
    template_ids = [s['template_id'] for s in dataset]
    freq = Counter(template_ids)
    return freq


def compute_validation_embeddings(model, tokenizer, val_dataset):
    """计算验证集所有样本的CLS嵌入"""
    all_embeds = []
    for sample in val_dataset:
        emb = compute_sample_embedding(model, tokenizer, sample['text'])
        all_embeds.append(emb)
    return np.array(all_embeds)  # shape: (N_val, hidden_dim)

def latest_checkpoint(base_dir: str) -> str:
    if os.path.basename(base_dir).startswith("checkpoint-") and os.path.isdir(base_dir):
        return base_dir
    cks = [d for d in os.listdir(base_dir) if d.startswith("checkpoint-") and os.path.isdir(os.path.join(base_dir, d))]
    if not cks:
        return base_dir
    cks.sort(key=lambda x: int(x.split("-")[-1]))
    return os.path.join(base_dir, cks[-1])
########################################
# === 多维评分软更新 ===
########################################

def compute_confidence_change(sample, prev_conf_map, curr_conf_map):
    prev_conf = prev_conf_map.get(sample['id'], 0.0)
    curr_conf = curr_conf_map.get(sample['id'], 0.0)
    return abs(curr_conf - prev_conf)


def compute_validation_similarity(sample_embedding, val_embeddings):
    sims = cosine_similarity(sample_embedding.reshape(1, -1), val_embeddings)
    return sims.mean()


def soft_update_with_multiscore(old_dataset, new_dataset, keep_ratio,
                                 prev_conf_map, curr_conf_map,
                                 template_frequency, val_embeddings,
                                 alpha, beta, gamma, delta):

    # loss 中位数用作难度计算基准
    losses = np.array([s['loss'] for s in old_dataset])
    median_loss = np.median(losses)

    scored_samples = []
    for sample in old_dataset:
        # ① 置信度变化分
        conf_score = compute_confidence_change(sample, prev_conf_map, curr_conf_map)

        # ② 验证集相似度分
        val_score = compute_validation_similarity(sample['embedding'], val_embeddings)

        # ③ 模板多样性分
        template_id = sample['template_id']
        template_score = 1.0 / (template_frequency.get(template_id, 0) + 1)

        # ④ 难度分
        loss = sample['loss']
        diff_score = 1.0 - abs(loss - median_loss) / median_loss

        # 综合评分
        total_score = (alpha * conf_score +
                       beta * val_score +
                       gamma * template_score +
                       delta * diff_score)
        scored_samples.append((total_score, sample))

    # 按总分排序
    scored_samples.sort(key=lambda x: x[0], reverse=True)
    keep_count = int(len(scored_samples) * keep_ratio)

    kept_samples = [s[1] for s in scored_samples[:keep_count]]
    updated_dataset = kept_samples + new_dataset

    print(f"[Soft Update] Kept {keep_count} high-score samples, added {len(new_dataset)} new samples")
    return updated_dataset
# ===== Step1: 优先用最佳集，否则合成数据 =====
def generate_training_data_for_lang(lang, count):
    """
    生成指定语言的训练样本，返回 tokens_list, labels_list
    不写文件 — 由调用方决定是否保存
    """
    import os, re, json, random, unicodedata
    from transformers import AutoTokenizer

    per_00 = 0.10
    per_11 = 0.30
    NEG_ADDR_PROB = 0.2
    NEG_TIME_PROB = 0.2
    COMBINE_TPL_ROOT = "/content/drive/MyDrive/NER/combine_templates"
    REAL_ADDR_ROOT = "/content/drive/MyDrive/NER/real_address"
    REAL_TIME_ROOT = "/content/drive/MyDrive/NER/real_time"
    NON_ADDR_ROOT  = "/content/drive/MyDrive/NER/non_address"
    NON_TIME_ROOT  = "/content/drive/MyDrive/NER/non_time"
    TOKENIZER_NAME = "xlm-roberta-base"

    tokenizer = GLOBAL_TOKENIZER

    def read_lines(path):
        if not os.path.exists(path):
            return []
        with open(path, "r", encoding="utf-8") as f:
            return [ln.strip() for ln in f if ln.strip()]

    def read_jsonl_templates(path):
        items = []
        if not os.path.exists(path):
            return items
        with open(path, "r", encoding="utf-8") as f:
            for ln in f:
                try:
                    obj = json.loads(ln.strip())
                    t = obj.get("template")
                    if isinstance(t, str) and t.strip():
                        items.append(" ".join(t.split()))
                except:
                    continue
        return items

    def load_templates_for_lang(lang):
        pos_11_tpls, pos_other_tpls, neg_sent_tpls = [], [], []
        filename = f"{lang}_combine_templates.jsonl"
        for name in os.listdir(COMBINE_TPL_ROOT):
            if not name.startswith("combine_templates_"):
                continue
            subdir = os.path.join(COMBINE_TPL_ROOT, name)
            fpath = os.path.join(subdir, filename)
            items = read_jsonl_templates(fpath)
            if not items:
                continue
            slot = name[len("combine_templates_"):]
            if slot.isdigit():
                if int(slot) == 0:
                    neg_sent_tpls.extend(items)
                elif int(slot) == 11:
                    pos_11_tpls.extend(items)
                else:
                    pos_other_tpls.extend(items)
        return {"pos_11": pos_11_tpls, "pos_other": pos_other_tpls, "neg_sent": neg_sent_tpls}

    def apply_noise(text, spans):
        def toggle_case(s):
            return "".join(c.upper() if "a"<=c<="z" else c.lower() if "A"<=c<="Z" else c for c in s)
        def to_fullwidth(c):
            return chr(ord(c) + 0xFEE0) if 33 <= ord(c) <= 126 else "\u3000" if c==" " else c
        def to_halfwidth(c):
            return " " if c=="\u3000" else chr(ord(c) - 0xFEE0) if 65281 <= ord(c) <= 65374 else c
        def fullwidth_halfwidth_flip(s):
            out = []
            for ch in s:
                if unicodedata.east_asian_width(ch) in ("F","W") and random.random()<0.5:
                    out.append(to_halfwidth(ch))
                elif random.random() < 0.2:
                    out.append(to_fullwidth(ch))
                else:
                    out.append(ch)
            return "".join(out)
        def replace_separators(s):
            trans = {"-": random.choice(["—","-","–"]), "/": random.choice(["/","／","∕"]),
                     ".": random.choice([".","·","．"]), ":": random.choice([":","："])}
            return "".join(trans.get(ch, ch) for ch in s)

        if random.random() < 0.15:
            text = toggle_case(text)
        if random.random() < 0.10:
            text = fullwidth_halfwidth_flip(text)
        if random.random() < 0.10:
            text = replace_separators(text)
        return text, spans

    def fill_and_align_to_bio(tpl, lang, addrs_map, times_map, non_addr_map, non_time_map):
        parts, entities = [], []
        last_pos = 0
        for m in re.finditer(r'\{address\}|\{time\}', tpl):
            parts.append(tpl[last_pos:m.start()])
            if m.group() == "{address}":
                if random.random() < NEG_ADDR_PROB:
                    rep = random.choice(non_addr_map.get(lang, [""]))
                else:
                    rep = random.choice(addrs_map.get(lang, ["Main Street"]))
                    start_char = sum(len(p) for p in parts)
                    entities.append((start_char, start_char+len(rep), "ADDRESS"))
                parts.append(rep)
            elif m.group() == "{time}":
                if random.random() < NEG_TIME_PROB:
                    rep = random.choice(non_time_map.get(lang, [""]))
                else:
                    rep = random.choice(times_map.get(lang, ["2024-07-01"]))
                    start_char = sum(len(p) for p in parts)
                    entities.append((start_char, start_char+len(rep), "TIME"))
                parts.append(rep)
            last_pos = m.end()
        parts.append(tpl[last_pos:])
        text = "".join(parts)
        text, new_spans = apply_noise(text, [(s,e) for s,e,_ in entities])
        for i, (_, _, typ) in enumerate(entities):
            entities[i] = (new_spans[i][0], new_spans[i][1], typ)

        enc = tokenizer(text, add_special_tokens=False, return_offsets_mapping=True)
        tokens = tokenizer.convert_ids_to_tokens(enc["input_ids"])
        offsets = enc["offset_mapping"]
        labels = ["O"] * len(tokens)
        for start_char, end_char, ent_type in entities:
            first_flag = True
            for i, (tok_s, tok_e) in enumerate(offsets):
                if tok_e <= start_char or tok_s >= end_char:
                    continue
                labels[i] = f"B-{ent_type}" if first_flag else f"I-{ent_type}"
                first_flag = False
        return tokens, labels

    # 加载资源
    addrs_map    = {lang: read_lines(os.path.join(REAL_ADDR_ROOT,f"{lang}.txt"))}
    times_map    = {lang: read_lines(os.path.join(REAL_TIME_ROOT,f"{lang}.txt"))}
    non_addr_map = {lang: read_lines(os.path.join(NON_ADDR_ROOT,f"{lang}.txt"))}
    non_time_map = {lang: read_lines(os.path.join(NON_TIME_ROOT,f"{lang}.txt"))}

    templates = load_templates_for_lang(lang)
    neg_target       = int(count * per_00)
    pos_target_total = count - neg_target
    pos_11_target    = int(pos_target_total * per_11)
    pos_other_target = pos_target_total - pos_11_target
    pos_11_tpls  = templates.get("pos_11", ["{address} {time}"])
    pos_other_tpls = templates.get("pos_other", ["{address} {time}"])
    neg_tpls    = templates.get("neg_sent", [])

    samples_tokens, samples_labels = [], []
    for _ in range(pos_11_target):
        toks, lbls = fill_and_align_to_bio(random.choice(pos_11_tpls), lang, addrs_map, times_map, non_addr_map, non_time_map)
        samples_tokens.append(toks)
        samples_labels.append(lbls)
    for _ in range(pos_other_target):
        toks, lbls = fill_and_align_to_bio(random.choice(pos_other_tpls), lang, addrs_map, times_map, non_addr_map, non_time_map)
        samples_tokens.append(toks)
        samples_labels.append(lbls)
    for _ in range(neg_target):
        if neg_tpls:
            tpl = random.choice(neg_tpls)
            toks = tpl.split()
            lbls = ["O"]*len(toks)
            samples_tokens.append(toks)
            samples_labels.append(lbls)

    return samples_tokens, samples_labels
def soft_update_best_dataset(keep_ratio=0.6, w_midloss=0.7, w_conf=0.3):
    import torch
    import numpy as np
    from transformers import AutoTokenizer, AutoModelForTokenClassification

    print(f"[软更新] 保留loss接近中值的样本，比例={keep_ratio}")

    def read_conll(path):
        toks_all, labs_all = [], []
        toks, labs = [], []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                line=line.strip()
                if not line:
                    if toks:
                        toks_all.append(toks)
                        labs_all.append(labs)
                        toks,labs=[],[]
                    continue
                t,y = (line.split(" ",1)+["O"])[:2]
                toks.append(t); labs.append(y)
        if toks:
            toks_all.append(toks)
            labs_all.append(labs)
        return toks_all, labs_all

    def compute_loss_conf(model, tokenizer, tokens, labels, label2id):
        cls_id = tokenizer.cls_token_id
        sep_id = tokenizer.sep_token_id
        input_ids = [cls_id] + tokenizer.convert_tokens_to_ids(tokens) + [sep_id]
        labels_ids = [-100] + [label2id.get(l, 0) for l in labels] + [-100]
        attention_mask = [1]*len(input_ids)
        inputs = {
            "input_ids": torch.tensor([input_ids], dtype=torch.long),
            "attention_mask": torch.tensor([attention_mask], dtype=torch.long),
            "labels": torch.tensor([labels_ids], dtype=torch.long)
        }
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k,v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            loss = outputs.loss.item()
            probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
            max_probs = probs.max(axis=-1).mean()
        return loss, max_probs

    ckpt = latest_checkpoint(MODEL_OUTPUT_DIR)
    tokenizer = AutoTokenizer.from_pretrained(ckpt, use_fast=True)
    model = AutoModelForTokenClassification.from_pretrained(ckpt)
    if torch.cuda.is_available():
        model.cuda()
    model.eval()
    label2id = {l: i for i, l in enumerate(LABELS)}

    updated_langs = []

    for lang in LANGS:
        best_file = os.path.join(BEST_SAVE_DIR, f"{lang}.conll")
        if not os.path.isfile(best_file):
            continue

        all_tokens, all_labels = read_conll(best_file)
        orig_count = len(all_tokens)
        if orig_count == 0:
            continue

        losses, confs = [], []
        for tks, lbs in zip(all_tokens, all_labels):
            loss, conf = compute_loss_conf(model, tokenizer, tks, lbs, label2id)
            losses.append(loss)
            confs.append(conf)

        # 归一化
        loss_arr = np.array(losses)
        conf_arr = np.array(confs)
        loss_min, loss_max = loss_arr.min(), loss_arr.max()
        loss_norm = (loss_arr - loss_min) / (loss_max - loss_min + 1e-8)
        conf_norm = (conf_arr - conf_arr.min()) / (conf_arr.max() - conf_arr.min() + 1e-8)

        # 找中值
        median_loss_norm = np.median(loss_norm)

        # 分数：loss接近中值 + 置信度低
        scores = w_midloss * (1 - np.abs(loss_norm - median_loss_norm)) \
               + w_conf * (1 - conf_norm)

        sorted_idx = np.argsort(scores)[::-1]
        keep_count = int(len(sorted_idx) * keep_ratio)
        keep_idx = set(sorted_idx[:keep_count])

        kept_tokens = [all_tokens[i] for i in keep_idx]
        kept_labels = [all_labels[i] for i in keep_idx]

        new_count = orig_count - keep_count
        new_tokens, new_labels = generate_training_data_for_lang(lang, count=new_count)
        merged_tokens = kept_tokens + new_tokens
        merged_labels = kept_labels + new_labels

        train_file = os.path.join(TRAIN_DATA_DIR, f"train_{lang}.conll")
        with open(train_file, "w", encoding="utf-8") as f:
            for toks, labs in zip(merged_tokens, merged_labels):
                for t, l in zip(toks, labs):
                    f.write(f"{t} {l}\n")
                f.write("\n")

        updated_langs.append(lang)
        print(f"[软更新] {lang}: 保留{keep_count}条(中值loss优先), 新增{new_count}条 → {train_file}")

    print(f"[软更新] 完成，更新语言: {', '.join(updated_langs) if updated_langs else '无'}")
def prepare_training_data(iteration, last_replace_happened):
    """
    Step1: 数据准备逻辑（软更新可拔插 + 拔掉时用彻底随机生成）
    iteration: 当前迭代轮次 (从1开始)
    last_replace_happened: 上一轮是否有最佳集替换（性能提升）
    """
    os.makedirs(TRAIN_DATA_DIR, exist_ok=True)
    merge_path = os.path.join(TRAIN_DATA_DIR, "train_all.conll")

    # 检查最佳集文件是否完整存在
    best_files_exist = all(
        os.path.isfile(os.path.join(BEST_SAVE_DIR, f"{lang}.conll"))
        for lang in LANGS
    )

    # 情况 1：没有最佳集文件
    if not best_files_exist:
        print(f"[Step1] 第{iteration}轮：无最佳集文件 → 全量随机生成数据")
        generate_training_data()

    # 情况 2：第一轮 + 有最佳集文件
    elif iteration == 1:
        print(f"[Step1] 第{iteration}轮：初始化为最佳集（不软更新、不随机生成）")
        for lang in LANGS:
            src = os.path.join(BEST_SAVE_DIR, f"{lang}.conll")
            dst = os.path.join(TRAIN_DATA_DIR, f"train_{lang}.conll")
            shutil.copyfile(src, dst)

    # 情况 3：非第一轮 + 上一轮无替换
    elif not last_replace_happened:
        if ENABLE_SOFT_UPDATE:
            print(f"[Step1] 第{iteration}轮：上一轮无替换 → 软更新探索")
            soft_update_best_dataset(keep_ratio=keep_ratios,w_midloss=1.0, w_conf=0)
        else:
            print(f"[Step1] 第{iteration}轮：上一轮无替换 → 彻底随机生成数据")
            generate_training_data()

    # 情况 4：非第一轮 + 上一轮有替换
    else:
        if ENABLE_SOFT_UPDATE:
            print(f"[Step1] 第{iteration}轮：上一轮有提升 → 软更新探索（不覆盖）")
            soft_update_best_dataset(keep_ratio=keep_ratios,w_midloss=1.0, w_conf=0)
        else:
            print(f"[Step1] 第{iteration}轮：上一轮有提升 → 彻底随机生成数据")
            generate_training_data()

    # 合并所有训练数据到 train_all.conll
    with open(merge_path, "w", encoding="utf-8") as fout:
        for lang in LANGS:
            src = os.path.join(TRAIN_DATA_DIR, f"train_{lang}.conll")
            if os.path.isfile(src):
                with open(src, "r", encoding="utf-8") as fin:
                    fout.write(fin.read())
                    fout.write("\n")

    print(f"[Step1] 第{iteration}轮：train_all.conll 文件已生成，共包含 {len(LANGS)} 个语言数据")

def generate_training_data():
    """
    生成训练数据（多语言、多标签），输出到 TRAIN_DATA_DIR
    """
    import os, re, json, random, unicodedata
    from typing import List, Tuple, Dict
    from transformers import AutoTokenizer

    per_00 = 0.05#@param
    per_11 = 0.25#@param
    NEG_ADDR_PROB = 0.2
    NEG_TIME_PROB = 0.2
    PREVIEW_N = 3
    INSERT_CHARS = [",", ".", "·", "—", "-", "/", ":", "、", "・",
                    "?", "？", "！", "!", "，", "。", "；", ";", " "]
    COMBINE_TPL_ROOT = "/content/drive/MyDrive/NER/combine_templates"
    REAL_ADDR_ROOT = "/content/drive/MyDrive/NER/real_address"
    REAL_TIME_ROOT = "/content/drive/MyDrive/NER/real_time"
    NON_ADDR_ROOT  = "/content/drive/MyDrive/NER/non_address"
    NON_TIME_ROOT  = "/content/drive/MyDrive/NER/non_time"
    TOKENIZER_NAME = "xlm-roberta-base"

    PER_LANG_CONLL = lambda lang: os.path.join(TRAIN_DATA_DIR, f"train_{lang}.conll")
    MERGED_CONLL = os.path.join(TRAIN_DATA_DIR, "train_all.conll")

    tokenizer = GLOBAL_TOKENIZER

    def read_lines(path: str) -> List[str]:
        if not os.path.exists(path): return []
        with open(path, "r", encoding="utf-8") as f:
            return [ln.strip() for ln in f if ln.strip()]

    def read_jsonl_templates(path: str) -> list:
        items = []
        if not os.path.exists(path): return items
        with open(path, "r", encoding="utf-8") as f:
            for ln in f:
                try:
                    obj = json.loads(ln.strip())
                    t = obj.get("template")
                    if isinstance(t, str) and t.strip():
                        items.append(" ".join(t.split()))
                except: continue
        return items

    def load_templates_for_lang(lang: str) -> Dict[str, list]:
        pos_11_tpls, pos_other_tpls, neg_sent_tpls = [], [], []
        filename = f"{lang}_combine_templates.jsonl"
        for name in os.listdir(COMBINE_TPL_ROOT):
            if not name.startswith("combine_templates_"): continue
            subdir = os.path.join(COMBINE_TPL_ROOT, name)
            fpath = os.path.join(subdir, filename)
            items = read_jsonl_templates(fpath)
            if not items: continue
            slot = name[len("combine_templates_"):]
            if slot.isdigit():
                if int(slot) == 0:
                    neg_sent_tpls.extend(items)
                elif int(slot) == 11:
                    pos_11_tpls.extend(items)
                else:
                    pos_other_tpls.extend(items)
        return {"pos_11": pos_11_tpls, "pos_other": pos_other_tpls, "neg_sent": neg_sent_tpls}

    def apply_noise(text: str, spans: List[Tuple[int,int]]) -> Tuple[str, List[Tuple[int,int]]]:
        def toggle_case(s: str):
            return "".join(c.upper() if "a"<=c<="z" else c.lower() if "A"<=c<="Z" else c for c in s)
        def to_fullwidth(c: str):
            return chr(ord(c) + 0xFEE0) if 33 <= ord(c) <= 126 else "\u3000" if c==" " else c
        def to_halfwidth(c: str):
            return " " if c=="\u3000" else chr(ord(c) - 0xFEE0) if 65281 <= ord(c) <= 65374 else c
        def fullwidth_halfwidth_flip(s: str):
            out=[]
            for ch in s:
                if unicodedata.east_asian_width(ch) in ("F","W") and random.random()<0.5: out.append(to_halfwidth(ch))
                elif random.random() < 0.2: out.append(to_fullwidth(ch))
                else: out.append(ch)
            return "".join(out)
        def replace_separators(s: str):
            trans = {"-": random.choice(["—","-","–"]), "/": random.choice(["/","／","∕"]),
                     ".": random.choice([".","·","．"]), ":": random.choice([":","："])}
            return "".join(trans.get(ch, ch) for ch in s)

        if random.random() < 0.15: text = toggle_case(text)
        if random.random() < 0.10: text = fullwidth_halfwidth_flip(text)
        if random.random() < 0.10: text = replace_separators(text)
        return text, spans

    def fill_and_align_to_bio(tpl, lang, addrs_map, times_map, non_addr_map, non_time_map):
        parts=[]; entities=[]; last_pos=0
        for m in re.finditer(r'\{address\}|\{time\}', tpl):
            parts.append(tpl[last_pos:m.start()])
            if m.group() == "{address}":
                if random.random() < NEG_ADDR_PROB: rep = random.choice(non_addr_map.get(lang, [""]))
                else:
                    rep = random.choice(addrs_map.get(lang, ["Main Street"]))
                    start_char = sum(len(p) for p in parts)
                    entities.append((start_char, start_char+len(rep), "ADDRESS"))
                parts.append(rep)
            elif m.group() == "{time}":
                if random.random() < NEG_TIME_PROB: rep = random.choice(non_time_map.get(lang, [""]))
                else:
                    rep = random.choice(times_map.get(lang, ["2024-07-01"]))
                    start_char = sum(len(p) for p in parts)
                    entities.append((start_char, start_char+len(rep), "TIME"))
                parts.append(rep)
            last_pos = m.end()
        parts.append(tpl[last_pos:])
        text = "".join(parts)
        text, new_spans = apply_noise(text, [(s,e) for s, e, _ in entities])
        for i, (_, _, typ) in enumerate(entities):
            entities[i] = (new_spans[i][0], new_spans[i][1], typ)

        enc = tokenizer(text, add_special_tokens=False, return_offsets_mapping=True)
        tokens = tokenizer.convert_ids_to_tokens(enc["input_ids"])
        offsets = enc["offset_mapping"]
        labels = ["O"] * len(tokens)
        for start_char, end_char, ent_type in entities:
            first_flag = True
            for i, (tok_s, tok_e) in enumerate(offsets):
                if tok_e <= start_char or tok_s >= end_char: continue
                labels[i] = f"B-{ent_type}" if first_flag else f"I-{ent_type}"
                first_flag = False
        return tokens, labels

    addrs_map={lang: read_lines(os.path.join(REAL_ADDR_ROOT,f"{lang}.txt")) for lang in LANGS}
    times_map={lang: read_lines(os.path.join(REAL_TIME_ROOT,f"{lang}.txt")) for lang in LANGS}
    non_addr_map={lang: read_lines(os.path.join(NON_ADDR_ROOT,f"{lang}.txt")) for lang in LANGS}
    non_time_map={lang: read_lines(os.path.join(NON_TIME_ROOT,f"{lang}.txt")) for lang in LANGS}

    merged=[]
    for lang in LANGS:
        templates = load_templates_for_lang(lang)
        total_target = N_PER_LANG
        neg_target = int(total_target * per_00)
        pos_target_total = total_target - neg_target
        pos_11_target = int(pos_target_total * per_11)
        pos_other_target = pos_target_total - pos_11_target
        pos_11_tpls = templates.get("pos_11", ["{address} {time}"])
        pos_other_tpls = templates.get("pos_other", ["{address} {time}"])
        neg_tpls = templates.get("neg_sent", [])

        samples=[]
        for _ in range(pos_11_target):
            toks, lbls = fill_and_align_to_bio(random.choice(pos_11_tpls), lang, addrs_map, times_map, non_addr_map, non_time_map)
            samples.append((toks, lbls))
        for _ in range(pos_other_target):
            toks, lbls = fill_and_align_to_bio(random.choice(pos_other_tpls), lang, addrs_map, times_map, non_addr_map, non_time_map)
            samples.append((toks, lbls))
        for _ in range(neg_target):
            if neg_tpls:
                tpl = random.choice(neg_tpls)
                toks = tpl.split(); lbls = ["O"]*len(toks)
                samples.append((toks, lbls))
        merged.extend(samples)

        with open(PER_LANG_CONLL(lang), "w", encoding="utf-8") as f:
            for toks, lbls in samples:
                for t, y in zip(toks, lbls): f.write(f"{t} {y}\n")
                f.write("\n")
        print(f"[生成]{lang}: {len(samples)} 条样本")

    import random; random.shuffle(merged)
    with open(MERGED_CONLL, "w", encoding="utf-8") as f:
        for toks, lbls in merged:
            for t,y in zip(toks,lbls): f.write(f"{t} {y}\n")
            f.write("\n")
    print(f"[生成] 合并集 {len(merged)} 条 -> {MERGED_CONLL}")


# ===== Step2: 训练模型 =====
def train_model():
    import torch
    from sklearn.model_selection import train_test_split
    from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, set_seed
    from datasets import Dataset
    import evaluate
    from packaging import version
    import transformers

    set_seed(42)
    LABELS = ["O", "B-ADDRESS", "I-ADDRESS", "B-TIME", "I-TIME"]
    label2id = {l: i for i, l in enumerate(LABELS)}
    id2label = {i: l for l, i in label2id.items()}

    def read_conll(path):
        toks_all, labs_all = [], []
        toks, labs = [], []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                line=line.strip()
                if not line:
                    if toks: toks_all.append(toks); labs_all.append(labs); toks,labs=[],[]
                    continue
                t,y = (line.split(" ",1)+["O"])[:2]
                if y not in label2id: y="O"
                toks.append(t); labs.append(y)
        if toks: toks_all.append(toks); labs_all.append(labs)
        return toks_all, labs_all

    tokens_all, tags_all = read_conll(os.path.join(TRAIN_DATA_DIR, "train_all.conll"))
    tr_idx, va_idx = train_test_split(range(len(tokens_all)), test_size=0.1, random_state=42)
    train_tokens = [tokens_all[i] for i in tr_idx]; train_tags = [tags_all[i] for i in tr_idx]
    val_tokens   = [tokens_all[i] for i in va_idx]; val_tags   = [tags_all[i] for i in va_idx]

    tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", use_fast=True)

    def build_dataset(tokens_list, tags_list):
        CLS_ID = tokenizer.cls_token_id
        SEP_ID = tokenizer.sep_token_id
        recs=[]
        for toks, labs in zip(tokens_list, tags_list):
            ids = tokenizer.convert_tokens_to_ids(toks)
            input_ids = [CLS_ID] + ids + [SEP_ID]
            labels_ids = [-100] + [label2id[l] for l in labs] + [-100]
            attn = [1]*len(input_ids)
            recs.append({"input_ids": input_ids, "labels": labels_ids, "attention_mask": attn})
        return Dataset.from_list(recs)

    class CollatorPT:
        def __init__(self, pad_id): self.pad_id=pad_id
        def __call__(self, batch):
            import torch
            maxlen = max(len(x["input_ids"]) for x in batch)
            ids=[x["input_ids"]+[self.pad_id]*(maxlen-len(x["input_ids"])) for x in batch]
            attn=[x["attention_mask"]+[0]*(maxlen-len(x["attention_mask"])) for x in batch]
            labs=[x["labels"]+[-100]*(maxlen-len(x["labels"])) for x in batch]
            return {"input_ids": torch.tensor(ids), "attention_mask": torch.tensor(attn), "labels": torch.tensor(labs)}

    ds_train=build_dataset(train_tokens, train_tags)
    ds_val=build_dataset(val_tokens, val_tags)
    model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(LABELS), id2label=id2label, label2id=label2id)
    seqeval = evaluate.load("seqeval")
    def compute_metrics(p):
        preds = p.predictions.argmax(-1)
        labels = p.label_ids
        preds_list, labels_list = [], []
        for pl, gl in zip(preds, labels):
            preds_list.append([id2label[i] for i,lab in zip(pl, gl) if lab != -100])
            labels_list.append([id2label[lab] for i,lab in zip(pl, gl) if lab != -100])
        res = seqeval.compute(predictions=preds_list, references=labels_list)
        return {"f1": res["overall_f1"], "precision": res["overall_precision"], "recall": res["overall_recall"]}

    common_args = dict(
        output_dir=MODEL_OUTPUT_DIR, num_train_epochs=3,
        per_device_train_batch_size=128, per_device_eval_batch_size=128,
        learning_rate=2.5e-5, logging_steps=50,
        save_strategy="epoch",load_best_model_at_end=True,
        metric_for_best_model="f1", greater_is_better=True,
        report_to="none", fp16=torch.cuda.is_available(), seed=42
    )
    if version.parse(transformers.__version__) >= version.parse("4.55.0"):
        args = TrainingArguments(**common_args, eval_strategy="epoch")
    else:
        args = TrainingArguments(**common_args, evaluation_strategy="epoch")

    trainer = Trainer(model=model, args=args, train_dataset=ds_train, eval_dataset=ds_val,
                      tokenizer=tokenizer, data_collator=CollatorPT(tokenizer.pad_token_id),
                      compute_metrics=compute_metrics)
    trainer.train()
    trainer.save_model(MODEL_OUTPUT_DIR)
    tokenizer.save_pretrained(MODEL_OUTPUT_DIR)
    print("[训练] 模型已保存到:", MODEL_OUTPUT_DIR)

# ===== Step3: 测试、整体平均提升才更新最佳集 =====
def test_and_update_best_dataset_per_lang():
    import os, json, torch, regex as re, shutil
    import plotly.graph_objects as go
    from transformers import AutoTokenizer, AutoModelForTokenClassification

    replace_happened_this_round = False

    # 历史最佳分数
    if os.path.isfile(BEST_RECORD_FILE):
        with open(BEST_RECORD_FILE, "r", encoding="utf-8") as f:
            best_scores = json.load(f)
    else:
        best_scores = {"overall_best_mean": None, "per_lang_scores": {}}

    tokenizer = AutoTokenizer.from_pretrained(MODEL_OUTPUT_DIR, use_fast=True)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_OUTPUT_DIR)
    model.eval()
    if torch.cuda.is_available():
        model.cuda()
    id2label = {i: l for i, l in enumerate(LABELS)}

    time_F1_list, addr_F1_list = [], []
    per_lang_results = {}

    for lang in LANGS:
        json_file = os.path.join(TEST_DATA_ROOT, f"{lang}.json")
        cur_time_f1, cur_addr_f1 = 0.0, 0.0

        if os.path.isfile(json_file):
            with open(json_file, "r", encoding="utf-8") as f:
                data = json.load(f)

            metrics_time_tok, metrics_addr_tok = [], []

            for sample in data:
                sent = sample.get("sentence")
                gold_time = sample.get("time_tokens", [])
                gold_addr = sample.get("location_tokens", [])

                enc = tokenizer(sent, return_offsets_mapping=True, truncation=True,
                                max_length=256, return_tensors="pt")
                offsets = enc.pop("offset_mapping")
                if torch.cuda.is_available():
                    enc = {k: v.cuda() for k, v in enc.items()}
                with torch.no_grad():
                    logits = model(**enc).logits[0]
                pred_ids = logits.argmax(-1).detach().cpu().numpy().tolist()
                input_ids = enc["input_ids"][0].detach().cpu().tolist()
                offsets = offsets[0].detach().cpu().tolist()
                toks = tokenizer.convert_ids_to_tokens(input_ids)
                keep = []
                for tok, (st, ed) in zip(toks, offsets):
                    if tok in (tokenizer.cls_token, tokenizer.sep_token, "<s>", "</s>", tokenizer.pad_token):
                        keep.append(False)
                    elif (st == 0 and ed == 0):
                        keep.append(False)
                    else:
                        keep.append(True)
                toks = [t for t, k in zip(toks, keep) if k]
                labs = [id2label[i] for i, k in zip(pred_ids, keep) if k]

                pred_time_toks = [t for t, l in zip(toks, labs) if l.endswith("TIME")]
                pred_addr_toks = [t for t, l in zip(toks, labs) if l.endswith("ADDRESS")]

                def token_level_metrics(pred_tokens, gold_tokens):
                    pred_set, gold_set = set(pred_tokens), set(gold_tokens)
                    if not gold_set: return None
                    tp = len(pred_set & gold_set)
                    precision = tp / len(pred_set) if pred_set else 0.0
                    recall = tp / len(gold_set) if gold_set else 0.0
                    f1 = 2 * precision * recall / (precision + recall) if (precision+recall) > 0 else 0.0
                    return precision, recall, f1

                m_time = token_level_metrics(pred_time_toks, sum([tokenizer.tokenize(gt) for gt in gold_time], []))
                m_addr = token_level_metrics(pred_addr_toks, sum([tokenizer.tokenize(ga) for ga in gold_addr], []))
                if m_time: metrics_time_tok.append(m_time)
                if m_addr: metrics_addr_tok.append(m_addr)

            def avg_metrics(metrics):
                if not metrics: return (0,0,0)
                p = sum(m[0] for m in metrics) / len(metrics)
                r = sum(m[1] for m in metrics) / len(metrics)
                f1 = sum(m[2] for m in metrics) / len(metrics)
                return p, r, f1

            cur_time_f1 = avg_metrics(metrics_time_tok)[2]
            cur_addr_f1 = avg_metrics(metrics_addr_tok)[2]

        time_F1_list.append(cur_time_f1)
        addr_F1_list.append(cur_addr_f1)
        per_lang_results[lang] = {
            "addr_f1": cur_addr_f1,
            "time_f1": cur_time_f1,
            "mean_f1": (cur_time_f1 + cur_addr_f1) / 2
        }

    # 计算整体平均
    overall_mean_f1 = sum([(t + a) / 2 for t, a in zip(time_F1_list, addr_F1_list)]) / len(LANGS)
    prev_best_mean = best_scores.get("overall_best_mean")

    if prev_best_mean is None or overall_mean_f1 > prev_best_mean + IMPROVE_THRESHOLD:
        print(f"[整体更新] 平均F1提升: {prev_best_mean} → {overall_mean_f1}")
        # 全量替换 current_best
        for lang in LANGS:
            src_file = os.path.join(TRAIN_DATA_DIR, f"train_{lang}.conll")
            if os.path.isfile(src_file):
                shutil.copyfile(src_file, os.path.join(BEST_SAVE_DIR, f"{lang}.conll"))

        # 更新记录
        best_scores["overall_best_mean"] = overall_mean_f1
        best_scores["per_lang_scores"] = per_lang_results
        replace_happened_this_round = True
    else:
        print(f"[保持] 整体平均F1无提升: {prev_best_mean} → {overall_mean_f1}")
        best_scores["per_lang_scores"] = per_lang_results

    with open(BEST_RECORD_FILE, "w", encoding="utf-8") as f:
        json.dump(best_scores, f, ensure_ascii=False, indent=2)

    return replace_happened_this_round
# ===== 主迭代流程 =====
for it in range(1, ITERATIONS+1):
    print(f"\n=== 迭代 {it}/{ITERATIONS} ===")
    prepare_training_data(it, last_replace_happened)   # Step1
    train_model()
    last_replace_happened = test_and_update_best_dataset_per_lang()

print("[完成] 最优训练集保存在:", BEST_SAVE_DIR)
# 迭代全部完成后可视化每个语言的F1
import plotly.graph_objects as go

if os.path.isfile(BEST_RECORD_FILE):
    with open(BEST_RECORD_FILE, "r", encoding="utf-8") as f:
        best_scores = json.load(f)
    per_lang = best_scores.get("per_lang_scores", {})

    langs = list(per_lang.keys())
    addr_f1 = [per_lang[lang]["addr_f1"] for lang in langs]
    time_f1 = [per_lang[lang]["time_f1"] for lang in langs]
    mean_f1 = [per_lang[lang]["mean_f1"] for lang in langs]

    fig = go.Figure()
    fig.add_trace(go.Bar(x=langs, y=addr_f1, name="Address F1"))
    fig.add_trace(go.Bar(x=langs, y=time_f1, name="Time F1"))
    fig.add_trace(go.Bar(x=langs, y=mean_f1, name="Mean F1"))
    fig.update_layout(title="各语言F1表现", barmode="group", xaxis_title="语言", yaxis_title="F1分数")
    fig.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.536800,0.043118,0.939858,0.928363,0.951642
2,0.038300,0.010884,0.984259,0.979314,0.989254
3,0.013800,0.008294,0.991361,0.989298,0.993433


[训练] 模型已保存到: /content/ner_xlmr_out
[保持] 整体平均F1无提升: 0.7215325783548443 → 0.6871634003745675

=== 迭代 3/3 ===
[Step1] 第3轮：上一轮无替换 → 软更新探索
[软更新] 保留loss接近中值的样本，比例=0.95
[软更新] 简体中文: 保留475条(中值loss优先), 新增25条 → /content/drive/MyDrive/NER/train_datasets_AC/train_简体中文.conll
[软更新] 繁体中文: 保留475条(中值loss优先), 新增25条 → /content/drive/MyDrive/NER/train_datasets_AC/train_繁体中文.conll
[软更新] 英语: 保留475条(中值loss优先), 新增25条 → /content/drive/MyDrive/NER/train_datasets_AC/train_英语.conll
[软更新] 丹麦语: 保留475条(中值loss优先), 新增25条 → /content/drive/MyDrive/NER/train_datasets_AC/train_丹麦语.conll
[软更新] 俄语: 保留475条(中值loss优先), 新增25条 → /content/drive/MyDrive/NER/train_datasets_AC/train_俄语.conll
[软更新] 土耳其语: 保留475条(中值loss优先), 新增25条 → /content/drive/MyDrive/NER/train_datasets_AC/train_土耳其语.conll
[软更新] 德语: 保留475条(中值loss优先), 新增25条 → /content/drive/MyDrive/NER/train_datasets_AC/train_德语.conll
[软更新] 意大利语: 保留475条(中值loss优先), 新增25条 → /content/drive/MyDrive/NER/train_datasets_AC/train_意大利语.conll
[软更新] 日语: 保留475条(中值loss优先), 新增25条 → /content/drive/M

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2007143435.py:742: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args, train_dataset=ds_train, eval_dataset=ds_val,


Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.529300,0.035312,0.933491,0.920163,0.947211
2,0.032400,0.008390,0.982970,0.979167,0.986803
3,0.013700,0.007809,0.988038,0.985092,0.991002


[训练] 模型已保存到: /content/ner_xlmr_out
[保持] 整体平均F1无提升: 0.7215325783548443 → 0.7035207090452592
[完成] 最优训练集保存在: /content/drive/MyDrive/NER/current_best


### **测试模型**

##### **使用json测试集测试**
批量测试，得到准确率指标与错误集

有后处理措施，非纯模型输出（比如去掉常见错识别字符等）

数据集格式如下，命名为语言名.txt(中文.txt)：

```
[
  {
    "id": 1,
    "sentence": "请确认我们的见面地点是在天凝镇政府。",
    "time_tokens": [],
    "location_tokens": [
      "天凝镇政府"
    ]
  },
  {
    "id": 2,
    "sentence": "你能告诉我东川府铸钱局遗址（新局）的具体方向吗？",
    "time_tokens": [],
    "location_tokens": [
      "东川府铸钱局遗址（新局）"
    ]
  }

]
```
输出各语言的测试结果；保存测试结果为txt文件；保存识别错误的部分为json。输出示例：

```
== Exact-match (entity text equal) ==
ADDRESS   P=0.8924  R=0.7582  F1=0.8198  (#tp=2123, #fp=256, #fn=677)

== Partial-match (char-overlap>0, greedy) ==
ADDRESS   P=0.9996  R=0.8811  F1=0.9366  (#tp=2378, #fp=1, #fn=321)
total:700, mismatch:359,acc:48.714285714285715%

[Saved] non-exact matched cases -> /content/mismatches_4/意大利语_mismatches.json (count=359)

== Address-specific Accuracies ==
First-Address Exact Acc   : 0.9343  (654/700)
First-Address Partial Acc : 0.9757  (683/700)
First vs Any    Exact Acc : 0.9557  (669/700)
First vs Any    Partial Acc: 1.0000 (700/700)
```




In [ ]:
# 使用json数据集测试tflite模型

import os, json, numpy as np, tensorflow as tf
from typing import List, Tuple, Dict
from collections import Counter, defaultdict

import io
from contextlib import redirect_stdout

# 测试集名字列表（请按实际情况修改，或把测试数据命名为"语言名.txt"）
LANGUAGES = ["中文","英语","丹麦语","俄语","土耳其语","德语","意大利语","日语","法语","瑞典语","荷兰语","葡萄牙语","西班牙语","韩语"]

TEST_JSON_DIR  = "/content/4"            # 测试集所在目录
OUTPUT_TXT_DIR = "/content/eval_txt_4"        # 各语言评测结果(txt)输出目录
MISMATCH_DIR   = "/content/mismatches_4"      # 各语言错误项(mismatches.json)输出目录
os.makedirs(OUTPUT_TXT_DIR, exist_ok=True)
os.makedirs(MISMATCH_DIR,   exist_ok=True)



# --- Paths (edit as needed) ---
TFLITE_PATH     = "/content/drive/MyDrive/model_dy_quant.tflite"     # TFLite模型
TOKENIZER_DIRS  = ["/content/drive/MyDrive/checkpoint-1235", "xlm-roberta-base"]  # 优先从checkpoint加载

# TEST_JSON_PATH  = "/content/中文.json"                            # 测试集JSON
# MISMATCH_OUT    = "/content/mismatches.json"                          # 未完全匹配样本输出

# # 错误统计输出（6个文件）
# ERR_TIME_MISSED   = "/content/errors_time_missed.json"
# ERR_TIME_PARTIAL  = "/content/errors_time_partial.json"
# ERR_TIME_SPURIOUS = "/content/errors_time_spurious.json"
# ERR_ADDR_MISSED   = "/content/errors_address_missed.json"
# ERR_ADDR_PARTIAL  = "/content/errors_address_partial.json"
# ERR_ADDR_SPURIOUS = "/content/errors_address_spurious.json"

# --- Labels (fallback) ---
LABELS = ["O","B-TIME","I-TIME","B-ADDRESS","I-ADDRESS"]
ID2LABEL = {i: l for i, l in enumerate(LABELS)}

# --- 1) Tokenizer ---
from transformers import AutoTokenizer
tok = None
for p in TOKENIZER_DIRS:
    try:
        tok = AutoTokenizer.from_pretrained(p, use_fast=True)
        # 尝试从 config 里取 id2label（若存在）
        cfg_path = os.path.join(p, "config.json")
        if os.path.isfile(cfg_path):
            with open(cfg_path, "r", encoding="utf-8") as f:
                cfg = json.load(f)
            if "id2label" in cfg and isinstance(cfg["id2label"], dict):
                _id2 = {int(k): v for k, v in cfg["id2label"].items()}
                ID2LABEL = {i: _id2[i] for i in sorted(_id2.keys())}
        print(f"[OK] Loaded tokenizer from: {p} | labels: {[ID2LABEL[i] for i in range(len(ID2LABEL))]}")
        break
    except Exception:
        pass
assert tok is not None, "Tokenizer not found from TOKENIZER_DIRS."

# --- 2) TFLite Interpreter ---
try:
    from ai_edge_litert.python.interpreter import Interpreter  # type: ignore
    interpreter = Interpreter(model_path=TFLITE_PATH,num_threads=os.cpu_count())
    print(f"[OK] Using ai_edge_litert.Interpreter: {TFLITE_PATH}")
except Exception:
    interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH, num_threads=os.cpu_count())
    print(f"[OK] Using tf.lite.Interpreter: {TFLITE_PATH} (deprecated warning is OK)")

interpreter.allocate_tensors()

# --- 3) IO mapping helpers ---
def map_io(interp):
    inps = interp.get_input_details()
    outs = interp.get_output_details()
    idx = {"input_ids": None, "attention_mask": None, "token_type_ids": None}
    for i, info in enumerate(inps):
        name = info["name"].lower()
        if "input_ids" in name or "ids" in name:
            if idx["input_ids"] is None:
                idx["input_ids"] = i
        elif "attention_mask" in name or "mask" in name:
            idx["attention_mask"] = i
        elif "token_type_ids" in name or "segment" in name:
            idx["token_type_ids"] = i
    if len(inps) == 1 and idx["input_ids"] is None:
        idx["input_ids"] = 0
    if len(inps) == 2 and idx["attention_mask"] is None:
        other = 1 if idx["input_ids"] == 0 else 0
        idx["attention_mask"] = other
    return inps, outs, idx

INPS, OUTS, IDX = map_io(interpreter)
print("[IO] Inputs:", [d["name"] for d in INPS])
print("[IO] Outputs:", [d["name"] for d in OUTS])
print("[IO] Mapped index:", IDX)

def get_fixed_len(interp, fallback=128) -> int:
    inps_local = interp.get_input_details()
    L = None
    if IDX["input_ids"] is not None:
        shape = inps_local[IDX["input_ids"]]["shape"]
        if len(shape) == 2 and shape[1] > 0:
            L = int(shape[1])
    return int(L if L else fallback)

FIX_LEN = get_fixed_len(interpreter, fallback=128)
print("[IO] Model fixed/padded sequence length:", FIX_LEN)

# --- 4) Inference ---
def tflite_infer_sentence(sentence: str):
    enc = tok(
        sentence,
        return_tensors="np",
        padding="max_length",
        truncation=True,
        max_length=FIX_LEN,
        return_offsets_mapping=True,
    )
    input_ids = enc["input_ids"].astype(np.int32)
    attn      = enc["attention_mask"].astype(np.int32)
    ttids     = enc.get("token_type_ids", np.zeros_like(input_ids, dtype=np.int32))


    if IDX["input_ids"] is not None:
        interpreter.set_tensor(INPS[IDX["input_ids"]]["index"], input_ids)
    if IDX["attention_mask"] is not None:
        interpreter.set_tensor(INPS[IDX["attention_mask"]]["index"], attn)
    if IDX["token_type_ids"] is not None:
        interpreter.set_tensor(INPS[IDX["token_type_ids"]]["index"], ttids)

    interpreter.invoke()
    logits = interpreter.get_tensor(OUTS[0]["index"])
    if logits.ndim == 2:  # [L, C] -> [1, L, C]
        logits = logits[None, ...]
    return enc, logits

# --- 5) Decode logits -> entity char-spans ---

# ADDRESS 的前后去除字符

TRIM_CHARS_ADDR = set([' ', '\t', '在', '的', '.', ',', 'の', '，'])

def trim_addr_edges(sentence: str, start: int, end: int) -> Tuple[int, int]:
    """收缩 span：去掉前后空格/在/的/./,（只作用于 ADDRESS）"""
    while start < end and sentence[start] in TRIM_CHARS_ADDR:
        start += 1
    while end > start and sentence[end - 1] in TRIM_CHARS_ADDR:
        end -= 1
    return start, end


def decode_entities_by_span(sentence: str, enc, logits) -> Dict[str, List[Tuple[int,int,str]]]:
    """
    返回：
      {
        "TIME":    [(start,end,text), ...],
        "ADDRESS": [(start,end,text), ...]
      }
    """
    offsets = enc["offset_mapping"][0].tolist()
    pred_ids = logits[0].argmax(-1).tolist()
    labels = [ID2LABEL.get(i, "O") for i in pred_ids]

    entities: Dict[str, List[Tuple[int,int,str]]] = {"TIME": [], "ADDRESS": []}
    cur_type, cur_start, cur_end = None, None, None
    last_label = "O"

    for (s,e), y in zip(offsets, labels):
        if (s == e) or (s is None) or (e is None):
            if y.startswith("B-") or y == "O":
                if cur_type is not None:
                    text = sentence[cur_start:cur_end]
                    if text.strip():
                        entities[cur_type].append((cur_start, cur_end, text))
                cur_type, cur_start, cur_end = None, None, None
            last_label = y
            continue

        if y.startswith("B-") and y != last_label:
            if cur_type is not None:
                text = sentence[cur_start:cur_end]
                if text.strip():
                    entities[cur_type].append((cur_start, cur_end, text))
            cur_type = y[2:]
            cur_start, cur_end = s, e
        elif (y.startswith("I-") and cur_type == y[2:]) or (y.startswith("B-") and y == last_label):
            if cur_start is None:
                cur_type = y[2:]
                cur_start, cur_end = s, e
            else:
                cur_end = e
        else:
            if cur_type is not None:
                text = sentence[cur_start:cur_end]
                if text.strip():
                    entities[cur_type].append((cur_start, cur_end, text))
            cur_type, cur_start, cur_end = None, None, None

        last_label = y

    if cur_type is not None:
        text = sentence[cur_start:cur_end]
        if text.strip():
            entities[cur_type].append((cur_start, cur_end, text))

    # 对 ADDRESS 的 span 做边界收缩，并同步更新文本（去除"在"、"的"、","等）
    addr_trimmed = []
    for (a, b, t) in entities["ADDRESS"]:
        a2, b2 = trim_addr_edges(sentence, a, b)
        if a2 < b2:
            addr_trimmed.append((a2, b2, sentence[a2:b2]))
    entities["ADDRESS"] = addr_trimmed

    # 统一再过滤一次空串
    for k in list(entities.keys()):
        entities[k] = [(a, b, t) for (a, b, t) in entities[k] if t.strip() != ""]

    return entities


# --- 6) Metric helpers ---
def normalize_text(s: str) -> str:
    return "".join(s.split())

def exact_match_sets(pred_texts: List[str], gold_texts: List[str]) -> Tuple[int,int,int]:
    P = [normalize_text(x) for x in pred_texts]
    G = [normalize_text(x) for x in gold_texts]
    from collections import Counter
    cP, cG = Counter(P), Counter(G)
    tp = sum(min(cP[k], cG.get(k, 0)) for k in cP)
    fp = sum(cP[k] - min(cP[k], cG.get(k, 0)) for k in cP)
    fn = sum(cG[k] - min(cG[k], cP.get(k, 0)) for k in cG)
    return tp, fp, fn

def span_overlap(a: Tuple[int,int], b: Tuple[int,int]) -> int:
    s = max(a[0], b[0]); e = min(a[1], b[1])
    return max(0, e - s)

def partial_match_tp(pred_spans: List[Tuple[int,int]], gold_spans: List[Tuple[int,int]]) -> int:
    used = [False]*len(gold_spans)
    tp = 0
    for p in pred_spans:
        for j, g in enumerate(gold_spans):
            if not used[j] and span_overlap(p, g) > 0:
                used[j] = True
                tp += 1
                break
    return tp

def prf(tp:int, fp:int, fn:int) -> Tuple[float,float,float]:
    prec = tp / (tp + fp + 1e-9)
    rec  = tp / (tp + fn + 1e-9)
    f1   = 2*prec*rec / (prec+rec+1e-9) if (prec+rec)>0 else 0.0
    return prec, rec, f1

def evaluate_one_file(TEST_JSON_PATH: str, MISMATCH_OUT: str) -> str:
    # --- 7) Load test set ---
    with open(TEST_JSON_PATH, "r", encoding="utf-8") as f:
        test_data = json.load(f)
        print(f"Loaded {len(test_data)} test records from {TEST_JSON_PATH}")
    assert isinstance(test_data, list), "Test JSON must be a list of records."

    # --- 8) Evaluate + error buckets ---
    exact_tp = {"TIME":0, "ADDRESS":0}
    exact_fp = {"TIME":0, "ADDRESS":0}
    exact_fn = {"TIME":0, "ADDRESS":0}

    part_tp  = {"TIME":0, "ADDRESS":0}
    part_fp  = {"TIME":0, "ADDRESS":0}
    part_fn  = {"TIME":0, "ADDRESS":0}

    mismatches = []

    # --- New: Acc counters for 2 address-specific metrics ---
    first_exact_hits = 0
    first_partial_hits = 0
    first_denom = 0          # 仅统计“gold 至少有一个地址”的句子

    first_any_exact_hits = 0
    first_any_partial_hits = 0


    # 错误桶（按频次）
    missed_cnt   = {"TIME": Counter(), "ADDRESS": Counter()}               # gold 无重叠
    spurious_cnt = {"TIME": Counter(), "ADDRESS": Counter()}               # pred 无重叠
    partial_cnt_gold_pred = {"TIME": Counter(), "ADDRESS": Counter()}      # (gold_text, pred_text) 配对
    partial_cnt_gold_only = {"TIME": Counter(), "ADDRESS": Counter()}      # 只有gold可定位但pred_text为空时的兜底

    def find_spans(sentence: str, texts: List[str]) -> List[Tuple[int,int,str]]:
        spans = []
        cur = 0
        for t in texts:
            t_ = t.strip()
            if not t_:  # 空串不计
                continue
            pos = sentence.find(t_, cur)
            if pos < 0:
                # 去空白再找（粗糙，若找到也无法精确映射，忽略）
                s_norm = "".join(sentence.split())
                t_norm = "".join(t_.split())
                pos_n = s_norm.find(t_norm)
                if pos_n >= 0:
                    # 无法回映到原文精确位置，跳过
                    pass
            else:
                spans.append((pos, pos+len(t_), t_))
                cur = pos + len(t_)
        return spans

    i = 0
    for rec in test_data:
        if(i%1000 == 0):
          print(i)
        i+=1
        sid = rec.get("id")
        sent = rec.get("sentence", "")
        gold_time = rec.get("time_tokens", []) or []
        gold_addr = rec.get("location_tokens", []) or []

        # 推理
        enc, logits = tflite_infer_sentence(sent)
        pred = decode_entities_by_span(sent, enc, logits)  # {"TIME":[(s,e,text)...], "ADDRESS":[...]}

        # 预测文本
        pred_time_texts = [t for (_,_,t) in pred["TIME"]]
        pred_addr_texts = [t for (_,_,t) in pred["ADDRESS"]]

        # ------ Exact by text ------
        tp_t, fp_t, fn_t = exact_match_sets(pred_time_texts, gold_time)
        tp_a, fp_a, fn_a = exact_match_sets(pred_addr_texts, gold_addr)
        exact_tp["TIME"] += tp_t; exact_fp["TIME"] += fp_t; exact_fn["TIME"] += fn_t
        exact_tp["ADDRESS"] += tp_a; exact_fp["ADDRESS"] += fp_a; exact_fn["ADDRESS"] += fn_a

        # ------ Partial by span-overlap ------
        gold_time_spans = find_spans(sent, gold_time)
        gold_addr_spans = find_spans(sent, gold_addr)
        pred_time_spans = [(s,e,t) for (s,e,t) in pred["TIME"]]
        pred_addr_spans = [(s,e,t) for (s,e,t) in pred["ADDRESS"]]

        # 首地址完全/部分匹配的准确率 ---
        # 仅在 gold 至少有一个地址时计入分母
        if len(gold_addr) > 0:
            first_denom += 1
            # exact：文本完全一致（去空白对齐）
            if len(pred_addr_texts) > 0:
                if normalize_text(pred_addr_texts[0]) == normalize_text(gold_addr[0]):
                    first_exact_hits += 1

            # partial：首地址的 span 有任意重叠（包含 exact 情况）
            if len(pred_addr_spans) > 0 and len(gold_addr_spans) > 0:
                p0 = (pred_addr_spans[0][0], pred_addr_spans[0][1])
                g0 = (gold_addr_spans[0][0], gold_addr_spans[0][1])
                if span_overlap(p0, g0) > 0:
                    first_partial_hits += 1

        # 预测首地址 vs 任意 gold 地址（exact / partial）
        if len(gold_addr) > 0:
          # 分母沿用 first_denom，不需要单独 any_denom
          gold_set_norm = set(map(normalize_text, gold_addr))

          # exact：预测首地址文本是否等于任一 gold 文本（去空白）
          if len(pred_addr_texts) > 0:
              if normalize_text(pred_addr_texts[0]) in gold_set_norm:
                  first_any_exact_hits += 1

          # partial：预测首地址的 span 是否与任一 gold span 有重叠
          if len(pred_addr_spans) > 0 and len(gold_addr_spans) > 0:
              ps, pe, _ = pred_addr_spans[0]
              has_overlap = any(span_overlap((ps, pe), (gs, ge)) > 0 for (gs, ge, _) in gold_addr_spans)
              if has_overlap:
                  first_any_partial_hits += 1



        # 计算TP，预测集合大小=TP+FP，标注集合大小=TP+FN
        t_tp = partial_match_tp([(s,e) for (s,e,_) in pred_time_spans], [(s,e) for (s,e,_) in gold_time_spans])
        a_tp = partial_match_tp([(s,e) for (s,e,_) in pred_addr_spans], [(s,e) for (s,e,_) in gold_addr_spans])
        part_tp["TIME"] += t_tp
        part_tp["ADDRESS"] += a_tp
        part_fp["TIME"] += max(0, len(pred_time_spans) - t_tp)
        part_fp["ADDRESS"] += max(0, len(pred_addr_spans) - a_tp)
        part_fn["TIME"] += max(0, len(gold_time_spans) - t_tp)
        part_fn["ADDRESS"] += max(0, len(gold_addr_spans) - a_tp)

        # - 错误桶填充 -
        # 1) missed: 对于每个 gold，没有与任何 pred 重叠
        for (gs, ge, gt) in gold_time_spans:
            if all(span_overlap((gs,ge), (ps,pe)) == 0 for (ps,pe,_) in pred_time_spans):
                missed_cnt["TIME"][gt] += 1
        for (gs, ge, gt) in gold_addr_spans:
            if all(span_overlap((gs,ge), (ps,pe)) == 0 for (ps,pe,_) in pred_addr_spans):
                missed_cnt["ADDRESS"][gt] += 1

        # 2) spurious: 对于每个 pred，没有与任何 gold 重叠
        for (ps, pe, pt) in pred_time_spans:
            if all(span_overlap((ps,pe), (gs,ge)) == 0 for (gs,ge,_) in gold_time_spans):
                spurious_cnt["TIME"][pt] += 1
        for (ps, pe, pt) in pred_addr_spans:
            if all(span_overlap((ps,pe), (gs,ge)) == 0 for (gs,ge,_) in gold_addr_spans):
                spurious_cnt["ADDRESS"][pt] += 1

        # 3) partial: gold 与 pred 有重叠但文本不完全一致（记录 (gold_text, pred_text) 对）
        #    一个 gold 只取与之重叠的第一个 pred 作为代表（简单贪心）
        def collect_partial(gold_spans, pred_spans, bucket_pair, bucket_gold_only):
            used_pred = [False]*len(pred_spans)
            for (gs,ge,gt) in gold_spans:
                matched = False
                for j,(ps,pe,pt) in enumerate(pred_spans):
                    if not used_pred[j] and span_overlap((gs,ge), (ps,pe)) > 0:
                        used_pred[j] = True
                        matched = True
                        if normalize_text(gt) != normalize_text(pt):
                            bucket_pair[(gt, pt)] += 1
                        break
                if not matched:
                    # 已在 missed 统计；这里不重复记
                    pass

        collect_partial(gold_time_spans, pred_time_spans,
                        partial_cnt_gold_pred["TIME"], partial_cnt_gold_only["TIME"])
        collect_partial(gold_addr_spans, pred_addr_spans,
                        partial_cnt_gold_pred["ADDRESS"], partial_cnt_gold_only["ADDRESS"])

        # 记录未“完全匹配”的样本（只要任一类型不完全匹配）
        def as_set_norm(arr: List[str]): return set("".join(x.split()) for x in arr)
        if (as_set_norm(pred_time_texts) != as_set_norm(gold_time)) or \
          (as_set_norm(pred_addr_texts) != as_set_norm(gold_addr)):
            mismatches.append({
                "id": sid,
                "sentence": sent,
                "gold": {"TIME": gold_time, "ADDRESS": gold_addr},
                "pred": {"TIME": pred_time_texts, "ADDRESS": pred_addr_texts},
            })

    # --- 9) Report ---

    def show_metrics(title: str, tp: Dict[str,int], fp: Dict[str,int], fn: Dict[str,int]):
        print("\n==", title, "==")
        for k in ["ADDRESS"]:
            if k == "ALL":
                _tp = sum(tp.values()); _fp = sum(fp.values()); _fn = sum(fn.values())
            else:
                _tp, _fp, _fn = tp[k], fp[k], fn[k]
            P, R, F = prf(_tp, _fp, _fn)
            print(f"{k:8s}  P={P:.4f}  R={R:.4f}  F1={F:.4f}  (#tp={_tp}, #fp={_fp}, #fn={_fn})")

    def _safe_div(a, b):
        return a / b if b > 0 else 0.0


    show_metrics("Exact-match (entity text equal)", exact_tp, exact_fp, exact_fn)
    show_metrics("Partial-match (char-overlap>0, greedy)", part_tp, part_fp, part_fn)

    with open(MISMATCH_OUT, "w", encoding="utf-8") as f:
        json.dump(mismatches, f, ensure_ascii=False, indent=2)

    print(f"total:{len(test_data)}, mismatch:{len(mismatches)},acc:{(len(test_data)-len(mismatches))/len(test_data)*100}%")
    print(f"\n[Saved] non-exact matched cases -> {MISMATCH_OUT} (count={len(mismatches)})")

    print("\n== Address-specific Accuracies ==")
    print(f"First-Address Exact Acc   : {_safe_div(first_exact_hits, first_denom):.4f}  ({first_exact_hits}/{first_denom})")
    print(f"First-Address Partial Acc : {_safe_div(first_partial_hits, first_denom):.4f}  ({first_partial_hits}/{first_denom})")
    print(f"First vs Any    Exact Acc : {_safe_div(first_any_exact_hits, first_denom):.4f}  ({first_any_exact_hits}/{first_denom})")
    print(f"First vs Any    Partial Acc: {_safe_div(first_any_partial_hits, first_denom):.4f} ({first_any_partial_hits}/{first_denom})")


# --- 10) Save six error files (sorted by frequency) ---
def counter_to_sorted_list(cnt: Counter, as_pairs=False):
    if as_pairs:
        # cnt: Counter[(gold, pred)]
        items = [ {"gold": g, "pred": p, "count": c}
                  for (g,p), c in cnt.most_common() ]
    else:
        items = [ {"text": t, "count": c} for t, c in cnt.most_common() ]
    return items



# TIME
# with open(ERR_TIME_MISSED, "w", encoding="utf-8") as f:
#     json.dump(counter_to_sorted_list(missed_cnt["TIME"]), f, ensure_ascii=False, indent=2)
# with open(ERR_TIME_SPURIOUS, "w", encoding="utf-8") as f:
#     json.dump(counter_to_sorted_list(spurious_cnt["TIME"]), f, ensure_ascii=False, indent=2)
# with open(ERR_TIME_PARTIAL, "w", encoding="utf-8") as f:
#     json.dump(counter_to_sorted_list(partial_cnt_gold_pred["TIME"], as_pairs=True), f, ensure_ascii=False, indent=2)

# ADDRESS
# # 写入文件
# with open(ERR_ADDR_MISSED, "w", encoding="utf-8") as f:
#     json.dump(counter_to_sorted_list(missed_cnt["ADDRESS"]), f, ensure_ascii=False, indent=2)
# with open(ERR_ADDR_SPURIOUS, "w", encoding="utf-8") as f:
#     json.dump(counter_to_sorted_list(spurious_cnt["ADDRESS"]), f, ensure_ascii=False, indent=2)
# with open(ERR_ADDR_PARTIAL, "w", encoding="utf-8") as f:
#     json.dump(counter_to_sorted_list(partial_cnt_gold_pred["ADDRESS"], as_pairs=True), f, ensure_ascii=False, indent=2)


# print("\n[Saved error buckets]")
# print("  TIME   missed  ->", ERR_TIME_MISSED)
# print("  TIME   partial ->", ERR_TIME_PARTIAL)
# print("  TIME   spurious->", ERR_TIME_SPURIOUS)
# print("  ADDRESS missed  ->", ERR_ADDR_MISSED)
# print("  ADDRESS partial ->", ERR_ADDR_PARTIAL)
# print("  ADDRESS spurious->", ERR_ADDR_SPURIOUS)

if __name__ == "__main__":
    for lang in LANGUAGES:
        test_json = os.path.join(TEST_JSON_DIR, f"{lang}.json")
        if not os.path.isfile(test_json):
            print(f"[SKIP] {test_json} 不存在")
            continue

        mismatch_out = os.path.join(MISMATCH_DIR, f"{lang}_mismatches.json")
        txt_out      = os.path.join(OUTPUT_TXT_DIR, f"{lang}_eval.txt")

        print(f"\n===== Evaluating: {lang} ({test_json}) =====")
        # 把本次评测的所有 print 同时写入 txt
        with io.StringIO() as buf, redirect_stdout(buf):
            _ = evaluate_one_file(test_json, mismatch_out)
            text_log = buf.getvalue()

        with open(txt_out, "w", encoding="utf-8") as f:
            f.write(text_log)

        # 同时也把摘要打印到控制台
        print(text_log)
        print(f"[Saved] TXT -> {txt_out}")
        print(f"[Saved] Mismatches -> {mismatch_out}")



[OK] Loaded tokenizer from: /content/drive/MyDrive/checkpoint-1235 | labels: ['O', 'B-TIME', 'I-TIME', 'B-ADDRESS', 'I-ADDRESS']
[OK] Using tf.lite.Interpreter: /content/drive/MyDrive/model_dy_quant.tflite (deprecated warning is OK)
[IO] Inputs: ['serving_default_attention_mask:0', 'serving_default_input_ids:0']
[IO] Outputs: ['StatefulPartitionedCall:0']
[IO] Mapped index: {'input_ids': 1, 'attention_mask': 0, 'token_type_ids': None}
[IO] Model fixed/padded sequence length: 128

===== Evaluating: 中文 (/content/4/中文.json) =====
Loaded 700 test records from /content/4/中文.json
0

== Exact-match (entity text equal) ==
ADDRESS   P=0.8489  R=0.7607  F1=0.8024  (#tp=2130, #fp=379, #fn=670)

== Partial-match (char-overlap>0, greedy) ==
ADDRESS   P=0.9980  R=0.8943  F1=0.9433  (#tp=2504, #fp=5, #fn=296)
total:700, mismatch:350,acc:50.0%

[Saved] non-exact matched cases -> /content/mismatches_4/中文_mismatches.json (count=350)

== Address-specific Accuracies ==
First-Address Exact Acc   : 0.9257  

**“正确匹配”判定：**

实体级：

* **Exact-match:**
预测出的实体文本要和标注的实体文本完全一致才算正确

* **Partial-match:**
预测出的实体文本和标注的实体文本有重叠就算正确


句子级：

* **first-first:** 模型输出的第一个地址和真实的第一个地址匹配就算正确


* **first-any:** 模型输出的第一个地址和真实地址中的任意一个地址匹配就算正确



---



**指标：**


实体级：

  * **TP：** 预测得到的实体和标注的完全相同

  * **FP：** 预测出标注中没有的实体

  * **FN：** 标注里有但预测里没有的

  * **p（精确率）:** tp/(tp+fp),所有预测值中正确的比例

  * **r（召回率）:** tp/(tp+fn)所有标注的实体中有多少被预测出来

  * **f1:** 2pr/(p+r) p和r的综合指标


句子级：
  * **acc（准确率）：** 识别正确的句子数/所有句子数





---












##### **自定义数据查看tflite模型输出**
用于简单查看模型表现，无后处理，无准确率指标输出

In [ ]:
# 自定义数据查看tflite模型输出

import os, numpy as np, torch
import tensorflow as tf
from transformers import AutoTokenizer
import json, os

# 要测试的句子
samples = [
    "The packages have been delivered to Elmton Court, Woodside Primary School, and Picardy Manorway / Belvedere Station.",
    "明天上午9:30在上海市徐汇区漕溪北路398号开会。",
    "我今天十点去广州塔",
    "下午两点去电影院",
    "我要去影院",
    "2024年6月15日早上8:10，计划去北京天坛公园跑步",
    "导航去公司上班",
    "最近有一家新的咖啡店开在了南京玄武湖，要不要去试试？",
    "这家影院的空调是真猛，看电影到一半突然打了个喷嚏，幸好剧情够精彩。",
    "电影院","公园","牧场","超市","便利店","电影","长隆欢乐世界","深业上城","深圳野生动物园","国际E城","同乐村","乐尚林居",
    "保利五和电影院","重庆解放碑的炉鼎火锅",
    "晚上9:50才下班,回到家发现楼下便利店刚打烊。",
    "2024年6月10日早上7:50在南京玄武湖跑圈,跑完一肚子饿",
    "山姆会员店",
    "Late afternoon on 圣诞夜",
    "今天上午去南山图书馆，下午去山姆会员店",
    "3:00",
    "导航去万象天地",
    "活动一：next Wednesday，活动二：今天, 活动三：九点半",
    "下午去深圳世界之窗和妹妹玩",
    "亚马逊州",
    "小谷围镇穗石村的美食好好吃",
    "4月30号，也就是星期三，来和我见面",
    "明天上午9:30在上海市徐汇区漕溪北路398号开会。",
    "Meet me at 221B Baker Street at 7 pm tomorrow.",
    "Vi mødes kl. 1979-W07-6 ved DK-6000 Kolding, Østergade 16.",
    "6月5日（月曜日）に新潟県新潟市中央区万代3丁目7-8で集合するのはいかがですか？",
    "Perşembe 22 Ağustos 2024 saatinde bir kahve molası verelim.",
    "The party kicks off at October 12th, 1986 at Vicolo della Cancelleria 6, 00186 Roma RM, Italia.",
    "下周三上午九点提醒我去保利万和国际影城看电源",
    "【活动提醒】6月18日10:00在杭州西湖音乐喷泉集合拍照，下午换到柳浪闻莺野餐，不要迟到。",
    "据交通运输部通告，假期返程高峰预计出现在最后两天。",
    "上午8:50到桂林象鼻山，10:45爬到象眼平台，下午3:35沿江散步回酒店。",
    "我要去电影院50米外的小摊"
]



TFLITE_PATH = "/content/drive/MyDrive/model_dy_quant.tflite"   # 模型路径
# 训练时保存的 tokenizer 目录（若没有则回退到基座）
TOKENIZER_DIRS = ["/content/drive/MyDrive/checkpoint-1235", "xlm-roberta-base"] # tokenizer路径（也可不设置）

# 固定映射逻辑，请勿修改
# O: 不是时间也不是地址的token
# B-TIME:一个时间token串的第一个token
# I-TIME：一个时间token串的非开头的部分
# B-ADDRESS:一个地址token串的第一个token
# I-ADDRESS：一个地址token串的非开头的部分
LABELS = ["O","B-TIME","I-TIME","B-ADDRESS","I-ADDRESS"]
ID2LABEL = {i:l for i,l in enumerate(LABELS)}

# tokenizer
tok = None
for p in TOKENIZER_DIRS:
    try:
        tok = AutoTokenizer.from_pretrained(p, use_fast=True)
        print(f"Loaded tokenizer from: {p}")
        break
    except Exception:
        pass
assert tok is not None, "Tokenizer not found."

print(f"tokenizer model: {tok.backend_tokenizer.model}")

# Interpreter（优先新版 LiteRT，不存在则用 tf.lite）
try:
    from ai_edge_litert.python.interpreter import Interpreter  # type: ignore
    interpreter = Interpreter(model_path=TFLITE_PATH)
    print(f"Using ai_edge_litert.Interpreter:{TFLITE_PATH}")

except Exception:
    interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
    print(f"Using tf.lite.Interpreter:{TFLITE_PATH} (deprecated warning is OK)")

interpreter.allocate_tensors()


def load_id2label_from_config(model_dir: str):
    # 既可以读 checkpoint-1235 下的 config.json，也可以读 SavedModel 导出目录旁边的 config.json
    cfg_path = os.path.join(model_dir, "config.json")
    with open(cfg_path, "r", encoding="utf-8") as f:
        cfg = json.load(f)
    id2label = {int(k): v for k, v in cfg["id2label"].items()}
    label2id = {v: int(k) for k, v in id2label.items()}
    labels_order = [id2label[i] for i in range(len(id2label))]
    return id2label, label2id, labels_order



# 解析输入输出张量信息（自动匹配名称）
def _io_info(interp):
    inps = interp.get_input_details()
    outs = interp.get_output_details()

    # 记录索引
    idx = {"input_ids": None, "attention_mask": None, "token_type_ids": None}
    for i, info in enumerate(inps):
        name = info["name"].lower()
        if "input_ids" in name or "ids" in name:
            if idx["input_ids"] is None: idx["input_ids"] = i
        elif "attention_mask" in name or "mask" in name:
            idx["attention_mask"] = i
        elif "token_type_ids" in name or "segment" in name:
            idx["token_type_ids"] = i
    # 若只有一个输入，则当作 input_ids；若两个，多半是 ids+mask
    if len(inps) == 1 and idx["input_ids"] is None:
        idx["input_ids"] = 0
    if len(inps) == 2 and idx["attention_mask"] is None:
        other = 1 if idx["input_ids"] == 0 else 0
        idx["attention_mask"] = other
    return inps, outs, idx

inps, outs, IDX = _io_info(interpreter)
print("Inputs:", [d["name"] for d in inps])
print("Outputs:", [d["name"] for d in outs])
print("Mapped index:", IDX)

# 组装推理函数（动态长度 resize）
def infer(text, fallback_len=128):
    # 读取模型输入的固定长度（[1, 128]）
    inps_local = interpreter.get_input_details()
    # 通过 input_ids 这个输入拿到固定长度；如果拿不到就用 fallback
    fixed_len = None
    if IDX["input_ids"] is not None:
        shape = inps_local[IDX["input_ids"]]["shape"]
        if len(shape) == 2 and shape[1] > 0:
            fixed_len = int(shape[1])
    if fixed_len is None:
        fixed_len = int(fallback_len)

    # 关键：在 tokenizer 侧做 padding='max_length' + truncation 到 fixed_len
    enc = tok(
        text,
        return_tensors="np",
        padding="max_length",
        truncation=True,
        max_length=fixed_len,
    )
    print("PY ids:", enc["input_ids"][0][:64].tolist())
    print("PY mask:", enc["attention_mask"][0][:64].tolist())
    print("tokens:", tok.convert_ids_to_tokens(enc["input_ids"][0])[:32])

    input_ids = enc["input_ids"].astype(np.int32)          # [1, fixed_len]
    attn      = enc["attention_mask"].astype(np.int32)     # [1, fixed_len]
    ttids     = enc.get("token_type_ids", np.zeros_like(input_ids, dtype=np.int32))

    # 不要 resize，直接 allocate + set_tensor
    interpreter.allocate_tensors()
    if IDX["input_ids"] is not None:
        interpreter.set_tensor(inps_local[IDX["input_ids"]]["index"], input_ids)
    if IDX["attention_mask"] is not None:
        interpreter.set_tensor(inps_local[IDX["attention_mask"]]["index"], attn)
    if IDX["token_type_ids"] is not None:
        interpreter.set_tensor(inps_local[IDX["token_type_ids"]]["index"], ttids)

    interpreter.invoke()
    logits = interpreter.get_tensor(interpreter.get_output_details()[0]["index"])
    return enc, logits


# 将 logits -> BIO 标签，并合并子词为实体片段
def decode_entities(enc, logits):
    ids = enc["input_ids"][0].tolist()
    toks = tok.convert_ids_to_tokens(ids)
    pred_ids = logits[0].argmax(-1).tolist()
    labels = [ID2LABEL[i] for i in pred_ids]

    # 去掉特殊符号（<s>, </s>, <pad>）
    clean = []
    for t, y in zip(toks, labels):
        if t in (tok.cls_token, tok.sep_token, tok.pad_token, "<s>", "</s>"):
            continue
        clean.append((t, y))

    # 合并为实体
    entities, cur_type, cur_text = [], None, ""
    for t, y in clean:
        if t == "▁":
            continue

        surf = t.replace("▁", " ").strip()

        if y.startswith("B-"):
            if cur_type:
                entities.append((cur_type, cur_text.strip()))
            cur_type, cur_text = y[2:], surf
        elif y.startswith("I-") and cur_type == y[2:]:
            cur_text += surf
        else:
            if cur_type:
                entities.append((cur_type, cur_text.strip()))
            cur_type, cur_text = None, ""
    if cur_type:
        entities.append((cur_type, cur_text.strip()))

    return [t for t,_ in clean], [y for _,y in clean], entities



# 跑并打印
for i, s in enumerate(samples, 1):
    enc, logits = infer(s)      # 固定128
    toks, labs, ents = decode_entities(enc, logits)
    print("="*70)
    print(f"[{i}] Text:", s)
    print("TOK:", " ".join(toks[:120]))
    print("PRD:", " ".join(labs[:120]))
    print("ENTS:", ents)



Loaded tokenizer from: /content/drive/MyDrive/checkpoint-1235
tokenizer model: Unigram(unk_id=3, vocab=[("<s>", 0), ("<pad>", 0), ("</s>", 0), ("<unk>", 0), (",", -3.4635426998138428), ...], byte_fallback=False)
Using tf.lite.Interpreter:/content/drive/MyDrive/model_dy_quant.tflite (deprecated warning is OK)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Inputs: ['serving_default_attention_mask:0', 'serving_default_input_ids:0']
Outputs: ['StatefulPartitionedCall:0']
Mapped index: {'input_ids': 1, 'attention_mask': 0, 'token_type_ids': None}
PY ids: [0, 581, 98169, 7, 765, 2809, 75060, 297, 47, 540, 39, 1507, 52341, 4, 39076, 8752, 19012, 1294, 19188, 4, 136, 27860, 147, 1459, 1572, 748, 7514, 248, 12628, 187796, 38488, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
PY mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tokens: ['<s>', '▁The', '▁package', 's', '▁have', '▁been', '▁deliver', 'ed', '▁to', '▁El', 'm', 'ton', '▁Court', ',', '▁Wood', 'side', '▁Prima', 'ry', '▁School', ',', '▁and', '▁Pic', 'ar', 'dy', '▁Man', 'or', 'way', '▁/', '▁Bel', 'vedere', '▁Station', '.']
[1] Text: The packages have been delivered to Elmton Court, Woodside 

### 其他代码

检查训练数据集

In [ ]:
import random

# 数据文件路径
DATA_PATH = "/content/drive/MyDrive/NER/train_datasets_AC/train_葡萄牙语.conll"

# 标签定义
LABELS = ["O", "B-ADDRESS", "I-ADDRESS", "B-TIME", "I-TIME"]

def read_conll(path):
    """读取 CoNLL 文件 -> [(tokens, labels), ...]"""
    sents = []
    toks, labs = [], []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if toks:
                    sents.append((toks, labs))
                    toks, labs = [], []
                continue
            if " " in line:
                t, y = line.split(" ", 1)
            else:
                t, y = line, "O"
            if y not in LABELS:
                y = "O"
            toks.append(t)
            labs.append(y)
    if toks:
        sents.append((toks, labs))
    return sents


def extract_entities(tokens, labels):
    """从 tokens + BIO labels 中抽取 time/address 实体"""
    entities = {"TIME": [], "ADDRESS": []}
    cur_type, cur_tokens = None, []

    for t, y in zip(tokens, labels):
        if y == "O":
            if cur_type:
                entities[cur_type].append("".join(cur_tokens))
                cur_type, cur_tokens = None, []
            continue

        prefix, etype = y.split("-", 1)
        if prefix == "B":
            if cur_type:
                entities[cur_type].append("".join(cur_tokens))
            cur_type, cur_tokens = etype, [t]
        elif prefix == "I" and cur_type == etype:
            cur_tokens.append(t)
        else:
            if cur_type:
                entities[cur_type].append("".join(cur_tokens))
            cur_type, cur_tokens = etype, [t]

    if cur_type:
        entities[cur_type].append("".join(cur_tokens))

    return entities


# 读取训练集
sents = read_conll(DATA_PATH)
print(f"Loaded {len(sents)} sentences from {DATA_PATH}")

# 随机抽取 100 条
samples = random.sample(sents, 100)

for i, (tokens, labels) in enumerate(samples, 1):
    ents = extract_entities(tokens, labels)
    print("="*60)
    print(f"[{i}] TOK:", " ".join(tokens))
    print("LBL:", " ".join(labels))
    print("TIME:", ents["TIME"])
    print("ADDR:", ents["ADDRESS"])


Loaded 1000 sentences from /content/drive/MyDrive/NER/train_datasets_AC/train_葡萄牙语.conll
[1] TOK: ▁25 ▁de ▁agosto ▁às ▁22 h 15 ▁visite i ▁o ▁Está dio ▁do ▁Campo ▁Futebol ▁Club e ▁para ▁comprar ▁ingredientes ▁fresco s .
LBL: B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME O O O B-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS I-ADDRESS O O O O O O
TIME: ['▁25▁de▁agosto▁às▁22h15']
ADDR: ['▁Estádio▁do▁Campo▁Futebol▁Clube']
[2] TOK: ▁Che gamos ▁a ▁Edi fíci o ▁29 ▁2 ▁de ▁mai . ▁de ▁2021 , ▁explora mos ▁os ▁arredor es , ▁e ▁depois ▁seguimos ▁para ▁22 # ▁No ite ▁de ▁31 ▁de ▁outubro , ▁volta ndo ▁para ▁casa ▁18 ▁de ▁janeiro ▁de ▁20 34 ▁às ▁06 :36 .
LBL: O O O O O O O B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME O O O O O O O O O O O O O B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME O O O O O B-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME I-TIME O
TIME: ['▁2▁de▁mai.▁de▁2021', '▁Noite▁de▁31▁de▁outubro', '▁18▁de▁janeiro▁de▁2034▁às▁06:36']
ADDR: []
[3] TOK: ▁10 ▁de ▁outubro ▁de ▁2023

In [ ]:
%pip install -q evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00


In [ ]:

from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig, TFAutoModelForTokenClassification
import numpy as np, tensorflow as tf, torch, os, json

CHECKPOINT_DIR = "/content/drive/MyDrive/checkpoint-1235"
SAVEDMODEL_DIR = "/content/export_xlmr_tf/saved_model"
TFLITE_PATH    = "/content/model_int8_full.tflite"


def load_id2label_from_config(model_dir: str):
    # 既可以读 checkpoint-1235 下的 config.json，也可以读 SavedModel 导出目录旁边的 config.json
    cfg_path = os.path.join(model_dir, "config.json")
    with open(cfg_path, "r", encoding="utf-8") as f:
        cfg = json.load(f)
    id2label = {int(k): v for k, v in cfg["id2label"].items()}
    label2id = {v: int(k) for k, v in id2label.items()}
    labels_order = [id2label[i] for i in range(len(id2label))]
    return id2label, label2id, labels_order


# 读取标签映射（一定用config.json）
id2label, label2id, labels_order = load_id2label_from_config(CHECKPOINT_DIR)

tok = AutoTokenizer.from_pretrained(CHECKPOINT_DIR, use_fast=True)

# 1) PyTorch
cfg = AutoConfig.from_pretrained(CHECKPOINT_DIR)
pt_model = AutoModelForTokenClassification.from_pretrained(CHECKPOINT_DIR, config=cfg).eval()

# 2) TF SavedModel
tf_model = TFAutoModelForTokenClassification.from_pretrained(CHECKPOINT_DIR, from_pt=True)
@tf.function
def tf_infer(input_ids, attention_mask):
    return tf_model(input_ids=input_ids, attention_mask=attention_mask, training=False).logits

# 3) TFLite
interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
interpreter.allocate_tensors()
inps = interpreter.get_input_details(); outs = interpreter.get_output_details()
# 映射 input_ids / attention_mask
name2idx = {d["name"]: i for i,d in enumerate(inps)}
idx_ids = [i for i,d in enumerate(inps) if "input_ids" in d["name"]][0]
idx_mask = [i for i,d in enumerate(inps) if "attention_mask" in d["name"]][0]
FIXED_LEN = inps[idx_ids]["shape"][1]

def run_all(text):
    enc = tok(text, return_tensors="np", padding="max_length", truncation=True, max_length=FIXED_LEN)
    # 1) PyTorch
    with torch.no_grad():
        ipt = {k: torch.tensor(v) for k,v in enc.items()}
        logits_pt = pt_model(**ipt).logits[0].cpu().numpy()

    # 2) TF
    logits_tf = tf_infer(enc["input_ids"], enc["attention_mask"]).numpy()[0]

    # 3) TFLite
    interpreter.set_tensor(inps[idx_ids]["index"], enc["input_ids"].astype(np.int32))
    interpreter.set_tensor(inps[idx_mask]["index"], enc["attention_mask"].astype(np.int32))
    interpreter.invoke()
    logits_tfl = interpreter.get_tensor(outs[0]["index"])[0]

    return logits_pt, logits_tf, logits_tfl, enc

def decode_logits(logits, enc):
    # 按最后一维 argmax
    pred_ids = logits.argmax(-1)  # [seq]
    pred_labs = [labels_order[i] for i in pred_ids.tolist()]
    toks = tok.convert_ids_to_tokens(enc["input_ids"][0].tolist())
    return toks, pred_labs

# 跑几条检查三路是否一致
texts = [
    "明天上午9:30在上海市徐汇区漕溪北路398号开会。",
    "Meet me at 221B Baker Street at 7 pm tomorrow.",
    "Vi mødes kl. 1979-W07-6 ved DK-6000 Kolding, Østergade 16."
]
for s in texts:
    l_pt, l_tf, l_tfl, enc = run_all(s)
    t1, y1 = decode_logits(l_pt, enc)
    t2, y2 = decode_logits(l_tf, enc)
    t3, y3 = decode_logits(l_tfl, enc)
    print("\nTEXT:", s)
    print("PT  :", " ".join(y1[:50]))
    print("TF  :", " ".join(y2[:50]))
    print("TFL :", " ".join(y3[:50]))



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/checkpoint-1235/config.json'

In [ ]:
# 训练：XLM-RoBERTa NER (DATE/ADDRESS)
import sys, subprocess, os

# 依赖
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-U",
                       "transformers>=4.55.0", "datasets>=2.14.0",
                       "evaluate>=0.4.0", "seqeval>=1.2.2", "accelerate>=0.21.0"])

import torch, numpy as np
from dataclasses import dataclass

# 配置
@dataclass
class CFG:
    data_dir: str = "/content/data"
    model_name: str = "xlm-roberta-base"
    output_dir: str = "/content/outputs/xlmr-ner-date-address"
    max_len: int = 128
    epochs: int = 10
    lr: float = 5e-5
    batch_size: int = 128
    seed: int = 42
    fp16: bool = True
    grad_ckpt: bool = True
    warmup_ratio: float = 0.1
    label_smoothing: float = 0.05
    patience: int = 3
    eval_steps: int = 200

CFG = CFG()
os.makedirs(CFG.output_dir, exist_ok=True)
os.makedirs(CFG.data_dir, exist_ok=True)

# 标签
LABEL_LIST = ["O", "B-DATE", "I-DATE", "B-ADDRESS", "I-ADDRESS"]
LABEL2ID = {l: i for i, l in enumerate(LABEL_LIST)}
ID2LABEL = {i: l for l, i in LABEL2ID.items()}
ALLOWED = set(LABEL_LIST)

# 4) demo 数据（若已放好 train/dev/test 就不会覆盖）
train_fp = os.path.join(CFG.data_dir, "train.conll")
dev_fp   = os.path.join(CFG.data_dir, "dev.conll")
test_fp  = os.path.join(CFG.data_dir, "test.conll")

def _write_demo(path: str):
    demo = """明天 O
      上午 O
      9:30 B-DATE
      在 O
      上海市 B-ADDRESS
      徐汇区 I-ADDRESS
      漕溪北路 I-ADDRESS
      398号 I-ADDRESS
      开会 O

      Meet O
      me O
      at O
      221B B-ADDRESS
      Baker I-ADDRESS
      Street I-ADDRESS
      at O
      7 B-DATE
      pm I-DATE
      tomorrow I-DATE
      . O
      """.strip()+"\n"
    with open(path, "w", encoding="utf-8") as f:
        f.write(demo)

for p in [train_fp, dev_fp]:
    if not os.path.isfile(p):
        _write_demo(p)
if not os.path.isfile(test_fp):
    with open(test_fp, "w", encoding="utf-8") as f_out, open(dev_fp, "r", encoding="utf-8") as f_in:
        f_out.write(f_in.read())

# 读 CoNLL -> HF datasets
from typing import List, Tuple
def read_conll(path: str) -> Tuple[List[List[str]], List[List[str]]]:
    tokens_list, tags_list, tokens, tags = [], [], [], []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    tokens_list.append(tokens); tags_list.append(tags)
                    tokens, tags = [], []
                continue
            parts = line.split()
            tok, tag = (parts[0], parts[-1]) if len(parts) > 1 else (parts[0], "O")
            if tag not in ALLOWED: tag = "O"
            tokens.append(tok); tags.append(tag)
    if tokens:
        tokens_list.append(tokens); tags_list.append(tags)
    return tokens_list, tags_list

from datasets import Dataset, DatasetDict
def build_dataset_from_conll(train_path: str, dev_path: str, test_path: str) -> DatasetDict:
    def to_records(tokens_list, tags_list):
        for toks, tags in zip(tokens_list, tags_list):
            yield {"tokens": toks, "ner_tags": tags}
    tr_tokens, tr_tags = read_conll(train_fp)
    dv_tokens, dv_tags = read_conll(dev_fp)
    te_tokens, te_tags = read_conll(test_fp)
    return DatasetDict(
        train=Dataset.from_list(list(to_records(tr_tokens, tr_tags))),
        validation=Dataset.from_list(list(to_records(dv_tokens, dv_tags))),
        test=Dataset.from_list(list(to_records(te_tokens, te_tags))),
    )
ds = build_dataset_from_conll(train_fp, dev_fp, test_fp)

# 分词与标签对齐
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def tokenize_and_align_labels(examples, max_length=CFG.max_len):
    tokenized = tokenizer(
        examples["tokens"], is_split_into_words=True,
        truncation=True, padding=False, max_length=max_length,
    )
    aligned_labels = []
    for i, labels in enumerate(examples["ner_tags"]):
        word_ids = tokenized.word_ids(i)
        prev_word_id = None
        label_ids = []
        for wid in word_ids:
            if wid is None:
                label_ids.append(-100)
            elif wid != prev_word_id:
                label_ids.append(LABEL2ID[labels[wid]])
            else:
                lab = labels[wid]
                if lab.startswith("B-"): lab = "I-" + lab[2:]
                label_ids.append(LABEL2ID[lab])
            prev_word_id = wid
        aligned_labels.append(label_ids)
    tokenized["labels"] = aligned_labels
    return tokenized

ds_tok = ds.map(tokenize_and_align_labels, batched=True, desc="Tokenizing & aligning labels")

#  模型 & 训练器
from transformers import (AutoConfig, AutoModelForTokenClassification,
                          DataCollatorForTokenClassification, Trainer, TrainingArguments,
                          EarlyStoppingCallback, set_seed)
import evaluate
set_seed(CFG.seed)

config = AutoConfig.from_pretrained(
    CFG.model_name, num_labels=len(LABEL_LIST),
    id2label=ID2LABEL, label2id=LABEL2ID,
)
model = AutoModelForTokenClassification.from_pretrained(CFG.model_name, config=config)
if CFG.grad_ckpt:
    model.gradient_checkpointing_enable()

data_collator = DataCollatorForTokenClassification(tokenizer)

seqeval = evaluate.load("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=-1)
    true_predictions, true_labels = [], []
    for pred, lab in zip(predictions, labels):
        cur_pred, cur_lab = [], []
        for p_i, l_i in zip(pred, lab):
            if l_i == -100: continue
            cur_pred.append(ID2LABEL[p_i]); cur_lab.append(ID2LABEL[l_i])
        true_predictions.append(cur_pred); true_labels.append(cur_lab)
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


training_args = TrainingArguments(
    output_dir=CFG.output_dir,
    save_strategy="steps",
    save_steps=CFG.eval_steps,
    save_total_limit=2,
    load_best_model_at_end=True, metric_for_best_model="f1",
    greater_is_better=True,

    eval_strategy="steps",
    eval_steps=CFG.eval_steps,

    learning_rate=CFG.lr, per_device_train_batch_size=CFG.batch_size,
    per_device_eval_batch_size=CFG.batch_size, num_train_epochs=CFG.epochs,
    warmup_ratio=CFG.warmup_ratio, weight_decay=0.01,
    fp16=CFG.fp16 and torch.cuda.is_available(),
    logging_steps=50, report_to="none",
    label_smoothing_factor=CFG.
    label_smoothing,
    gradient_accumulation_steps=1,
    dataloader_num_workers=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["validation"],
    tokenizer=tokenizer, data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=CFG.patience)],
)


# 训练 / 评测 / 保存
trainer.train()
print("== Dev metrics ==", trainer.evaluate(ds_tok["validation"]))
if "test" in ds_tok and len(ds_tok["test"]) > 0:
    print("== Test metrics ==", trainer.evaluate(ds_tok["test"]))
trainer.save_model(CFG.output_dir); tokenizer.save_pretrained(CFG.output_dir)
print(f"模型已保存到：{CFG.output_dir}")

# 推理
def predict_spans(text: str, max_length: int = CFG.max_len):
    model.eval()
    with torch.no_grad():
        enc = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
        if torch.cuda.is_available():
            model.cuda(); enc = {k: v.cuda() for k, v in enc.items()}
        logits = model(**enc).logits[0].cpu().numpy()
    pred_ids = logits.argmax(-1)
    pred_labels = [ID2LABEL[i] for i in pred_ids]
    toks = tokenizer.convert_ids_to_tokens(enc["input_ids"][0].cpu().tolist())
    spans, cur_type, cur_text = [], None, ""
    for tok, lab in zip(toks, pred_labels):
        if tok in (tokenizer.cls_token, tokenizer.sep_token, "<s>", "</s>", "<pad>"): continue
        surface = tok.replace("▁", " ").strip()
        if lab.startswith("B-"):
            if cur_type: spans.append((cur_type, cur_text.strip()))
            cur_type, cur_text = lab[2:], surface
        elif lab.startswith("I-") and cur_type == lab[2:]:
            cur_text += surface
        else:
            if cur_type: spans.append((cur_type, cur_text.strip()))
            cur_type, cur_text = None, ""
    if cur_type: spans.append((cur_type, cur_text.strip()))
    return spans

print("Demo spans:", predict_spans("明天上午9:30在上海市徐汇区漕溪北路398号开会。Meet me at 221B Baker Street at 7 pm tomorrow."))


In [ ]:
# ===== 推理：读取最新的 checkpoint (ner_xlmr_out 根目录) =====
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import numpy as np

# 路径配置
MODEL_DIR = "/content/ner_xlmr_out"
DATA_PATH = "/content/train_all.conll"

# 标签映射（需和训练时一致）
LABELS = ["O", "B-TIME", "I-TIME", "B-ADDRESS", "I-ADDRESS"]
label2id = {l:i for i,l in enumerate(LABELS)}
id2label = {i:l for l,i in label2id.items()}

# 加载模型和分词器（最新的）
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, use_fast=True)
model = AutoModelForTokenClassification.from_pretrained(MODEL_DIR)

#  读取验证集
def read_conll(path: str):
    sents_tok, sents_lab = [], []
    toks, labs = [], []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line:
                if toks:
                    sents_tok.append(toks); sents_lab.append(labs)
                    toks, labs = [], []
                continue
            if " " in line:
                t, y = line.split(" ", 1)
            else:
                t, y = line, "O"
            if y not in label2id: y = "O"
            toks.append(t); labs.append(y)
    if toks:
        sents_tok.append(toks); sents_lab.append(labs)
    return sents_tok, sents_lab

tokens_all, tags_all = read_conll(DATA_PATH)

# 随机取 10 条验证样例
import random
random.seed(42)
sample_idx = random.sample(range(len(tokens_all)), 10)

#  推理
model.eval()
for idx in sample_idx:
    toks = tokens_all[idx]
    labs = tags_all[idx]

    enc = tokenizer(toks, is_split_into_words=True, return_tensors="pt", truncation=True, max_length=256)
    with torch.no_grad():
        logits = model(**enc).logits
    preds = logits.argmax(-1).squeeze().tolist()

    # 对齐 labels（忽略 special tokens）
    word_ids = enc.word_ids()
    pred_labels = []
    for p, wid in zip(preds, word_ids):
        if wid is None: continue
        pred_labels.append(id2label[p])

    print("="*40)
    print("原始 Tokens:", " ".join(toks))
    print("真实 Labels:", " ".join(labs))
    print("预测 Labels:", " ".join(pred_labels))


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

templates = [
    "请在{time}到{address}开会。",
    "We will meet at {address} on {time}.",
]

times = ["明天上午9:30", "2025年8月24日15:00", "7 pm tomorrow"]
addresses = ["上海市-徐汇区-漕溪北路-398号", "221B Baker Street", "北京市东城区景山前街4号"]

def make_sample(template, time, address):
    sent = template.format(time=time, address=address)

    # span 边界
    time_start = sent.index(time)
    time_end   = time_start + len(time)
    addr_start = sent.index(address)
    addr_end   = addr_start + len(address)

    # 分词 + offset
    enc = tokenizer(sent, return_offsets_mapping=True, add_special_tokens=True)

    tokens = enc.tokens()
    print(tokens)
    offsets = enc["offset_mapping"]
    # 兼容单条/批量两种返回形状
    if isinstance(offsets[0], tuple):      # 单条：[(s,e), ...]
        offsets_seq = offsets
    else:                                  # 批量：[[(s,e), ...]]
      offsets_seq = offsets[0]

    def in_span(s, e, a, b):
      # 与 [a,b) 是否有交集（更稳，避免严格包含带来的漏标）
      return max(0, min(e, b) - max(s, a)) > 0

    labels = []
    for tok, (s, e) in zip(tokens, offsets_seq):
      if s == e:                # special tokens（CLS/SEP等）offset 常为 (0,0)
          labels.append(-100)
          continue

      if in_span(s, e, time_start, time_end):
          # 只要这个 token 覆盖了 time 的起点，就打 B，否则 I
          prefix = "B-" if (s <= time_start < e) else "I-"
          labels.append(prefix + "DATE")
      elif in_span(s, e, addr_start, addr_end):
          prefix = "B-" if (s <= addr_start < e) else "I-"
          labels.append(prefix + "ADDRESS")
      else:
          labels.append("O")

    return sent, labels

print(make_sample(templates[0], times[1], addresses[0]))



### **文件操作**

##### 清空文件夹(**左侧文件栏中非drive文件夹里的文件删除后无法找回，请谨慎删除！**)

In [ ]:
import os
import shutil

dirs_to_clear = ["/content/drive/MyDrive/NER/current_best"] # 想清空的文件夹路径

for dir_path in dirs_to_clear:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print(f"Cleared directory: {dir_path}")
    os.makedirs(dir_path, exist_ok=True)
    print(f"Recreated directory: {dir_path}")

Cleared directory: /content/drive/MyDrive/NER/current_best
Recreated directory: /content/drive/MyDrive/NER/current_best


##### 将指定文件**移动**（非复制）到谷歌云盘中

In [ ]:
import os
import shutil

source_dirs = ["/content/ner_xlmr_out/training_args.bin"] # 想移动的文件原始路径
destination_dir = "/content/drive/MyDrive/NER/CheckPoints/bestCheck"  # 目标路径（谷歌云盘）

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

for src_dir in source_dirs:
    if os.path.exists(src_dir):
        try:
            shutil.move(src_dir, destination_dir)
            print(f"Moved '{src_dir}' to '{destination_dir}'")
        except FileNotFoundError:
            print(f"Error: Source directory '{src_dir}' not found.")
        except Exception as e:
            print(f"Error moving '{src_dir}': {e}")
    else:
        print(f"Source directory '{src_dir}' does not exist.")

Moved '/content/ner_xlmr_out/training_args.bin' to '/content/drive/MyDrive/NER/CheckPoints/bestCheck'


##### 文件改名

In [ ]:
import os

# 文件夹路径
folder_path = r"/content/drive/MyDrive/NER/real_address"   # 修改为你的文件夹路径

# 缩写和语言名的映射关系
LANG_MAP = {
    "zh": "中文",
    "en": "英语",
    "da": "丹麦语",
    "ru": "俄语",
    "tr": "土耳其语",
    "de": "德语",
    "it": "意大利语",
    "ja": "日语",
    "fr": "法语",
    "sv": "瑞典语",
    "nl": "荷兰语",
    "pt": "葡萄牙语",
    "es": "西班牙语",
    "ko": "韩语",
}

def rename_files(folder):
    for filename in os.listdir(folder):
        old_path = os.path.join(folder, filename)

        # 跳过文件夹
        if not os.path.isfile(old_path):
            continue

        # 遍历缩写映射
        for abbr, lang in LANG_MAP.items():
            if filename.startswith(abbr):  # 文件名前缀是缩写
                new_name = filename.replace(abbr, lang, 1)
                new_path = os.path.join(folder, new_name)
                os.rename(old_path, new_path)
                print(f"重命名: {filename} -> {new_name}")
                break

if __name__ == "__main__":
    rename_files(folder_path)


重命名: en.txt -> 英语.txt
重命名: da.txt -> 丹麦语.txt
重命名: ru.txt -> 俄语.txt
重命名: tr.txt -> 土耳其语.txt
重命名: de.txt -> 德语.txt
重命名: it.txt -> 意大利语.txt
重命名: ja.txt -> 日语.txt
重命名: fr.txt -> 法语.txt
重命名: sv.txt -> 瑞典语.txt
重命名: nl.txt -> 荷兰语.txt
重命名: pt.txt -> 葡萄牙语.txt
重命名: es.txt -> 西班牙语.txt
重命名: ko.txt -> 韩语.txt
重命名: zh.txt -> 中文.txt


##### 简体转繁体

In [ ]:
!pip install opencc-python-reimplemented

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 8.4 MB/s eta 0:00:00


In [ ]:
# 安装依赖：
# pip install opencc-python-reimplemented

import opencc

def convert_simplified_to_traditional(input_file, output_file):
    # 创建转换器（简体到繁体）
    converter = opencc.OpenCC('s2t')

    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()

    # 转换文本
    converted_text = converter.convert(text)

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(converted_text)

    print(f"转换完成：{input_file} -> {output_file}")

if __name__ == "__main__":
    # 修改为你的文件路径
    input_path = "/content/drive/MyDrive/NER/real_time/简体中文.txt"
    output_path = "/content/drive/MyDrive/NER/real_time/繁体中文.txt"
    convert_simplified_to_traditional(input_path, output_path)


转换完成：/content/drive/MyDrive/NER/negatives/zh/normal.txt -> /content/drive/MyDrive/NER/negatives/tw/normal.txt
